In [20]:
# 1 - Install des dependances necessaires : 
%pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [2]:
# 2 - Chargements des modules 
import os
from pathlib import Path
from typing import Optional, Union, Dict
import json
import boto3
import io
from urllib.parse import urlparse
from IPython.display import Markdown, display
from datetime import datetime

In [3]:


def _is_s3_uri(path: Union[str, Path]) -> bool:
    return str(path).lower().startswith("s3://")

def _split_s3_uri(s3_uri: str):
    # s3://bucket/key/with/any chars
    parsed = urlparse(s3_uri)
    bucket = parsed.netloc
    key = parsed.path.lstrip("/")
    if not bucket or not key:
        raise ValueError(f"URI S3 invalide: {s3_uri}")
    return bucket, key

def _read_s3_bytes(s3_uri: str) -> bytes:
    s3 = boto3.client("s3")
    bucket, key = _split_s3_uri(s3_uri)
    obj = s3.get_object(Bucket=bucket, Key=key)
    return obj["Body"].read()

def extract_text_from_any_file(file_path: Union[str, Path]) -> Optional[str]:
    """
    Extrait le texte de différents types de fichiers (local OU S3).
    - Texte: txt, md, log, csv, json, xml, html, py, js, java, cpp
    - PDF: PyPDF2
    - DOCX: python-docx
    - Excel: pandas (xlsx/xls)
    - HTML: BeautifulSoup
    - Images: pytesseract (si tesseract installé) — sinon voir AWS Textract ci-dessous
    """
    file_path = str(file_path)
    is_s3 = _is_s3_uri(file_path)
    extension = Path(urlparse(file_path).path if is_s3 else file_path).suffix.lower()

    # Charge le contenu en mémoire (bytes ou texte) selon la source
    raw_bytes = None
    raw_text = None

    try:
        # ---- Lecture source (S3 ou local) ----
        if is_s3:
            raw_bytes = _read_s3_bytes(file_path)
        else:
            p = Path(file_path)
            if not p.exists():
                print(f"Erreur: Le fichier {file_path} n'existe pas")
                return None
            # Pour les formats texte simples on peut lire directement en str
            if extension in ['.txt', '.md', '.log', '.csv', '.json', '.xml', '.html', '.htm', '.py', '.js', '.java', '.cpp']:
                raw_text = p.read_text(encoding='utf-8', errors='ignore')
            else:
                raw_bytes = p.read_bytes()

        # ---- Dispatch par extension ----

        # Fichiers texte simples
        if extension in ['.txt', '.md', '.log', '.py', '.js', '.java', '.cpp']:
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            return raw_text

        # CSV
        elif extension == '.csv':
            # si juste “extraire du texte”, on renvoie tel quel
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            return raw_text

        # JSON
        elif extension == '.json':
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            # jolisage optionnel
            try:
                return json.dumps(json.loads(raw_text), ensure_ascii=False, indent=2)
            except Exception:
                return raw_text

        # XML
        elif extension == '.xml':
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            return raw_text

        # HTML
        elif extension in ['.html', '.htm']:
            from bs4 import BeautifulSoup
            if raw_text is None:
                raw_text = raw_bytes.decode('utf-8', errors='ignore')
            soup = BeautifulSoup(raw_text, 'html.parser')
            return soup.get_text(separator='\n', strip=True)

        # PDF
        elif extension == '.pdf':
            import PyPDF2
            f = io.BytesIO(raw_bytes)
            reader = PyPDF2.PdfReader(f)
            pages = []
            for page in reader.pages:
                pages.append(page.extract_text() or "")
            return "\n".join(pages).strip()

        # Word (.docx)
        elif extension == '.docx':
            from docx import Document
            f = io.BytesIO(raw_bytes)
            doc = Document(f)
            return "\n".join(para.text for para in doc.paragraphs)

        # Excel (.xlsx, .xls)
        elif extension in ['.xlsx', '.xls']:
            import pandas as pd
            f = io.BytesIO(raw_bytes)
            sheets = pd.read_excel(f, sheet_name=None)
            text = []
            for sheet_name, df in sheets.items():
                text.append(f"=== Sheet: {sheet_name} ===")
                text.append(df.to_string(index=False))
            return "\n\n".join(text)

        # Images (OCR local via pytesseract)
        elif extension in ['.png', '.jpg', '.jpeg', '.tiff', '.bmp']:
            # ⚠️ nécessite tesseract installé dans l’environnement
            from PIL import Image
            import pytesseract
            img = Image.open(io.BytesIO(raw_bytes))
            return pytesseract.image_to_string(img)

        # Autres formats: tentative texte brut
        else:
            if raw_text is None and raw_bytes is not None:
                try:
                    raw_text = raw_bytes.decode('utf-8', errors='ignore')
                except Exception:
                    raw_text = None
            if raw_text:
                print(f"Avertissement: {extension} non spécifiquement supporté, retour texte brut.")
                return raw_text
            print(f"Avertissement: Type de fichier {extension} non supporté.")
            return None

    except Exception as e:
        print(f"Erreur lors de l'extraction du texte de {file_path}: {e}")
        return None


In [4]:
# 3 - fomction permettant de detecter la langue : 
def detect_language(text: str) -> str:
    """
    Détecte la langue d'un texte.
    
    Args:
        text: Texte à analyser
        
    Returns:
        Code de langue (ex: 'en', 'fr', 'es') ou message d'erreur
    """
    if not text or len(text.strip()) < 10:
        return "Erreur: Texte trop court ou vide"
    
    try:
        from langdetect import detect
        return detect(text)
    except ImportError:
        return "Erreur: La bibliothèque langdetect n'est pas installée"
    except Exception as e:
        return f"Erreur detection de langue : {str(e)}"

In [5]:
# 4 - Traduction automatique du texte en anglais : 
def translate_and_structure_bedrock(text: str, source_lang: str) -> str:
    """Traduit + structure en Markdown avec Bedrock (1 appel optimisé)."""
    
    client = boto3.client('bedrock-runtime')
    
    if source_lang == 'en':
        # Si déjà anglais, juste structurer
        prompt = f"""Structure this English regulatory document into clean Markdown.

Rules: Clear headings (# ## ###), lists, preserve all data, remove formatting artifacts.

Document:
{text}

Return ONLY Markdown:"""
    
    else:
        # Traduction en anglais + structuration
        prompt = f"""Translate this document to English AND structure it as Markdown.

Rules:
- Accurate translation to English (preserve dates, numbers, names)
- Clean Markdown structure (# ## ###, lists)
- Full content, no summary
- Remove formatting artifacts

Document:
{text}

Return ONLY English Markdown:"""
    # Limite le texte à ~400k caractères pour rester dans les limites
    truncated_text = text[:1000]
    final_prompt = prompt.replace("{text}", truncated_text)

    response = client.invoke_model(
        modelId='global.anthropic.claude-sonnet-4-5-20250929-v1:0',
        body=json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 100000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1
        })
    )
    
    result = json.loads(response['body'].read())
    return result['content'][0]['text']

In [6]:
# 4 - Traduction automatique du texte en anglais : 
def translate_and_structure_bedrock_aws(text: str, source_lang: str) -> str:
    """Traduit + structure en Markdown avec Bedrock (1 appel optimisé)."""
    
    client = boto3.client('bedrock-runtime')
    
    if source_lang == 'en':
        # Si déjà anglais, juste structurer
        prompt = f"""Structure this English regulatory document into clean Markdown.

Rules: Clear headings (# ## ###), lists, preserve all data, remove formatting artifacts.

Document:
{text}

Return ONLY Markdown:"""
    
    else:
        # Traduction en anglais + structuration
        prompt = f"""Translate this document to English AND structure it as Markdown.

Rules:
- Accurate translation to English (preserve dates, numbers, names)
- Clean Markdown structure (# ## ###, lists)
- Full content, no summary
- Remove formatting artifacts

Document:
{text}

Return ONLY English Markdown:"""
    # Limite le texte à ~400k caractères pour rester dans les limites
    truncated_text = text[:1000]
    final_prompt = prompt.replace("{text}", truncated_text)

    response = client.converse(
        modelId='global.anthropic.claude-sonnet-4-5-20250929-v1:0',
          messages=[
        {
            "role": "user",
            "content": [{"text": prompt}]
        }
    ],
    inferenceConfig={
        "temperature": 0.1,
    }
    )
    
    result = response["output"]["message"]["content"][0]["text"]
    return result





In [8]:
filepath = 's3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/shared/directives/1.DIRECTIVE (UE) 20192161 DU PARLEMENT EUROPÉEN ET DU CONSEIL.html'
extract_text = extract_text_from_any_file(filepath)
# text_in_english = translate_and_structure_bedrock(extract_text, detect_language(extract_text))
# display(Markdown(text_in_english))



In [39]:
# text_in_english = translate_and_structure_bedrock_aws(extract_text, detect_language(extract_text))
# display(Markdown(extract_text))

In [13]:
A_10k_files = 's3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/shared/fillings/A/2024-12-20-10k-A.html'
a_extract_text_aws = extract_text_from_any_file(A_10k_files)
# text_in_english_a = translate_and_structure_bedrock(a_extract_text_aws, detect_language(a_extract_text_aws))
# display(Markdown(text_in_english_a))

In [15]:
def translate_and_structure_bedrock_aws(text: str, source_lang: str, chunk_size: int = 15000, overlap: int = 1000) -> str:
    """
    Traduit + structure en Markdown avec Bedrock en découpant en chunks avec overlay.
    
    Args:
        text: Texte à traiter
        source_lang: Code langue source ('en', 'fr', etc.)
        chunk_size: Taille maximale d'un chunk en caractères (réduit à 15k pour respecter limite tokens)
        overlap: Nombre de caractères de chevauchement entre chunks
    
    Returns:
        Texte traduit et structuré en Markdown
    """
    
    client = boto3.client('bedrock-runtime')
    
    # Définir le prompt selon la langue
    if source_lang == 'en':
        task = "Structure this English regulatory document into clean Markdown."
    else:
        task = "Translate this document to English AND structure it as Markdown."
    
    prompt_template = f"""{task}

Rules:
- Accurate translation to English (preserve dates, numbers, names)
- Clean Markdown structure (# ## ###, lists)
- Full content, no summary
- Remove formatting artifacts

Document:
{{chunk}}

Return ONLY Markdown:"""
    
    # Si le texte est court, traiter en un seul appel
    if len(text) <= chunk_size:
        print(f"   📄 Texte court ({len(text):,} chars), traitement en 1 chunk\n")
        
        final_prompt = prompt_template.replace("{chunk}", text)
        
        response = client.converse(
            modelId='anthropic.claude-3-haiku-20240307-v1:0',
            messages=[{"role": "user", "content": [{"text": final_prompt}]}],
            inferenceConfig={
                "temperature": 0.1,
                "maxTokens": 4000  # ✅ MODIFIÉ : limite à 4000 au lieu de 100000
            }
        )
        
        return response["output"]["message"]["content"][0]["text"]
    
    # Sinon, découper en chunks avec overlay
    print(f"   📊 Texte long ({len(text):,} chars)")
    print(f"   ✂️  Découpage en chunks de {chunk_size:,} chars avec overlap de {overlap:,} chars\n")
    
    chunks = []
    start = 0
    chunk_num = 0
    
    while start < len(text):
        end = start + chunk_size
        
        # Prendre le chunk
        chunk = text[start:end]
        
        # Essayer de couper à un paragraphe pour éviter de couper au milieu d'une phrase
        if end < len(text):
            # Chercher le dernier \n\n dans les 500 derniers caractères
            last_para = chunk.rfind('\n\n', -500)
            if last_para > chunk_size - 1000:  # Si trouvé pas trop loin de la fin
                chunk = chunk[:last_para]
                end = start + last_para
        
        chunk_num += 1
        chunks.append((chunk_num, chunk, start, end))
        
        print(f"   📦 Chunk {chunk_num}: {start:,} → {end:,} ({len(chunk):,} chars)")
        
        # Avancer avec overlap
        start = end - overlap
    
    print(f"\n   ✅ {len(chunks)} chunks créés\n")
    
    # Traiter chaque chunk
    translated_chunks = []
    
    for chunk_num, chunk, start_pos, end_pos in chunks:
        print(f"   🔄 Traitement chunk {chunk_num}/{len(chunks)}...", end=" ", flush=True)
        
        final_prompt = prompt_template.replace("{chunk}", chunk)
        
        try:
            response = client.converse(
                modelId='anthropic.claude-3-haiku-20240307-v1:0',
                messages=[{"role": "user", "content": [{"text": final_prompt}]}],
                inferenceConfig={
                    "temperature": 0.1,
                    "maxTokens": 4000  # ✅ MODIFIÉ : limite à 4000
                }
            )
            
            result = response["output"]["message"]["content"][0]["text"]
            translated_chunks.append(result)
            print("✅")
            
        except Exception as e:
            print(f"❌ Erreur: {str(e)}")
            # En cas d'erreur, garder le texte original
            translated_chunks.append(f"\n\n[CHUNK {chunk_num} - Original text kept due to error]\n{chunk}\n\n")
    
    print(f"\n   🔗 Fusion des {len(translated_chunks)} chunks...\n")
    
    # Fusionner les chunks traduits
    final_text = translated_chunks[0]
    
    for i in range(1, len(translated_chunks)):
        chunk_text = translated_chunks[i]
        
        # Enlever les premières lignes qui pourraient être des répétitions
        lines = chunk_text.split('\n')
        # Garder à partir de la 3ème ligne pour éviter les répétitions de l'overlap
        chunk_text = '\n'.join(lines[3:]) if len(lines) > 3 else chunk_text
        
        final_text += "\n\n" + chunk_text
    
    return final_text

In [16]:
A_10k_files = 's3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/shared/fillings/A/2024-12-20-10k-A.html'
a_extract_text_aws = extract_text_from_any_file(A_10k_files)
text_in_english_a = translate_and_structure_bedrock_aws(a_extract_text_aws, detect_language(a_extract_text_aws))
# display(Markdown(text_in_english_a))

   📊 Texte long (554,357 chars)
   ✂️  Découpage en chunks de 15,000 chars avec overlap de 1,000 chars

   📦 Chunk 1: 0 → 15,000 (15,000 chars)
   📦 Chunk 2: 14,000 → 29,000 (15,000 chars)
   📦 Chunk 3: 28,000 → 43,000 (15,000 chars)
   📦 Chunk 4: 42,000 → 57,000 (15,000 chars)
   📦 Chunk 5: 56,000 → 71,000 (15,000 chars)
   📦 Chunk 6: 70,000 → 85,000 (15,000 chars)
   📦 Chunk 7: 84,000 → 99,000 (15,000 chars)
   📦 Chunk 8: 98,000 → 113,000 (15,000 chars)
   📦 Chunk 9: 112,000 → 127,000 (15,000 chars)
   📦 Chunk 10: 126,000 → 141,000 (15,000 chars)
   📦 Chunk 11: 140,000 → 155,000 (15,000 chars)
   📦 Chunk 12: 154,000 → 169,000 (15,000 chars)
   📦 Chunk 13: 168,000 → 183,000 (15,000 chars)
   📦 Chunk 14: 182,000 → 197,000 (15,000 chars)
   📦 Chunk 15: 196,000 → 211,000 (15,000 chars)
   📦 Chunk 16: 210,000 → 225,000 (15,000 chars)
   📦 Chunk 17: 224,000 → 239,000 (15,000 chars)
   📦 Chunk 18: 238,000 → 253,000 (15,000 chars)
   📦 Chunk 19: 252,000 → 267,000 (15,000 chars)
   📦 Chunk 20

# Translated Document

## Overview
- Document ID: a-20241031
- Document Number: 0001090872
- False
- Fiscal Year: 2024

## Accounting Standards
- [Cost of Goods and Services Sold](http://fasb.org/us-gaap/2024#CostOfGoodsAndServicesSold)
- [Research and Development Expense](http://fasb.org/us-gaap/2024#ResearchAndDevelopmentExpense)
- [Selling, General and Administrative Expense](http://fasb.org/us-gaap/2024#SellingGeneralAndAdministrativeExpense)
- [Other Assets, Noncurrent](http://fasb.org/us-gaap/2024#OtherAssetsNoncurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Other Liabilities, Noncurrent](http://fasb.org/us-gaap/2024#OtherLiabilitiesNoncurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent) [Other Liabilities, Noncurrent](http://fasb.org/us-gaap/2024#OtherLiabilitiesNoncurrent)
- [Cost of Goods and Services Sold](http://fasb.org/us-gaap/2024#CostOfGoodsAndServicesSold)
- [Selling, General and Administrative Expense](http://fasb.org/us-gaap/2024#SellingGeneralAndAdministrativeExpense)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Other Liabilities, Current](http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent)
- [Costs and Expenses](http://fasb.org/us-gaap/2024#CostsAndExpenses)
- [Costs and Expenses](http://fasb.org/us-gaap/2024#CostsAndExpenses)
- [Costs and Expenses](http://fasb.org/us-gaap/2024#CostsAndExpenses)
- [Cost of Goods and Services Sold](http://fasb.org/us-gaap/2024#CostOfGoodsAndServicesSold)
- [Cost of Goods and Services Sold](http://fasb.org/us-gaap/2024#CostOfGoodsAndServicesSold)
- [Cost of Goods and Services Sold](http://fasb.org/us-gaap/2024#CostOfGoodsAndServicesSold)
- [Other Nonoperating Income (Expense)](http://fasb.org/us-gaap/2024#OtherNonoperatingIncomeExpense)
- [Other Nonoperating Income (Expense)](http://fasb.org/us-gaap/2024#OtherNonoperatingIncomeExpense)
- [Other Nonoperating Income (Expense)](http://fasb.org/us-gaap/2024#OtherNonoperatingIncomeExpense)
- [Cost of Goods and Services Sold](http://fasb.org/us-gaap/2024#CostOfGoodsAndServicesSold)
- [Cost of Goods and Services Sold](http://fasb.org/us-gaap/2024#CostOfGoodsAndServicesSold)
- [Research and Development Expense](http://fasb.org/us-gaap/2024#ResearchAndDevelopmentExpense)
- [Research and Development Expense](http://fasb.org/us-gaap/2024#ResearchAndDevelopmentExpense)
- [Selling, General and Administrative Expense](http://fasb.org/us-gaap/2024#SellingGeneralAndAdministrativeExpense)
- [Selling, General and Administrative Expense](http://fasb.org/us-gaap/2024#SellingGeneralAndAdministrativeExpense)

## Currency
- ISO Currency Code: USD
- XBRL Shares: shares
- Number of Customers: a:numberOfCustomers
- XBRL Pure: xbrli:pure
- Contracts: a:contracts
- Segment: a:segment

## Dates
- Document Number: 0001090872
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- As of Date: 2024-04-30
- Document Number: 0001090872
- Date: 2024-12-09
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember
- Restructuring Plan End Date: 2022-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember
- Restructuring Plan End Date: 2022-10-31
- Document Number: 0001090872
- Restructuring Plan End Date: 2022-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember
- Restructuring Plan Start Date: 2022-11-01
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember
- Restructuring Plan Start Date: 2022-11-01
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan Start Date: 2022-11-01
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember
- Restructuring Plan Start Date: 2023-11-01
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember
- Restructuring Plan Start Date: 2023-11-01
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Cost of Sales: us-gaap:CostOfSalesMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Cost of Sales: us-gaap:CostOfSalesMember
- Start Date: 2022-11-01
- End Date: 2023-10-31
- Document Number: 0001090872
- Research and Development Expense: us-gaap:ResearchAndDevelopmentExpenseMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Research and Development Expense: us-gaap:ResearchAndDevelopmentExpenseMember
- Start Date: 2022-11-01
- End Date: 2023-10-31
- Document Number: 0001090872
- Selling, General and Administrative Expenses: us-gaap:SellingGeneralAndAdministrativeExpensesMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Selling, General and Administrative Expenses: us-gaap:SellingGeneralAndAdministrativeExpensesMember
- Start Date: 2022-11-01
- End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: a:FY24RestructuringPlanMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember, a:FY24RestructuringPlanMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember, a:FY24RestructuringPlanMember
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember, a:FY24RestructuringPlanMember
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: a:FY24RestructuringPlanMember
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: a:FY23RestructuringPlanMember
- Start Date: 2022-11-01
- End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: a:FY23RestructuringPlanMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember, a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2022-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember, a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2022-10-31
- Document Number: 0001090872
- Restructuring Plan: a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2022-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember, a:FY23RestructuringPlanMember
- Start Date: 2022-11-01
- End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember, a:FY23RestructuringPlanMember
- Start Date: 2022-11-01
- End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember, a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember, a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2023-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember, a:FY23RestructuringPlanMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember, a:FY23RestructuringPlanMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:EmployeeSeveranceMember, a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: us-gaap:FacilityClosingMember, a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Restructuring Plan: a:FY23RestructuringPlanMember
- Restructuring Plan End Date: 2024-10-31
- Document Number: 0001090872
- Product: us-gaap:ProductMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Product: us-gaap:ProductMember
- Start Date: 2022-11-01
- End Date: 2023-10-31
- Document Number: 0001090872
- Product: us-gaap:ProductMember
- Start Date: 2021-11-01
- End Date: 2022-10-31
- Document Number: 0001090872
- Service Other: us-gaap:ServiceOtherMember
- Start Date: 2023-11-01
- End Date: 2024-10-31
- Document Number: 0001090872
- Service Other: us-gaap:ServiceOtherMember
- Start Date: 2022-11-01
- End Date: 2023-10-31
- Document Number: 0001090872
- Service Other: us-gaap:ServiceOtherMember
- Start Date: 2021-11-01
- End Date: 2022-10-31
- Document Number: 0001090872
- Start Date: 2021-11-01
- End Date: 2022-10-31
- Document Number: 0001090872
- As of Date: 2021-10-31
- Document Number: 0001090872
- Member: us-gaap:CommonStockMember
- As of Date: 2021-10-31
- Document Number: 0001090872
- Member: us-gaap:AdditionalPaidInCapitalMember
- As of Date: 2021-10-31
- Document Number: 0001090872
- Member: us-gaap:Retaine

- 2024-10-31
- 0001090872

## Performance Shares
- 2023-11-01 to 2024-10-31: Minimum Member
- 2023-11-01 to 2024-10-31: Maximum Member
- 2023-11-01 to 2024-10-31

## Cost of Sales
- 2021-11-01 to 2022-10-31

## Research and Development Expense
- 2021-11-01 to 2022-10-31

## Selling, General and Administrative Expenses
- 2021-11-01 to 2022-10-31

## Employee Stock Option
- 2022-11-01 to 2023-10-31
- 2021-11-01 to 2022-10-31

## Performance Shares
- 2022-11-01 to 2023-10-31
- 2021-11-01 to 2022-10-31

## LTPPRSU
- 2023-11-01 to 2024-10-31
- 2022-11-01 to 2023-10-31
- 2021-11-01 to 2022-10-31

## Range of Exercise Price 03
- 2023-11-01 to 2024-10-31
- 2024-10-31

## Range of Exercise Price 04
- 2023-11-01 to 2024-10-31
- 2024-10-31

## Range of Exercise Price 05
- 2023-11-01 to 2024-10-31
- 2024-10-31

## Foreign Country
- 2024-10-31

## Domestic Country
- 2024-10-31

## California Franchise Tax Board
- 2024-10-31

## Other Noncurrent Assets
- 2024-10-31
- 2023-10-31

## Other Noncurrent Liabilities
- 2024-10-31
- 2023-10-31

## Tax Interest and Penalties
- 2023-11-01 to 2024-10-31
- 2022-11-01 to 2023-10-31
- 2021-11-01 to 2022-10-31

## Life Sciences and Applied Markets
- 2022-10-31
- 2023-10-31
- 2024-10-31
- 2023-11-01 to 2024-01-31

## Diagnostics And Genomics
- 2022-10-31
- 2023-10-31
- 2024-10-31
- 2023-11-01 to 2024-01-31

## Agilent CrossLab
- 2022-10-31
- 2023-10-31
- 2024-10-31

## Developed Technology Rights
- 2023-10-31
- 2024-10-31
- 2023-11-01 to 2024-10-31

## Trademarks
- 2023-10-31
- 2024-10-31

## Customer Relationships
- 2023-10-31
- 2024-10-31
- 2024-10-31: BIOVECTRA Member

## Third Party Technology and Licenses
- 2023-10-31
- 2024-10-31

## Acquisition of Businesses
- 2022-11-01 to 2023-10-31

## Resolution Bioscience Inc
- 2022-11-01 to 2023-10-31: Cost of Sales
- 2022-11-01 to 2023-10-31: Selling, General and Administrative Expenses

## Fair Value Measurements Recurring
- 2024-10-31: Other Current Assets, Level 1, Level 2, Level 3
- 2024-10-31: Other Noncurrent Assets, Level 1, Level 2, Level 3
- 2024-10-31: Level 1, Level 2, Level 3
- 2024-10-31: Other Noncurrent Liabilities, Level 1, Level 2, Level 3
- 2023-10-31: Other Current Assets, Level 1, Level 2, Level 3
- 2023-10-31: Other Noncurrent Assets, Level 1, Level 2, Level 3
- 2023-10-31: Level 1, Level 2, Level 3
- 2023-10-31: Other Current Liabilities, Level 1, Level 2, Level 3
- 2023-10-31: Other Noncurrent Liabilities, Level 1, Level 2, Level 3

## Hedging Instruments
- 2016-02-01 to 2024-10-31: Senior Notes 2026, Interest Rate Swap, Cash Flow Hedging, Designated as Hedging Instrument
- 2019-08-01 to 2024-10-31: Senior Notes 2029, Treasury Lock, Cash Flow Hedging, Designated as Hedging Instrument
- 2024-10-31: Forward Contracts, Cash Flow Hedging, Designated as Hedging Instrument
- 2024-10-31: Forward Contracts, Short, Cash Flow Hedging, Designated as Hedging Instrument
- 2024-10-31: Forward Contracts, Net Investment Hedging, Designated as Hedging Instrument
- 2024-10-31: Forward Contracts, Short, Net Investment Hedging, Designated as Hedging Instrument
- 2024-10-31: Forward Contracts, Nondesignated
- 2024-10-31: Forward Contracts, Short, Nondesignated
- 2024-10-31 to 2023-10-31: Foreign Exchange Contract, Cash Flow Hedging, Designated as Hedging Instrument, Other Current Assets, Accrued Liabilities
- 2024-10-31 to 2023-10-31: Foreign Exchange Contract, Net Investment Hedging, Designated as Hedging Instrument, Other Current Assets, Accrued Liabilities
- 2024-10-31 to 2023-10-31: Foreign Exchange Contract, Nondesignated, Other Current Assets, Accrued Liabilities
- 2023-11-01 to 2021-11-01: Foreign Exchange Contract, Cash Flow Hedging, Designated as Hedging Instrument, Accumulated Other Comprehensive Income
- 2023-11-01 to 2021-11-01: Foreign Exchange Contract, Cash Flow Hedging, Designated as Hedging Instrument, Interest Expense
- 2023-11-01 to 2021-11-01: Foreign Exchange Contract, Net Investment Hedging, Designated as Hedging Instrument, Accumulated Other Comprehensive Income
- 2023-11-01 to 2021-11-01: Nondesignated
- 2023-11-01 to 2021-11-01: Cash Flow Hedging
- 2023-11-01 to 2021-11-01: Cost of Sales, Cash Flow Hedging

## Pension Plans Defined Benefit
- 2023-11-01 to 2024-10-31: Minimum, Maximum
- 2023-11-01 to 2024-10-31
- 2023-11-01 to 2024-10-31: US
- 2023-11-01 to 2024-10-31: Foreign Plan
- 2023-11-01 to 2024-10-31: US, Foreign Plan

## Other
- 2024-10-31: Deferred Profit Sharing
- 2023-10-31: Deferred Profit Sharing

### 2023-11-01 to 2024-10-31
- Member: `us-gaap:PensionPlansDefinedBenefitMember`
- Maximum: `srt:MaximumMember`
- Country: `US`

### 2022-11-01 to 2023-10-31
- Member: `us-gaap:PensionPlansDefinedBenefitMember`
- Country: `US`

### 2021-11-01 to 2022-10-31
- Member: `us-gaap:PensionPlansDefinedBenefitMember`
- Country: `US`
- Foreign Plan: `us-gaap:ForeignPlanMember`

### 2023-10-31
- Member: `us-gaap:PensionPlansDefinedBenefitMember`
- Country: `US`

### 2022-10-31
- Member: `us-gaap:PensionPlansDefinedBenefitMember`
- Country: `US`
- Foreign Plan: `us-gaap:ForeignPlanMember`

### 2024-10-31
- Member: `us-gaap:PensionPlansDefinedBenefitMember`
- Country: `US`
- Other Assets: `us-gaap:OtherAssetsMember`
- Employee Compensation and Benefits: `a:EmployeecompensationandbenefitsMember`
- Retirement and Postretirement Benefits: `a:RetirementandpostretirementbenefitsMember`

### 2023-10-31
- Member: `us-gaap:PensionPlansDefinedBenefitMember`
- Country: `US`
- Other Assets: `us-gaap:OtherAssetsMember`
- Employee Compensation and Benefits: `a:EmployeecompensationandbenefitsMember`
- Retirement and Postretirement Benefits: `a:RetirementandpostretirementbenefitsMember`

### Foreign Plan Subsequent Event
- Member: `us-gaap:ForeignPlanMember`, `us-gaap:PensionPlansDefinedBenefitMember`, `us-gaap:SubsequentEventMember`
- Date: 2024-11-28

## Defined Benefit Postretirement Health Coverage
### 2023-11-01 to 2024-10-31
- Member: `us-gaap:DefinedBenefitPostretirementHealthCoverageMember`
- Country: `US`

### 2022-11-01 to 2023-10-31
- Member: `us-gaap:DefinedBenefitPostretirementHealthCoverageMember`
- Country: `US`

### 2021-11-01 to 2022-10-31
- Member: `us-gaap:DefinedBenefitPostretirementHealthCoverageMember`
- Country: `US`

### 2023-10-31
- Member: `us-gaap:DefinedBenefitPostretirementHealthCoverageMember`
- Country: `US`

### 2022-10-31
- Member: `us-gaap:DefinedBenefitPostretirementHealthCoverageMember`
- Country: `US`

### 2024-10-31
- Member: `us-gaap:DefinedBenefitPostretirementHealthCoverageMember`
- Country: `US`
- Other Assets: `us-gaap:OtherAssetsMember`
- Employee Compensation and Benefits: `a:EmployeecompensationandbenefitsMember`
- Retirement and Postretirement Benefits: `a:RetirementandpostretirementbenefitsMember`

### 2023-10-31
- Member: `us-gaap:DefinedBenefitPostretirementHealthCoverageMember`
- Country: `US`
- Other Assets: `us-gaap:OtherAssetsMember`
- Employee Compensation and Benefits: `a:EmployeecompensationandbenefitsMember`
- Retirement and Postretirement Benefits: `a:RetirementandpostretirementbenefitsMember`

## Deferred Profit Sharing
### 2024-10-31
- Member: `us-gaap:DeferredProfitSharingMember`
- Equity Securities: `us-gaap:EquitySecuritiesMember`
- Fixed Income Securities: `us-gaap:FixedIncomeSecuritiesMember`

## Other Plan Assets
### 2024-10-31
- Pension Plans Defined Benefit: `us-gaap:PensionPlansDefinedBenefitMember`, `a:OtherPlanAssetsMember`
- Other Postretirement Benefit Plans Defined Benefit: `us-gaap:OtherPostretirementBenefitPlansDefinedBenefitMember`, `a:OtherPlanAssetsMember`

## Fair Value Measurements
### Cash and Cash Equivalents
#### 2024-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member` 
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

#### 2023-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member` 
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

### Equity Securities
#### 2024-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`
- Other Postretirement Benefit Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

#### 2023-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

### Fixed Income Securities
#### 2024-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`
- Other Postretirement Benefit Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

#### 2023-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

### Other Plan Assets
#### 2024-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

#### 2023-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

### Overall
#### 2024-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

#### 2023-10-31
- Pension Plans Defined Benefit
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

#### 2022-10-31
- Pension Plans Defined Benefit
  - Level 3: `us-gaap:FairValueInputsLevel3Member`

#### 2022-11-01 to 2023-10-31
- Pension Plans Defined Benefit
  - Level 3: `us-gaap:FairValueInputsLevel3Member`

### Defined Benefit Postretirement Health Coverage
#### 2024-10-31
- Cash and Cash Equivalents
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`
- Equity Securities
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`
- Fixed Income Securities
  - Level 1: `us-gaap:FairValueInputsLevel1Member`
  - Level 2: `us-gaap:FairValueInputsLevel2Member`
  - Level 3: `us-gaap:FairValueInputsLevel3Member`
  - Net Asset Value: `us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember`

### us-gaap:EquitySecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:EquitySecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:EquitySecuritiesMember
#### 2024-10-31
#### 0001090872
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### a:OtherPlanAssetsMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### a:OtherPlanAssetsMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### a:OtherPlanAssetsMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### a:OtherPlanAssetsMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:CashAndCashEquivalentsMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:CashAndCashEquivalentsMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:CashAndCashEquivalentsMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:CashAndCashEquivalentsMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:EquitySecuritiesMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:EquitySecuritiesMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:EquitySecuritiesMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:EquitySecuritiesMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### a:OtherPlanAssetsMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### a:OtherPlanAssetsMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### a:OtherPlanAssetsMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
### a:OtherPlanAssetsMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2023-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2022-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2023-11-01
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## country:US
### us-gaap:DefinedBenefitPostretirementHealthCoverageMember
#### 2022-11-01
#### 2023-10-31
#### 0001090872
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:CashAndCashEquivalentsMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:CashAndCashEquivalentsMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:CashAndCashEquivalentsMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:CashAndCashEquivalentsMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:DefinedBenefitPlanEquitySecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:DefinedBenefitPlanEquitySecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:DefinedBenefitPlanEquitySecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:DefinedBenefitPlanEquitySecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel3Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:FixedIncomeSecuritiesMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueMeasuredAtNetAssetValuePerShareMember
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:OtherContractMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel1Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:OtherContractMember
#### 2024-10-31
#### 0001090872
### us-gaap:FairValueInputsLevel2Member
## us-gaap:ForeignPlanMember
### us-gaap:PensionPlansDefinedBenefitMember
### us-gaap:OtherContractMember
#### 2024-10-31

- us-gaap:FairValueInputsLevel3Member
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember

## 2023-11-01 - 2024-10-31
### 0001090872
- us-gaap:FairValueInputsLevel3Member
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember

## 2022-11-01 - 2023-10-31
### 0001090872
- country:US
- us-gaap:OtherPostretirementBenefitPlansDefinedBenefitMember

## 2024-10-31
### 0001090872
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember
- srt:MinimumMember

## 2023-11-01 - 2024-10-31
### 0001090872
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember
- srt:MaximumMember

## 2022-11-01 - 2023-10-31
### 0001090872
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember
- srt:MinimumMember

## 2021-11-01 - 2022-10-31
### 0001090872
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember
- srt:MaximumMember

## 2021-10-31
### 0001090872
- country:US
- us-gaap:DefinedBenefitPostretirementHealthCoverageMember

## 2020-11-01 - 2021-10-31
### 0001090872
- country:US
- us-gaap:DefinedBenefitPostretirementHealthCoverageMember

## 2024-10-31
### 0001090872
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember
- srt:MinimumMember

## 2024-10-31
### 0001090872
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember
- srt:MaximumMember

## 2023-10-31
### 0001090872
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember
- srt:MinimumMember

## 2023-10-31
### 0001090872
- us-gaap:ForeignPlanMember
- us-gaap:PensionPlansDefinedBenefitMember
- srt:MaximumMember

## 2023-06-07
### 0001090872
- a:IncrementalRevolvingCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2023-06-07
### 0001090872
- a:A5YrUnsecuredCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2023-06-07
### 0001090872
- a:A5YrUnsecuredCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2023-06-07
### 0001090872
- a:IncrementalRevolvingCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2023-11-01 - 2024-10-31
### 0001090872
- a:A5YrUnsecuredCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2022-11-01 - 2023-10-31
### 0001090872
- a:A5YrUnsecuredCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2023-10-31
### 0001090872
- a:IncrementalRevolvingCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2024-10-31
### 0001090872
- a:A5YrUnsecuredCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2023-10-31
### 0001090872
- a:A5YrUnsecuredCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2024-10-31
### 0001090872
- a:IncrementalRevolvingCreditFacilityMember
- us-gaap:LineOfCreditMember

## 2023-06-02
### 0001090872
- a:UncommittedMoneyMarketLineCreditAgreementMember
- us-gaap:LineOfCreditMember

## 2023-06-02
### 0001090872
- a:UncommittedMoneyMarketLineCreditAgreementMember
- us-gaap:LineOfCreditMember

## 2023-11-01 - 2024-10-31
### 0001090872
- a:UncommittedMoneyMarketLineCreditAgreementMember
- us-gaap:LineOfCreditMember

## 2022-11-01 - 2023-10-31
### 0001090872
- a:UncommittedMoneyMarketLineCreditAgreementMember
- us-gaap:LineOfCreditMember

## 2024-10-31
### 0001090872
- a:UncommittedMoneyMarketLineCreditAgreementMember
- us-gaap:LineOfCreditMember

## 2023-10-31
### 0001090872
- a:UncommittedMoneyMarketLineCreditAgreementMember
- us-gaap:LineOfCreditMember

## 2023-06-02
### 0001090872
- us-gaap:CommercialPaperMember
- us-gaap:LineOfCreditMember

## 2023-11-01 - 2024-10-31
### 0001090872
- us-gaap:CommercialPaperMember
- us-gaap:LineOfCreditMember

## 2022-11-01 - 2023-10-31
### 0001090872
- us-gaap:CommercialPaperMember
- us-gaap:LineOfCreditMember

## 2024-10-31
### 0001090872
- us-gaap:CommercialPaperMember
- us-gaap:LineOfCreditMember

## 2023-10-31
### 0001090872
- us-gaap:CommercialPaperMember
- us-gaap:LineOfCreditMember

## 2024-10-31
### 0001090872
- a:StrategicInnovationFundMember

## 2024-10-31
### 0001090872
- a:AtlanticCanadaOpportunitiesAgencyACOAMember

## 2022-04-15
### 0001090872
- a:TermLoanMaturing2025Member

## 2022-05-04
### 0001090872
- a:SeniorNotes2023Member

## 2013-06-21
### 0001090872
- a:SeniorNotes2023Member

## 2023-10-31
### 0001090872
- a:TermLoanMaturing2025Member

## 2023-11-01 - 2024-10-31
### 0001090872
- a:TermLoanMaturing2025Member

## 2024-10-31
### 0001090872
- a:SeniorNotes2026Member
- us-gaap:SeniorNotesMember

## 2023-10-31
### 0001090872
- a:SeniorNotes2026Member
- us-gaap:SeniorNotesMember

## 2024-10-31
### 0001090872
- a:SeniorNotes2027Member
- us-gaap:SeniorNotesMember

## 2023-10-31
### 0001090872
- a:SeniorNotes2027Member
- us-gaap:SeniorNotesMember

## 2024-10-31
### 0001090872
- a:SeniorNotes2029Member
- us-gaap:SeniorNotesMember

## 2023-10-31
### 0001090872
- a:SeniorNotes2029Member
- us-gaap:SeniorNotesMember

## 2024-10-31
### 0001090872
- a:SeniorNotes2030Member
- us-gaap:SeniorNotesMember

## 2023-10-31
### 0001090872
- a:SeniorNotes2030Member
- us-gaap:SeniorNotesMember

## 2024-10-31
### 0001090872
- a:SeniorNotes2031Member
- us-gaap:SeniorNotesMember

## 2023-10-31
### 0001090872
- a:SeniorNotes2031Member
- us-gaap:SeniorNotesMember

## 2024-10-31
### 0001090872
- a:SeniorNotes2034Member
- us-gaap:SeniorNotesMember

## 2023-10-31
### 0001090872
- a:SeniorNotes2034Member
- us-gaap:SeniorNotesMember

## 2016-09-22
### 0001090872
- a:SeniorNotes2026Member

## 2016-09-22
### 0001090872
- a:SeniorNotes2026Member

## 2024-09-09
### 0001090872
- a:SeniorNotes2026Member

## 2016-09-15
### 0001090872
- a:SeniorNotes2026Member
- us-gaap:InterestRateSwapMember
- us-gaap:CashFlowHedgingMember
- us-gaap:DesignatedAsHedgingInstrumentMember

## 2024-09-09
### 0001090872
- a:SeniorNotes2027Member

## 2024-09-09
### 0001090872
- a:SeniorNotes2027Member

## 2019-09-16
### 0001090872
- a:SeniorNotes2029Member

## 2019-09-16
### 0001090872
- a:SeniorNotes2029Member

## 2019-08-01
### 0001090872
- us-gaap:TreasuryLockMember

## 2020-06-04
### 0001090872
- a:SeniorNotes2030Member

## 2020-06-04
### 0001090872
- a:SeniorNotes2030Member

## 2021-03-12
### 0001090872
- a:SeniorNotes2031Member

## 2021-03-12
### 0001090872
- a:SeniorNotes2031Member

## 2024-09-09
### 0001090872
- a:SeniorNotes2034Member

## 2024-09-09
### 0001090872
- a:SeniorNotes2034Member

## 2021-02-18
### 0001090872
- a:A2021RepurchaseProgramMember

## 2021-11-01 - 2022-10-31
### 0001090872
- a:A2021RepurchaseProgramMember

## 2022-11-01 - 2023-10-31
### 0001090872
- a:A2021RepurchaseProgramMember

## 2023-10-31
### 0001090872
- a:A2021RepurchaseProgramMember

## 2023-03-01
### 0001090872
- a:A2023RepurchaseProgramMember

## 2022-11-01 - 2023-10-31
### 0001090872
- a:A2023RepurchaseProgramMember

## 2023-11-01 - 2024-10-31
### 0001090872
- a:A2023RepurchaseProgramMember

## 2024-10-31
### 0001090872
- a:A2023RepurchaseProgramMember

## 2024-05-29
### 0001090872
- a:A2024RepurchaseProgramMember

## 2024-11-20
### 0001090872
- us-gaap:SubsequentEventMember

## 2022-10-31
### 0001090872
- us-gaap:AccumulatedTranslationAdjustmentMember

## 2022-10-31
### 0001090872
- us-gaap:AccumulatedDefinedBenefitPlansAdjustmentNetPriorServiceCostCreditMember

## 2022-10-31
### 0001090872
- us-gaap:AccumulatedDefinedBenefitPlansAdjustmentNetUnamortizedGainLossMember

## 2022-10-31
### 0001090872
- us-gaap:AociDerivativeQualifyingAsHedgeExcludedComponentParentMember

## 2022-11-01 - 2023-10-31
### 0001090872
- us-gaap:AccumulatedTranslationAdjustmentMember

## 2022-11-01 - 2023-10-31
### 0001090872
- us-gaap:AccumulatedDefinedBenefitPlansAdjustmentNetPriorServiceCostCreditMember

## 2022-11-01 - 2023-10-31
### 0001090872
- us-gaap:AccumulatedDefinedBenefitPlansAdjustmentNetUnamortizedGainLossMember

## 2022-11-01 - 2023-10-31
### 0001090872
- us-gaap:AociDerivativeQualifyingAsHedgeExcludedComponentParentMember

## 2023-10-31
### 0001090872
- us-gaap:AccumulatedTranslationAdjustmentMember

## 2023-10-31
### 0001090872
- us-gaap:AccumulatedDefinedBenefitPlansAdjustmentNetPriorServiceCostCreditMember

## 2023-10-31
### 0001090872
- us-gaap:AccumulatedDefinedBenefitPlansAdjustmentNetUnamortizedGainLossMember

## 2023-10-31
### 0001090872
- us-gaap:AociDerivativeQualifyingAsHedgeExcludedComponentParentMember

## 2023-11-01 - 2024-10-31
### 0001090872
- us-gaap:AccumulatedTranslationAdj


Agilent Technologies Inc. ("we", "Agilent" or the "company"), incorporated in Delaware in May 1999, is a global leader in life sciences, diagnostics and applied chemical markets, providing application focused solutions that include instruments, software, services and consumables for the entire laboratory workflow.

In the first quarter of fiscal year 2024, we announced a change in our operating segments to move our cell analysis business from our life sciences and applied markets segment to our diagnostics and genomics operating segment in order to further strengthen growth opportunities for both organizations. Following this reorganization, we continued to have three business segments comprised of life sciences and applied markets, diagnostics and genomics and Agilent CrossLab, each of which continues to comprise a reportable segment. All historical financial segment information has been recast to conform to this new presentation in our consolidated financial statements and accompanying notes. There was no change to our Agilent CrossLab business segment. See also Note 23, "Subsequent Event" for additional information on recent changes to our organizational structure.

Our life sciences and applied markets business provides application-focused solutions that include instruments, consumables and software that enable customers to identify, quantify and analyze the physical and biological properties of substances and products, as well as enable customers in the clinical and life sciences research areas to interrogate samples at the molecular and cellular level. Our consumables portfolio is designed to improve customer outcomes. Our diagnostics and genomics business is comprised of seven areas of activity providing active pharmaceutical ingredients ("APIs") for oligo-based therapeutics as well as solutions that include reagents, instruments, software and consumables which enable customers in the clinical and life sciences research areas to interrogate samples at the cellular and molecular level. The Agilent CrossLab business spans the entire lab with its extensive services portfolio, which is designed to improve customer outcomes. In addition, we conduct centralized order fulfillment and supply chain operations for our businesses through the order fulfillment and supply chain organization ("OFS"). OFS provides resources for manufacturing, engineering and strategic sourcing to our respective businesses. Each of our businesses, together with OFS, is supported by our global infrastructure organization, which provides shared services in the areas of finance, information technology, legal, certain procurement services, workplace services and human resources.

We sell our products primarily through direct sales, but we also utilize distributors, resellers, manufacturers' representatives and electronic commerce. As of October 31, 2024, we employed approximately 17,900 people worldwide. Our primary research and development and manufacturing sites are in California, Colorado, Delaware, Massachusetts, Texas and Vermont in the U.S. and in Australia, Canada, China, Denmark, Germany, Italy, Japan, Malaysia, Singapore and the United Kingdom.

## Life Sciences and Applied Markets Business

Our life sciences and applied markets business provides application-focused solutions that include instruments, consumables and software that enable customers to identify, quantify and analyze the physical and biological properties of substances and products, as well as enable customers in the clinical and life sciences research areas to interrogate samples at the molecular and cellular level. Key product categories include: liquid chromatography ("LC") systems and components; liquid chromatography mass spectrometry ("LCMS") systems; gas chromatography ("GC") systems and components; gas chromatography mass spectrometry ("GCMS") systems; inductively coupled plasma mass spectrometry ("ICP-MS") instruments; atomic absorption ("AA") instruments; microwave plasma-atomic emission spectrometry ("MP-AES") instruments; inductively coupled plasma optical emission spectrometry ("ICP-OES") instruments; raman spectroscopy; laboratory software for sample tracking; information management and analytics; laboratory automation and robotic systems; dissolution testing; vacuum pumps and measurement technologies. Our consumables portfolio is designed to improve customer outcomes. Most of the portfolio is vendor neutral, meaning Agilent can serve and supply customers regardless of their instrument purchase choices. Solutions range from chemistries to supplies. Key product categories in consumables include GC and LC columns, sample preparation products, custom chemistries, and a large selection of laboratory instrument supplies.

We employed approximately 6,000 people as of October 31, 2024 in our life sciences and applied markets business.

### Life Sciences and Applied Markets

#### The Pharmaceutical, Biopharmaceutical, CRO & CMO Market

This market consists of "for-profit" companies which participate across the pharmaceutical value chain in the areas of therapeutic research, discovery & development, clinical trials, manufacturing and quality assurance and quality control. One sub-segment of this market is core and emerging pharmaceutical companies ("pharma"). A second sub-segment includes biopharmaceutical companies ("biopharma"), contract research organizations ("CROs") and contract manufacturing organizations ("CMOs"). Biopharma companies and, to a somewhat lesser extent, CROs and CMOs typically participate in specific points in the pharmaceutical industry value chain. Additionally, due to the relatively low drug efficacy within oncology, pharma companies are partnering with diagnostic companies to bring validated tests to the market with their new drugs.

#### The Academic and Government Market

This market consists primarily of "not-for-profit" organizations and includes academic institutions, large government institutes and privately funded organizations. The academic and government market plays an influential role in technology adoption and therapeutic developments for pharmaceutical and molecular diagnostics companies. After decades of investment in basic biomedical research by government funding bodies, the focus has widened to include translational research - multidisciplinary scientific efforts directed at accelerating therapy development.

#### The Chemicals & Advanced Materials Market

Our products and solutions are used throughout the chemicals sector in the development, manufacturing, and quality control of commodity chemicals, specialty and agrochemicals, and fine chemicals. Chemical market customers use our products to determine chemical composition, perform impurity analysis, qualify raw materials, conduct materials characterization, and verify and ensure the environmental safety of operations and employees. Our products are used to test for safety, quality, and compliance across the value chains of advanced materials – including semiconductors/electronics, batteries, specially engineered polymers and polymeric materials, minerals & metals, thin film & optics, consumer products and packing materials – from the upstream raw materials, materials production, and final products to the end markets and recycling. The upstream petroleum exploration and refining markets use our products to analyze natural gas, crude oil composition, perform intermediate material analysis, verify and improve refining processes and ensure the overall quality of gasoline, fuels, lubricants and other products.

#### The Environmental & Forensics Market

Our instruments, software and workflow solutions are used by the environmental market for applications such as laboratory and field analysis of regulated and unregulated chemical pollutants in air, water, soil and solid waste. Environmental industry customers include all levels of government, the industrial and manufacturing sectors, engineering and consulting companies, commercial testing laboratories, public and private utilities and colleges and universities. Drug testing and forensics laboratories use our instruments, software and workflow solutions for applications such as analyzing evidence associated with crime, screening athletes for performance enhancing drugs, analyzing samples for recreational drugs, or detecting and identifying biological and chemical warfare agents. Some of our instruments are used in mobile laboratories as well. Customers include local, state, federal, and international law enforcement agencies and health laboratories.

#### The Food Market

Our instruments, software, and workflow solutions are used throughout the food production chain, including raw material testing, process monitoring, and final product testing and quality control. Customers include food and beverage manufacturers, contract testing laboratories, and regulatory agencies.

Our instruments, software, and workflow solutions are used throughout the food production chain, including incoming inspection, new product development, quality control and assurance, and packaging. For example, our mass spectrometer portfolio is used to analyze contaminants and residual pesticides in food. There is also a significant food safety market involved in analyzing food for pathogen contamination, accurate verification of species type and evidence of genetically modified content.

## The Diagnostics and Clinical Market
The diagnostics and clinical market focus within our life sciences and applied markets business is to provide instruments, software, reagents, and consumables that enable customers performing life sciences, pharmaceutical and clinical research to interrogate biologically relevant metabolites, lipids, protein, and cellular systems to understand fundamental biological processes, as well as the underlying mechanisms of cancer and other disease initiation and progression. The goal is to use this information to develop new therapeutic strategies and drugs as well as new diagnostic tests. Our mass spectrometry technologies are employed by researchers to identify and quantify individual or whole classes of metabolites, lipids, or proteins involved in basic cellular processes and elucidate those which are quantitatively or qualitatively altered in disease states, as well as to identify those which may be useful as biomarkers for a disease.

## Life Sciences and Applied Markets Products and Applications

### Liquid Chromatography
A liquid chromatograph, high-performance liquid chromatograph ("HPLC") or ultra-high performance liquid chromatograph ("UHPLC") is used to separate molecules of a liquid mixture to determine the quantity and identity of the molecules present. The Agilent LC portfolio is largely modular in construction and can be configured as analytical and preparative systems. These systems can be stepwise upgraded to highly sophisticated, automated workflow solutions such as method development, multi‑method/walk-up, high-capacity/high-throughput or multi‑dimensional LC and can be extended to application‑based analyzers (e.g., for bio-molecular separations, chiral analysis or size exclusion chromatography). As a leader in liquid chromatography, we continue to expand our application space with new HPLC columns, new services and ongoing instrument and software product enhancements.

### Gas Chromatography
Agilent is the world's leading provider of gas chromatographs, both laboratory and portable models. Gas chromatographs are used to separate any gas, liquid or solid that can be vaporized and then detect the molecules present to determine their identity and quantity. Agilent provides custom or standard analyzers configured for specific chemical analysis applications, such as detailed speciation of a complex hydrocarbon stream, calculation of gas calorific values in the field, or analysis of a new bio-fuel formulation. We also offer related software, accessories and consumable products for these and other similar instruments.

### Mass Spectrometry
A mass spectrometer ("MS") identifies and quantifies compounds based on their molecular mass and characteristic patterns of fragment ion masses that result when a molecule is broken apart. MS is an important tool in analyzing a broad spectrum of analytes, from small molecules, such as pesticides, to large molecules, such as intact proteins and other biological entities. Liquid chromatography ("LC") and gas chromatography ("GC") are commonly used to separate compounds and introduce them to the MS system. Agilent's LCMS portfolio includes instruments built around four main analyzer types - single quadrupole, triple quadrupole, time-of-flight ("TOF") and quadrupole time-of-flight ("QTOF"). Agilent's GC/MS portfolio includes instruments built around three main analyzer types - single quadrupole, triple quadrupole, and quadrupole time-of-flight ("QTOF"). We significantly expanded our mass spectrometry portfolio in recent years with a focus on improving performance, sensitivity, precision, robustness, ease of use and onboard intelligence.

### Spectroscopy
Spectroscopy is a technique for analyzing the individual chemical components of substances based on the absorption or emission of electromagnetic radiation of specific wavelengths of light. Our spectroscopy instruments include AA spectrometers, microwave plasma-atomic emission spectrometers ("MP-AES"), ICP-OES, ICP-MS, fluorescence spectrophotometers, ultraviolet-visible ("UV-Vis") spectrophotometers, Fourier Transform infrared ("FT-IR") spectrometers, near-infrared ("NIR") spectrometers, raman spectrometers and sample automation products. We also offer related software, accessories and consumable products for these and other similar instruments.

### Software and Informatics
We provide informatics and scientific software for instrument control, data acquisition, data analysis, secure storage of results, and laboratory information/workflow management. Our software facilitates the compliant use of instruments in pharmaceutical quality assurance/quality control environments. With our OpenLab Laboratory Software Suite, Agilent has a scalable, open software platform that enables customers to capture, analyze, and share scientific data throughout the lab and across the enterprise.

### Laboratory Automation and Robotics
We offer a portfolio of unique sample preparation automated solutions that are key to a comprehensive suite of workflow solutions to our life science and genomics customers. This includes liquid handling, plate management, unique consumables and scheduling software with solutions that range from standalone instrumentation to bench-top automation solutions. These solutions strengthen our offering of automated sample preparation across a broad range of applications which are integrated with several of our analytical and NGS platforms across the company.

### Vacuum Technology
Our vacuum technologies products are used to create, control, measure and test vacuum environments in life science, industrial and scientific applications where ultra-clean, high-vacuum environments are needed. Vacuum technologies' customers are typically OEMs that manufacture equipment for these applications, or government and research organizations that require vacuum solutions in their facilities. Products include a wide range of high and ultra-high vacuum pumps (diffusion, turbo molecular and ion getter), primary vacuum pumps (rotary vane and dry scroll), vacuum instrumentation (vacuum control instruments, sensor gauges and meters) and vacuum components (valves, flanges and other mechanical hardware). These products also include helium mass spectrometry and helium-sensing leak detection instruments used to identify and measure leaks in hermetic or vacuum environments. In addition to product sales, we also offer a wide range of services including an exchange and rebuild program, assistance with the design and integration of vacuum systems, applications support and training in basic and advanced vacuum technologies.

### Chemistries and Supplies
We offer a broad range of market specific consumables and supplies to complete customers' analytical workflows from sample preparation through separation and analysis to storage, with the support of our technology platforms. This includes sample preparation consumables such as solid phase extraction ("SPE") and filtration products, self-manufactured GC and LC columns, chemical standards, and instrument replacement parts. Consumable products also include scientific instrument parts and supplies such as filters and fittings for GC systems; xenon lamps and cuvettes for UV-Vis-NIR, fluorescence, FT-IR and raman spectroscopy instruments; and graphite furnace tubes, hollow cathode lamps and specialized sample introduction glassware for our AA, ICP-OES and ICP-MS products.

### Remarketed Instruments
We refurbish and resell certified pre-owned instruments to value-oriented customers who demand Agilent quality and performance at a budget conscious price.

## Life Sciences and Applied Markets Customers
We had approximately 52,000 customers for our life sciences and applied markets business in fiscal year 2024. A significant number of our life sciences and applied markets customers are also customers of our Agilent CrossLab business.

The life sciences and applied markets business is susceptible to seasonality in its orders and revenues primarily related to U.S. and foreign government budgets, chemicals and advanced materials and environmental customers and large pharmaceutical company budgets. Historically, the result is that our first and fourth fiscal quarters tend to deliver the strongest profits for the life sciences and applied markets business. However, general economic trends, new product introductions and competition might overshadow this trend in any given year.

## Life Sciences and Applied Markets Sales, Marketing and Support
The life sciences and applied markets channels focus on the therapeutics and human disease research customer base (pharma, biopharma, CRO, CMO and generics), clinical customer base (high complexity clinical testing labs), emerging life sciences opportunities in life science research institutes and applied markets (chemicals and advanced materials, food, environmental and forensics). We deploy a multi-channel approach, marketing products to our customers through direct sales, electronic commerce, resellers, manufacturers' representatives and distributors. We primarily use direct sales to market our solutions to our pharmaceutical, biopharmaceutical, clinical, life science research and applied market accounts. Sales agents supplement direct sales by providing broader geographic coverage and coverage of smaller accounts. Our active reseller program augments our ability to provide more complete solutions to our customers. We sell our consumable products through distributors, electronic commerce and direct sales.

Our products typically come with standard warranties, and extended warranties are available for additional cost.

## Life Sciences and Applied Markets Manufacturing
Our manufacturing supports our diverse product range and customer-centric focus. We assemble highly configurable products to individual customer orders and make standard products to stock. We employ advanced manufacturing techniques and supply chain management systems to reduce costs and manufacturing cycle times. Our manufacturing process then converts these designs into standard as well as custom products for shipment to customers. We selectively use third parties to provide some supply chain processes for manufacturing, warehousing and logistics. Inside the U.S., we have manufacturing facilities in California, Delaware and Rhode Island. Outside of the U.S., we have manufacturing facilities in Australia, China, Germany, Italy, Malaysia, Netherlands, Singapore and the United Kingdom. We have FDA registered sites in California, Germany and Singapore.

## Life Sciences and Applied Markets Competition
The markets for analytical instruments in which we compete are characterized by evolving industry standards and intense competition. Our principal competitors in the life sciences and applied markets arena include: Danaher Corporation, PerkinElmer Inc., Shimadzu Corporation, Thermo Fisher Scientific Inc. and Waters Corporation. We compete on the basis of product performance, reliability, support quality, applications expertise, global channel coverage and price.

## Diagnostics and Genomics Business
Our diagnostics and genomics business includes the cell analysis, advanced manufacturing partnerships and research and development, pathology, companion diagnostics, reagent partnership, genomics and biomolecular analysis businesses.

Our diagnostics and genomics business is comprised of seven areas of activity providing active pharmaceutical ingredients ("APIs") for oligo-based therapeutics as well as solutions that include reagents, instruments, software and consumables, which enable customers in the clinical and life sciences research areas to interrogate samples at the cellular and molecular level. First, our cell analysis business includes instruments, reagents, software, and labware associated with unique live-cell analysis platforms in addition to mainstream flow cytometers, plate-readers, and plate washers/dispensers which are used across a broad range of applications. Second, our advanced manufacturing partnerships business is a contract and development manufacturing organization that provides services related to and the production of synthesized oligonucleotides under pharmaceutical good manufacturing practices ("GMP") conditions for use as API in a class of drugs that utilize nucleic acid molecules for disease therapy. Together, our BIOVECTRA and nucleic acid solutions businesses offer a broader range of contract and development manufacturing services to our customers. They also provide clinical-to-commercial scale production capabilities focused mainly on mRNA manufacturing. Third, our pathology solutions business is focused on product offerings for cancer diagnostics and anatomic pathology workflows. The broad portfolio of offerings includes immunohistochemistry ("IHC"), in situ hybridization ("ISH"), hematoxylin and eosin ("H&E") staining and special staining. Fourth, we also collaborate with a number of major pharmaceutical companies to develop new potential tissue pharmacodiagnostics, also known as companion diagnostics, which may be used to identify patients most likely to benefit from a specific targeted therapy. Fifth, the reagent partnership business provides clinical flow cytometry r

1. [Diagnostics and Genomics Market](#diagnostics-and-genomics-market)
   1. [The Diagnostics and Clinical Market](#the-diagnostics-and-clinical-market)
   2. [The Pharmaceutical, Biopharmaceutical, CRO & CMO Market](#the-pharmaceutical-biopharmaceutical-cro--cmo-market)
2. [Diagnostics and Genomics Products](#diagnostics-and-genomics-products)
   1. [Pathology](#pathology)
   2. [Cell Analysis](#cell-analysis)
   3. [Bulk Antibodies and Flow Cytometry Reagents](#bulk-antibodies-and-flow-cytometry-reagents)
   4. [Companion Diagnostics](#companion-diagnostics)
   5. [Target Enrichment](#target-enrichment)
   6. [Cytogenetic Research Solutions and Microarrays](#cytogenetic-research-solutions-and-microarrays)
   7. [qPCR Instrumentation and Molecular Biology Reagents](#qpcr-instrumentation-and-molecular-biology-reagents)
   8. [Advanced Manufacturing Partnerships](#advanced-manufacturing-partnerships)
   9. [Automated Electrophoresis and Microfluidics](#automated-electrophoresis-and-microfluidics)
3. [Diagnostics and Genomics Customers](#diagnostics-and-genomics-customers)
4. [Diagnostics and Genomics Sales, Marketing and Support](#diagnostics-and-genomics-sales-marketing-and-support)
5. [Diagnostics and Genomics Manufacturing](#diagnostics-and-genomics-manufacturing)
6. [Diagnostics and Genomics Competition](#diagnostics-and-genomics-competition)

## Diagnostics and Genomics Market

### The Diagnostics and Clinical Market
The diagnostics and clinical market focus within the diagnostics and genomics business is to provide instruments, software, reagents, and consumables that enable customers to perform clinical research and routine testing. Our high-quality, automated pathology tissue staining platforms and solutions are used most heavily by the large labs located in hospitals, medical centers, and reference labs. The market focus is on mature economies primarily in North America, Western Europe and Japan. The mix is changing, however, as emerging markets increase spending on human health. The clinical market for genomics consists of high complexity clinical labs performing patient testing, including "for-profit" reference laboratories, hospital labs, and molecular diagnostic companies. While some labs purchase IVD labeled testing kits, others often develop and validate their own molecular based tests. Analyte Specific Reagents ("ASRs") are often used by these labs. Additionally, our Seahorse, xCELLigence, Novocyte, and BioTek platform technologies are used both stand-alone and in conjunction with mass spectrometry to understand underlying cellular physiology and interactions in normal and diseased states, as well to help understand how new drugs and therapies alter the composition, function, or interaction of cells.

### The Pharmaceutical, Biopharmaceutical, CRO & CMO Market
This market consists of "for-profit" companies which participate across the pharmaceutical value chain in the areas of therapeutic research, discovery & development, clinical trials, manufacturing and quality assurance and quality control. One sub-segment of this market is core and emerging pharmaceutical companies ("pharma"). A second sub-segment includes biopharmaceutical companies ("biopharma"), contract research organizations ("CROs") and contract manufacturing organizations ("CMOs"). Our primary focus is on biopharma working with advanced therapeutic modalities (e.g., cell and gene therapies) where we provide a suite of research tools and companion diagnostic development services. Additionally, we provide active pharmaceutical ingredient ("API") contract development and manufacturing services for oligonucleotide-based therapeutic modalities. With the acquisition of BIOVECTRA, we will offer our pharmaceutical customers even more specialized manufacturing capabilities for targeted therapeutics, from sterile-fill finish to a single source for gene editing solutions.

## Diagnostics and Genomics Products

### Pathology
This area consists of routine clinical solutions for tissue-based cancer diagnostics with solutions that comprise antibodies, reagents, instruments and software targeting both primary and advanced cancer diagnostics. Our CoverStainer and Artisan based product families target primary cancer diagnostics through hematoxylin and eosin staining as well as special stains for additional insights and detection of potentially carcinogenic tissue. Dako Omnis and Autostainer based IHC solution and Instant Quality Fluorescence In Situ Hybridization ("IQFISH") technologies provide advanced tumor typing through investigation of protein and gene expression. These products also include companion diagnostic tests that are used to help identify patients most likely to benefit from a specific targeted therapy.

### Cell Analysis
Our cell analysis tools are used to study cell signaling pathways, general cell function and behavior through metabolic profile analysis, real-time cellular impedance measurements, and traditional cytometry techniques. Characterizing cellular behavior and function is an increasingly critical step in understanding normal behavior versus diseased states, advancements of those diseases, and response to therapies, providing researchers with a more targeted approach for drug discovery and ultimately more effective therapeutics. Our cell analysis portfolio includes cell analysis plate-based assays, flow cytometer, real-time cell analyzer, microplate reader, cell imaging system and related consumables.

### Bulk Antibodies and Flow Cytometry Reagents
In our Bulk Antibodies business we partner with IVD manufacturers, biotechnology and pharmaceutical companies by offering antibodies as raw materials and a range of associated assay development services and solutions. We operate in several areas of clinical relevance for the customers and address multiple technologies such as turbidimetry, gel techniques and chemiluminescence immunoassays. In the area of Flow Cytometry Reagents we provide reagents and kits directly to clinical laboratories working in routine cancer diagnostics, with particular focus on blood cancers.

### Companion Diagnostics
In our companion diagnostics business, we partner with a number of major pharmaceutical companies to develop new potential pharmacodiagnostics, which may be used to identify patients most likely to benefit from a specific targeted therapy. We support pharmaceutical companies during each phase of their drug development process, from early pre-clinical through commercial launch activities. Companion diagnostics has a history of developing clinically relevant and validated tests, with accurate and effective scoring and interpretation guidelines, that enable successful regulatory approvals in our worldwide markets.

### Target Enrichment
We provide a target enrichment portfolio via our SureSelect products, which enables customers to select specific target regions of the genome for sequencing. Customers can customize our products for their regions of interest using the SureDesign software, or they can choose from a range of catalog products, including gene panels for specific applications and Exome designs, which allow analysis of the entire coding sequences of the genome. SureSelect provides a sample prep workflow that can be automated with the Agilent Bravo platform for scalability or leverages the Magnis NGS sample prep ecosystem of instruments and consumables for maximum ease-of-use. These products are used for mutation detection and genotyping. Our solutions also enable clinical labs to identify DNA variants associated with genetic diseases and help direct cancer therapy.

### Cytogenetic Research Solutions and Microarrays
We provide microarrays for comparative genomic hybridization ("CGH"), mostly used by customers in cytogenetic laboratories. The arrays allow customers to detect genome-wide copy number alterations, with high levels of resolution (from entire chromosomal copy number changes to specific microdeletions or duplications). The arrays are offered in many formats allowing the customers to choose from different levels of resolution and number of samples per arrays. Arrays can also be customized using the SureDesign software. In addition to the microarrays, our solution includes reagents for sample processing, hardware for reading the microarrays, and software to help users view the data in a meaningful way. In addition to the CGH portfolio, the cytogenetics solution comprises a line of oligonucleotide probes for fluorescent in situ hybridization ("FISH") called SureFISH. Additionally, we provide a wide range of microarrays to the research market for different types of applications: gene expression, microRNA, methylation, splice variants, and chromatin immunoprecipitation applications. Arrays are offered as catalog designs or customizable designs, with no minimum order size and short delivery time, which differentiates us from other vendors and enables researchers the maximum flexibility in their studies.

### qPCR Instrumentation and Molecular Biology Reagents
Quantitative PCR ("qPCR") or real time PCR is also a standard method used in genomic research facilities to measure the amount of a specific nucleic acid sequence within a sample. There are several applications for qPCR; among the most common are identifying the expression level of a specific gene or calculating the amount of a specific pathogen present in a sample. We offer a complete portfolio of qPCR instruments, as well as specialty enzymes for amplifying difficult sample types. In addition to qPCR enzymes, we offer a wide range of molecular biology reagents including tools for cloning and mutagenesis applications.

### Advanced Manufacturing Partnerships
Our advanced manufacturing partnerships business is a contract manufacturing and development services business with equipment and expertise focused on mid to large scale production of synthesized oligonucleotide APIs under pharmaceutical GMP conditions for a class of drugs that utilize oligonucleotide molecules for disease therapy. These drugs have advanced from single strand DNA molecules to complex, highly modified molecules including antisense, aptamers, double-stranded RNA, and guide RNA. These advancements in the technology have greatly improved the efficacy of delivery and stability of the oligos in-vivo. Our nucleic acid solutions business offers industry leading experience to efficiently advance our customers' oligo drug candidates from clinical trials to commercial scale volumes with a common goal of patient health and safety. With the acquisition of BIOVECTRA, we will offer our pharmaceutical customers even more specialized manufacturing capabilities for targeted therapeutics, from sterile-fill finish to a single source for gene editing solutions.

### Automated Electrophoresis and Microfluidics
Automated electrophoresis is a separation technique for biomolecules such as proteins, peptides and nucleic acids (RNA and DNA) and is used to determine the identity of a molecule by either size or charge. It is widely used as a QC tool to check sample integrity prior to subsequent analysis. Prominent examples are nucleic acid preparation products in front of polymerase chain reaction, NGS and microarrays. More recently, quality control based on automated electrophoresis products has become essential throughout in-vitro transcription ("IVT") mRNA workflows, including vaccine development and therapeutics.

## Diagnostics and Genomics Customers
We had approximately 14,000 customers for our diagnostics and genomics business in fiscal year 2024.

## Diagnostics and Genomics Sales, Marketing and Support
The diagnostics and genomics channels focus on the therapeutics and human disease research customer base (pharma, biopharma, CRO, CMO and generics), clinical customer base (pathology labs and high complexity clinical testing labs) and on emerging life sciences opportunities in life science research institutes. We deploy a multi-channel approach, marketing products to our customers through direct sales, electronic commerce, resellers, manufacturers' representatives and distributors. We primarily use direct sales to market our solutions to our pharmaceutical, biopharmaceutical and clinical accounts. Sales agents supplement direct sales by providing broader geographic coverage and coverage of smaller accounts. Our active reseller program augments our ability to provide more complete solutions to our customers. We sell our consumable products through distributors, telesales, electronic commerce and direct sales. We utilize telesales for more mature product lines, as well as for reorders of reagent products.

## Diagnostics and Genomics Manufacturing
Our manufacturing supports our diverse product range and customer-centric focus. We assemble highly configurable products to individual customer orders and make standard products to stock. We employ advanced manufacturing techniques and supply chain management systems to reduce costs and manufacturing cycle times. We selectively use third parties to provide some supply chain processes for manufacturing, warehousing and logistics. In the U.S., we have manufacturing facilities in California, Colorado, Iowa, Massachusetts, Texas and Vermont. Outside of the U.S., we have manufacturing facilities in Canada, China, Denmark, Germany, Malaysia and Singapore. Our FDA registered sites include California, Colorado, Texas, Vermont and Denmark.

## Diagnostics and Genomics Competition
The markets for diagnostics and genomics analytical products in which we compete are characterized by evolving industry standards and intense competition. Our principal competitors in the diagnostics and genomics arena include: Abbott Laboratories, Affymetrix, Inc., a division of Thermo Fisher Scientific Inc., Avecia, a division of Nitto Denko, Illumina, Inc., Leica Biosystems, Inc., a division of Danaher Corp.


The markets for diagnostics and genomics analytical products in which we compete are characterized by evolving industry standards and intense competition. Our principal competitors in the diagnostics and genomics arena include: Abbott Laboratories, Affymetrix, Inc., a division of Thermo Fisher Scientific Inc., Avecia, a division of Nitto Denko, Illumina, Inc., Leica Biosystems, Inc., a division of Danaher Corporation, Revvity, Inc., Roche Ventana Medical Systems, Inc., a member of the Roche Group, Sartorius and Twist Bioscience Corporation. We compete on the basis of product performance, reliability, support quality, applications expertise, whole solution offering, global channel coverage and price.

## Diagnostics and Genomics Government Regulation

Some of the products the diagnostics and genomics business sells are subject to regulatory approval by the FDA and other regulatory bodies throughout the world. These regulations govern a wide variety of product related activities, from quality management, design and development to labeling, manufacturing, promotion, sales and distribution. We continually invest in our manufacturing infrastructure to gain and maintain certifications necessary for the level of clearance.

## Agilent CrossLab Business

The Agilent CrossLab business spans the entire lab with its extensive services portfolio, which is designed to improve customer outcomes and represents a broad range of offerings designed to serve customer needs across end-markets regardless of instrument manufacturer. The services portfolio includes repairs, parts, maintenance, installations, training, compliance support, software as a service, asset management, consulting and various other custom services to support the customers' laboratory operations.

Custom services are tailored to meet the specific application needs of various industries and to keep instruments fully operational and compliant with the respective industry requirements.

Our Agilent CrossLab business employed approximately 5,400 people as of October 31, 2024.

## Agilent CrossLab Markets

### The Pharmaceutical, Biopharmaceutical, CRO, CDMO & CMO Market

Our services support customers in this market consisting of "for-profit" companies which participate across the pharmaceutical value chain in the areas of therapeutic research, discovery and development, clinical trials, manufacturing and quality assurance and quality control. One sub-segment of this market is core and emerging pharmaceutical companies ("pharma"). A second sub-segment includes biopharmaceutical companies ("biopharma"), contract research organizations ("CROs"), contract development and manufacturing organizations ("CDMOs") and contract manufacturing organizations ("CMOs"). Biopharma companies and, to a somewhat lesser extent, CROs, CDMOs and CMOs typically participate in specific points in the pharmaceutical industry value chain. Additionally, due to the relatively low drug efficacy within oncology, pharma companies are partnering with diagnostic companies to bring validated tests to the market with their new drugs.

### The Academic and Government Market

Our services support customers in this market that consist primarily of "not-for-profit" organizations and include academic institutions, large government institutes and privately funded organizations. The academic and government market plays an influential role in technology adoption and therapeutic developments for pharmaceutical and molecular diagnostics companies. After decades of investment in basic biomedical research by government funding bodies, the focus has widened to include translational research - multidisciplinary scientific efforts directed at accelerating therapy development.

### The Chemicals & Advanced Materials Market

Our services, software and technical support are used throughout the chemicals sector in the development, manufacturing, and quality control of commodity chemicals, specialty and agrochemicals, and fine chemicals. Chemical market customers use our services, software and technical support to maintain, optimize, and enable higher productivity and profitability for labs, and support quality control and compliance with environmental and safety regulations. Additionally, our services, software and technical support are used to support the testing for safety, quality, and compliance across the value chains of advanced materials – including semiconductors, batteries, and specially engineered polymers and polymeric materials. The natural gas and petroleum exploration and refining markets use our services, software and technical support to support quality control, environmental safety reviews, analysis of crude oil composition, and improve their refining processes and quality of products.

### The Environmental & Forensics Market

Our services support the environmental industry customers that perform laboratory and field analysis of chemical pollutants in air, water, soil and solid waste. Environmental industry customers include all levels of government, the industrial and manufacturing sectors, engineering and consulting companies, commercial testing laboratories and colleges and universities. Our services also support drug testing and forensics laboratories that are involved with analyzing evidence associated with crime, screening athletes for performance enhancing drugs, analyzing samples for recreational drugs, or detecting and identifying biological and chemical warfare agents. Customers include local, state, federal, and international law enforcement agencies and commercial testing laboratories.

### The Food Market

Our services support the food production chain, including incoming inspection, new product development, quality control and assurance, and packaging. Our services also support the food safety market in their work to analyze food for concerns ranging from pathogen contamination and genetic modification to species verification and others.

### The Diagnostics and Clinical Market

Our services support clinical diagnostic customers in pathology labs throughout the world.

## Agilent CrossLab Services and Applications

### Services and Support

We offer a wide range of startup, operational, educational and compliance support services for our measurement and data handling systems. Our support services include maintenance, troubleshooting, repair and training for all of our chemical and bioanalytical instrumentation hardware and software products. With advances in digital and virtual support technologies, many of those services can be offered remotely. Special service bundles have also been designed to meet the specific application needs of various industries. As customers continue to outsource laboratory operations and consolidate suppliers, our enterprise services consist of a broad portfolio of integrated laboratory management services including instrument services, lab supply management, asset management, procurement, informatics and scientific services. Advancements in our offering of software and service solutions will help our customers more efficiently operate a digitally connected smart lab that can derive value out of data analytics, artificial intelligence and robotics.

## Agilent CrossLab Customers

We had approximately 50,000 Agilent CrossLab customers in fiscal year 2024. A significant number of our Agilent CrossLab customers are also customers of our life sciences and applied markets business.

The service business is mostly recurring in nature and is less susceptible to market seasonality and industry cycles in comparison to our instrument businesses. The vendor neutral portion of the portfolio allows the business to perform relatively independent from our instrument business.

## Agilent CrossLab Sales, Marketing and Support

We deploy a multi-channel approach, marketing services to our customers through direct sales, electronic commerce, resellers, manufacturers' representatives and distributors. We primarily use direct sales to market our solutions to our large accounts. Sales agents supplement direct sales by providing broader geographic coverage and coverage of smaller accounts. Our active reseller program augments our ability to provide more complete solutions to our customers. Some of our service contract sales are processed by our digital commerce infrastructure. All channels are supported by technical product and application specialists to meet our customers' specific requirements.

We deliver our support services to customers in a variety of ways, including on-site assistance with repair or exchange of returned products, as well as a growing number of remote service delivery options. In addition to the traditional telephone support and on-site service, our teams remotely engage customers through various digital tools and omni-channel platforms. We also offer special industry-focused service bundles that are designed to meet the specific needs of pharmaceutical and biopharmaceutical, advanced materials, environmental and hydrocarbon processing customers to keep instruments fully operational and compliant with the respective industry requirements. Our products typically come with standard warranties, and extended warranties are available for additional cost.

## Agilent CrossLab Manufacturing

Our direct service delivery organization is regionally based and operating in 28 countries.

## Agilent CrossLab Competition

Our principal competitors in the services arena include many of our competitors from the instrument business such as: Danaher Corporation, PerkinElmer Inc., Shimadzu Corporation, Thermo Fisher Scientific Inc. and Waters Corporation, as well as numerous niche service providers. We compete on the basis of reliability, support quality, applications expertise, global channel coverage and price.

## Global Infrastructure Organization

We provide support to our businesses through our global infrastructure organization. This support includes services in the areas of finance, tax, treasury, legal, real estate, insurance services, workplace services, human resources, information technology services, quality and regulatory services, corporate development and other corporate infrastructure expenses. Generally, these organizations are managed from Santa Clara, California, with operations and services provided worldwide. As of October 31, 2024, our global infrastructure organization employed approximately 1,900 people worldwide.

## Agilent Order Fulfillment Organizations

Our order fulfillment and supply chain organization ("OFS") focuses on order fulfillment and supply chain operations in our businesses. OFS provides resources for manufacturing, engineering, logistics, and strategic sourcing to our respective businesses. In general, OFS employees are dedicated to specific businesses and the associated costs are directly allocated to those businesses.

The following discussions of Research and Development, Backlog, Intellectual Property, Materials, Environmental, Regulatory Affairs and Human Capital Management include information common to each of our businesses.

## Research and Development

We anticipate that we will continue to have significant research and development expenditures in order to maintain our competitive position with a continuing flow of innovative, high-quality products and services. Our research and development efforts focus on potential new products and product improvements covering a wide variety of technologies, none of which is individually significant to our operations. Our research seeks to improve on various technical competencies in software, systems and solutions. In each of these research fields, we conduct research that is focused on specific product development for release in the short-term as well as other research that is intended to be the foundation for future products over a longer time-horizon. Most of our product development research is designed to improve products already in production, focus on major new product releases, and develop new product segments for the future. We remain committed to invest significantly in research and development and have focused our development efforts on key strategic opportunities to align our business with available markets and position ourselves to capture market share.

## Backlog

We believe that backlog is not a meaningful indicator of future business prospects for our business segments since a significant portion of our revenue for a given quarter is derived from the current quarter's orders. Therefore, we believe that backlog information is not material to an understanding of our business.

## Intellectual Property

We generate patent and other intellectual property rights covering significant inventions and other innovations in order to create a competitive advantage. While we believe that our licenses, patents and other intellectual property rights have value, in general no single license, patent or other intellectual property right is in itself material. In addition, our intellectual property rights may be challenged, invalidated or circumvented or may otherwise not provide significant competitive advantage.

## Materials

Our life sciences and applied markets, diagnostics and genomics and Agilent CrossLab businesses all purchase materials from thousands of suppliers on a global basis. Some of the parts that require custom design work are not readily available from alternate suppliers due to their unique design or the length of time necessary for design work. Our long-term relationships with suppliers allow us to proactively manage technology road maps and product discontinuance plans and monitor their financial health. To address any potential disruption in our supply chain, we use a number of techniques, including qualifying multiple sources of supply and redesign of products for alternative components. In addition, while we generally attempt to keep our inventory at minimal levels, we do purchase incremental inventory as circumstances warrant to protect the supply chain.

## Environmental

Our research and development, manufacturing and distribution operations involve the use of hazardous substances and are regulated under international, federal, state and local laws governing health and safety and the environment. We apply strict standards for protection of the environment and occupational health and safety to sites inside and outs


Climate change may impact our business by increasing operating costs due to impairments of our facilities and distribution systems, disruptions to our manufacturing processes and additional regulatory requirements. Although we address these potential risks in our business continuity planning, such events could make it difficult for us to deliver products and services to our customers and cause us to incur substantial expense.

In addition to monitoring and managing compliance with environmental regulations, we strive to advance our sustainability practices. In 2021, we announced our goal to achieve net-zero greenhouse gas emissions by 2050. In 2023, we announced near and long-term greenhouse gas emission reduction targets which were validated by the Science Based Targets initiative ("SBTi"). We also aim to provide transparency on our approach to sustainability management through our annual ESG report.

## Regulatory Affairs
A number of our products and services are subject to regulation by the FDA, the U.S. Department of Health and Human Services, the Centers for Medicare and Medicaid Services and certain similar foreign regulatory agencies. These regulations govern a wide variety of product and service related activities, from quality management, design and development to manufacturing, labeling, promotion, sales and distribution. If we fail to comply with FDA regulations and other applicable regulatory requirements or are perceived to potentially have failed to comply, we may face, among other things, warning letters; adverse publicity; investigations or notices of non-compliance, fines, injunctions, and civil or criminal penalties; import or export restrictions; partial suspensions or total shutdown of production facilities or the imposition of operating restrictions; suspension or revocation of our license to operate; increased difficulty in obtaining required FDA clearances or approvals or foreign equivalents; seizures or recalls of our products or those of our customers; or the inability to sell our products. In Europe, the European Union has started to enforce new requirements, known as the EU In Vitro Diagnostic Regulation ("EU IVDR"), which imposes stricter requirements for the marketing and sale of in vitro diagnostics in the European Union. These regulations are more stringent in a variety of areas, including clinical evidence requirements, quality management systems and post-market surveillance activities. The EU IVDR requirements became effective starting in May 2022.

We are subject to laws and regulations governing government contracts, and failure to address these laws and regulations or comply with government contracts could harm our business by leading to a reduction in revenue associated with these customers. We have agreements relating to the sale of our products to government entities and, as a result, we are subject to various statutes and regulations that apply to companies doing business with the government. We are also subject to investigation for compliance with the regulations governing government contracts. A failure to comply with these regulations could result in suspension of these contracts, criminal, civil and administrative penalties or debarment.

We are also subject to various significant international, federal, state and local regulations in the areas of health and safety, packaging, product content, employment, labor and immigration, import/export controls, trade restrictions and anti-competition. Violations of these laws and regulations could result in fines and penalties, criminal sanctions, restrictions on our business conduct and on our ability to offer our products in one or more countries, and could also materially affect our brand, our ability to attract and retain employees, our international operations, our business and our operating results.

In addition, as a global organization, we are subject to data privacy and security laws, regulations, and customer-imposed controls in numerous jurisdictions as a result of having access to and processing confidential, personal, sensitive and/or patient health data in the course of our business. Global privacy laws, including the EU's General Data Protection Regulation ("GDPR"), Brazil's Lei Geral de Protecao de Dados, China's Personal Information Protection Law and Data Security Law, and the California Consumer Privacy Act, apply to our activities involving the processing of personal data, both in relation to our product and service offerings and the management of our workforce. The global proliferation of privacy laws, with governmental authorities around the world passing or considering passing legislative and regulatory proposals concerning privacy and data protection, continues to result in new requirements regarding the handling of personal data, with many such laws imposing significant penalties for non-compliance (including possible fines of up to four percent of total company revenue under the GDPR). Each of these privacy, security and data protection laws and regulations could impose significant limitations and increase our cost of providing our products and services where we process end user personal data and could harm our results of operations and expose us to significant fines, penalties and other damages.

While we believe we are in compliance in all material respects with such laws and regulations, any noncompliance could result in substantial fines or otherwise restrict our ability to operate and thereby have an adverse effect on our financial condition. To date, none has had a material impact on our operations.

## Human Capital Management
As of October 31, 2024, we employed approximately 17,900 persons, of whom approximately 7,000 were based in the Americas, 4,400 in Europe and 6,500 in Asia Pacific. We also leverage temporary workers to provide flexibility for our business and manufacturing needs.

### Mission
Our instruments, software, services, solutions and people provide trusted answers to customers' most challenging questions. Whether we are working with our customers to keep food supplies safe, improve the quality of air, water and soil, or fight cancer with more precise diagnoses and targeted treatments, our employees share a passion and commitment to advancing the quality of life. We believe that our future success largely depends upon our continued ability to attract and retain highly skilled employees in order to fulfill that commitment.

### Engagement
We engage with our employees through consultation, surveys, ad-hoc feedback and reviews. Our executive officers hold all-managers meetings on a quarterly basis to provide business updates and answer questions. We conduct an annual leadership survey that allows employees to provide feedback on leadership effectiveness, culture and job satisfaction. We have an open-door policy where employees are encouraged and empowered to bring issues to management's attention. Employees have regular performance reviews with immediate supervisors. Employee sessions are held regularly to share business and market updates and answer employee questions.

### Diversity and Inclusion
As a global company, much of our success is rooted in the diversity of our teams and our commitment to inclusion. We value diversity at all levels and continue to focus on extending our diversity and inclusion initiatives across our entire workforce, from providing managers transparency of their workforce pay equity to working with managers to develop strategies for building diverse teams to promoting the advancement of leaders from different backgrounds. Agilent is committed to creating a diverse work environment and is proud to be an equal opportunity employer. We believe in an inclusive workforce, where employees from a number of cultures and countries are engaged and encouraged to leverage their collective talents. As of October 31, 2024, approximately 38 percent of our full-time employees were female. Approximately 50 percent of our board is comprised of directors representing underrepresented groups as of the date of this report. We have launched a number of company-wide initiatives including employee-network groups aimed at promoting engagement of traditionally historically underrepresented groups of employees.

### Retention
We provide our employees with competitive salaries and bonuses, opportunities for equity ownership, development programs that enable continued learning and growth and a robust employment package that promotes well-being across all aspects of their lives, including health care, retirement planning and paid time off. Our benefits are offered to eligible employees and comply with local legal requirements. We have a number of programs and policies designed to help employees in our diverse workforce manage their work and personal lives while meeting company objectives for business success, including flexible work arrangements, health and welfare benefits, employee and family assistance plans and parental leave.

### Development
As part of our promotion and retention efforts, we also invest in ongoing leadership development for current and rising managers. Training at Agilent takes several forms: face-to-face classroom experiences, on-the-job learning, virtual classroom events and self-paced e-learning. We are committed to providing an environment in which employees can expand their knowledge, develop new skills, and contribute their best work. Our culture of continuous development instills in our employees the behaviors that bring our values to life every day. We encourage our people to stay up-to-date on current research and technology while enhancing their current skills and growing new skills to meet future needs; we also put special emphasis on training managers at all levels to effectively communicate, role model and reinforce our values and culture.

### Health and Safety
The health and safety of our employees is a top priority for us. Our environmental, health and safety ("EHS") management system provides a framework for assessing and managing risks relating to health and safety. We ensure managers and employees receive periodic workplace safety training and provide wellness programs that contribute to the productivity, health, and well-being of employees. In addition, our crisis management program includes a global tool that enables us to reach, locate and support employees in travel or in crisis areas. We regularly evaluate and review with senior management the performance of our programs and processes.

### Community
Each year our employees throughout the world devote thousands of volunteer hours to community service activities. Our employees may take up to six days of paid time off each year for volunteer activities with charities and organizations. We also support a giving program, which provides employees the opportunity to support a broad range of eligible non-profit organizations in their communities in the areas of health and human services, arts and culture, education and literacy, environment and conservation, and family and civic betterment.

## Information about our Executive Officers
### Henrik Ancher-Jensen
59, has served as our Senior Vice President, Agilent and President, Order Fulfillment and Supply chain since September 2013. From September 2012 to September 2013, Mr. Ancher-Jensen served as our Vice President, Global Product Supply, Diagnostics and Genomics Group. From September 2010 to September 2012 he served as Corporate Vice President, Global Operations of Dako A/S, a Danish diagnostics company, and as Dako's Vice President, Supply Chain and Chief Information Officer from 2006 to September 2010. Prior to joining Dako, he spent more than 15 years in senior management roles and management consulting with Chr. Hansen, Deloitte Consulting and NVE.

### Bret DiMarco
56, has served as our Senior Vice President, Agilent and Chief Legal Officer and Secretary since July 2024. Prior to joining Agilent, he served as the Chief Legal Officer at Pendo.io Incorporated, a privately held company, from September 2022 to June 2024. From June 2006 to July 2022, he held several positions at Coherent, Inc., including Executive Vice President, General Counsel, Chief Legal Officer and Corporate Secretary until its acquisition by II-VI Incorporated after which he served as a Special Advisor to the President until September 2022. Since September 2004, Mr. DiMarco has been an Adjunct Associate Professor of Law at the University of California College of the Law, San Francisco. From October 2023 to present, Mr. DiMarco has been the Chair of the Nasdaq Exchange Nominating Committee and a member of the Nasdaq Exchange Review Council. Mr. DiMarco was previously a member and associate at Wilson Sonsini Goodrich & Rosati, P.C., a multinational law firm.

### Rodney Gonsalves
59, has served as our Vice President, Corporate Controllership and Chief Accounting Officer since May 2015. From September 2009 to May 2015, Mr. Gonsalves served as Vice President and operational CFO for various business groups within the company, most recently for the L


### Michael DiMarco

From 2006 to July 2022, he held several positions at Coherent, Inc., including Executive Vice President, General Counsel, Chief Legal Officer and Corporate Secretary until its acquisition by II-VI Incorporated after which he served as a Special Advisor to the President until September 2022. Since September 2004, Mr. DiMarco has been an Adjunct Associate Professor of Law at the University of California College of the Law, San Francisco. From October 2023 to present, Mr. DiMarco has been the Chair of the Nasdaq Exchange Nominating Committee and a member of the Nasdaq Exchange Review Council. Mr. DiMarco was previously a member and associate at Wilson Sonsini Goodrich & Rosati, P.C., a multinational law firm.

### Rodney Gonsalves

59, has served as our Vice President, Corporate Controllership and Chief Accounting Officer since May 2015. From September 2009 to May 2015, Mr. Gonsalves served as Vice President and operational CFO for various business groups within the company, most recently for the Life Sciences and Applied Markets Group. Prior to that, Mr. Gonsalves served in various capacities for Agilent, including as vice president of Investor Relations, controller, corporate governance and customer financing in Agilent's Global Infrastructure Organization, and controller for the Photonics Systems Business Unit. Before joining Agilent, Mr. Gonsalves held a variety of positions in finance with Hewlett-Packard Company.

### Jonah Kirkwood

44, has served as our Senior Vice President, Agilent and Chief Commercial Officer, Commercial Organization since November 2024. From June 2023 to October 2024, Mr. Kirkwood led Agilent's Global Sales organization for Laboratory Solution Sales as well as the Greater China Sales organization. Mr. Kirkwood led our Commercial Marketing and Operations teams from November 2021 to May 2023. Prior to that, he held various positions in Agilent. Mr. Kirkwood first joined Agilent in 2010 after Agilent acquired Varian.

### Simon May

53, has served as our Senior Vice President, Agilent and President, Diagnostics and Genomics Group since May 2024. Prior to joining Agilent, he served as Executive Vice President and President of the Life Science Group at Bio-Rad Laboratories ("Bio-Rad") from January 2022 to May 2024. During his 10-year tenure at Bio-Rad, Mr. May held various leadership roles including that of Senior Vice President, General Manager of the Digital Biology Group from January 2020 to December 2021 and as Senior Vice President of Global Commercial Operations from October 2015 to January 2020. Before joining Bio-Rad in 2014, Mr. May held positions at Thermo Fisher Scientific for 10 years.

### Padraig McDonnell

53, has served as our President and Chief Executive Officer since May 2024. From February 2024 to May 2024, he served as Senior Vice President, Chief Operating Officer and CEO-elect. Mr McDonnell served as Chief Commercial Officer and President, Agilent CrossLab Group from November 2021 to February 2024. From May 2020 to November 2021, he served as Senior Vice President, Agilent and President, Agilent CrossLab Group. From November 2016 to April 2020, he served as our Vice President and General Manager of the Chemistries and Supplies Division. Prior to that, he served as our Vice President and General Manager of EMEAI Laboratory Solutions Sales. Mr. McDonnell has previously held a variety of positions with Agilent and Hewlett-Packard Company.

### Robert W. McMahon

56, has served as our Senior Vice President, Agilent since August 2018 and as our Chief Financial Officer since September 2018. He previously served as the Chief Financial Officer of Hologic, Inc., a medical technology company from May 2014 to August 2018. Prior to Hologic, Mr. McMahon spent 20 years with Johnson & Johnson most recently as Worldwide Vice President of Finance and Business Development for Ortho Clinical Diagnostics a division of Johnson & Johnson's Medical Device and Diagnostics Group. Since July 2023, Mr. McMahon has served as a member of the Board of Directors of Orasure Technologies, Inc.

### Angelica A. Reimann

54, has served as our Senior Vice President, Agilent and President, Agilent CrossLab Group since February 2024. From August 2021 to February 2024, she served as Vice President and General Manager of Agilent CrossLab Services Division. Ms. Riemann served as Vice President and General Manager of the Chemistries and Supplies Division from May 2020 to August 2021. From March 2019 to May 2020, she was Vice president and General Manager of the Chemistries Division. Prior to March 2019, she held leadership roles in the Chemistries Division, Mass Spectrometry Division and the Americas Field Organization sales organization.

### Mike Zhang

49, has served as our Senior Vice President, Agilent and President, Applied Markets Group since November 2024. From August to November 2024, he served as Vice President and General Manager of the Gas Phase Division within the former Life Sciences and Applied Markets Group. Prior to that, Mr. Zhang was Vice President and General Manager for the Gas Phase Separations Division from January 2020 to August 2024. He previously held various leadership roles in manufacturing over the span of his 22 years at Agilent. He was a manufacturing engineer at Agilent's Shanghai site from March 2002 to December 2019. He also held various leadership roles in Agilent's Order Fulfillment and Supply Chain organization and was named global manufacturing manager for GC operations and general manager of Agilent's Shanghai site in April 2016.

## Investor Information

We are subject to the informational requirements of the Securities Exchange Act of 1934 ("Exchange Act"). Therefore, we file periodic reports, proxy statements and other information with the Securities and Exchange Commission ("SEC"). The SEC maintains an Internet site (https://www.sec.gov) that contains reports, proxy and information statements and other information regarding issuers that file electronically.

Our financial and other information can be accessed at our Investor Relations website. The address is www.investor.agilent.com. We make available, free of charge on our website, electronic copies of our annual report on Form 10-K, quarterly reports on Form 10-Q, current reports on Form 8-K and amendments to those reports filed or furnished pursuant to Section 13(a) or 15(d) of the Exchange Act as soon as reasonably practicable after filing such material electronically or otherwise furnishing it to the SEC.

Our Amended and Restated Bylaws, Corporate Governance Standards, the charters of our Audit and Finance Committee, our Compensation Committee, our Executive Committee and our Nominating/Corporate Governance Committee, as well as our Standards of Business Conduct (including code of ethics provisions that apply to our principal executive officer, principal financial officer, principal accounting officer and senior financial officers) are available on our website at www.investor.agilent.com under "Governance".

## Item 1A. Risk Factors

### Business and Strategic Risks

#### General economic conditions may adversely affect our operating results and financial condition.

Our business is sensitive to negative changes in general economic conditions, both inside and outside the United States. Slower global economic growth, increasing interest rates, inflationary pressures, instability and uncertainty in the markets in which we operate may adversely impact our business resulting in:

- reduced demand and longer sales cycle for our products, delays in the shipment of orders, or increases in order cancellations;
- increased risk of excess and obsolete inventories;
- increased price pressure for our products and services; and
- greater risk of impairment to the value, and a detriment to the liquidity, of our investment portfolio.

#### Our operating results and financial condition could be harmed if the markets into which we sell our products decline or do not grow as anticipated.

Visibility into our markets is limited. Our quarterly sales and operating results are highly dependent on the volume and timing of orders received during the fiscal quarter, which are difficult to forecast and may be cancelled by our customers. In addition, our revenue and earnings forecasts for future fiscal quarters are often based on the expected seasonality of our markets. However, the markets we serve do not always experience the seasonality that we expect as customer spending policies and budget allocations, particularly for capital items, may change. Any decline in our customers' markets or in general economic conditions has in the past and may in the future result in a reduction in demand for our products and services. Also, if our customers' markets decline, we may not be able to collect on outstanding amounts due to us. Such declines could harm our consolidated financial position, results of operations, cash flows and stock price, and could limit our profitability. Also, in such an environment, pricing pressures could intensify. Since a significant portion of our operating expenses is relatively fixed in nature due to sales, research and development and manufacturing costs, if we were unable to respond quickly enough, these pricing pressures could further reduce our operating margins.

#### If we do not introduce successful new products and services in a timely manner to address increased competition through frequent new product and service introductions, rapid technological changes and changing industry standards, our products and services may become obsolete, and our operating results may suffer.

We generally sell our products in industries that are characterized by increased competition through frequent new product and service introductions, rapid technological changes and changing industry standards. Without the timely introduction of new products, services and enhancements, our products and services may become technologically obsolete over time, in which case our revenue and operating results could suffer. The success of our new products and services will depend on several factors, including our ability to:

- properly identify customer needs and predict future needs;
- innovate and develop new technologies, services and applications;
- appropriately allocate our research and development spending to products and services with higher growth prospects;
- successfully commercialize new technologies in a timely manner;
- manufacture and deliver new products in sufficient volumes and on time;
- differentiate our offerings from our competitors' offerings;
- price our products competitively;
- anticipate our competitors' development of new products, services or technological innovations; and
- control product quality in our manufacturing process.

In addition, if we fail to accurately predict future customer needs and preferences or fail to produce viable technologies, we may invest in research and development of products and services that do not lead to significant revenue, which would adversely affect our profitability. Even if we successfully innovate and develop new and enhanced products and services, we may incur substantial costs in doing so, and our operating results may suffer. In addition, promising new products may fail to reach the market or realize only limited commercial success because of real or perceived concerns of our customers. Furthermore, as we collaborate with pharmaceutical customers to develop drugs such as companion diagnostics assays or provide drug components like active pharmaceutical ingredients, we face risks that those drug programs may be cancelled upon clinical trial failures.

#### Economic, political, foreign currency and other risks associated with international sales and operations could adversely affect our results of operations.

Because we sell our products worldwide, our business is subject to risks associated with doing business internationally. We anticipate that revenue from international operations will continue to represent a majority of our total revenue. International revenue and costs are subject to the risk that fluctuations in foreign currency exchange rates could adversely affect our financial results when translated into U.S. dollars for financial reporting purposes. Overall, foreign currency movements for the year ended October 31, 2024, had no overall impact on revenue growth when compared to the same period last year. Typically, when movements in foreign currency exchange rates have a negative impact on revenue, they will also have a positive impact by reducing our costs and expenses. In addition, many of our employees, contract manufacturers, suppliers, job functions, outsourcing activities and manufacturing facilities are located outside the United States. Accordingly, our future results could be harmed by a variety of factors, including:

- interruption to transportation flows for delivery of parts to us and finished goods to our customers;
- ongoing instability or changes in a specific country's or region's political, economic or other conditions, including inflation, recession, interest rate fluctuations and actual or anticipated military or political conflicts, including uncertainties and instability in economic and market conditions caused by pandemics like COVID-19, the current conflicts in Ukraine/Russia and the Middle East, and political and trade uncertainties in the greater China region;
- changes in diplomatic and trade relationships, as well as new tariffs, trade protection measures, import or export licensing requirements, new or different customs duties, trade embargoes and sanctions and other trade barriers;
- tariffs imposed by the U.S. on goods from other countries and tariffs imposed by other countries on U.S. goods, including the tariffs enacted by the U.S. government on various imports from China and by the Chinese government on certain U.S. goods;
- negative consequences from changes in or differing interpretations of laws and regulations, including those related to tax and import/export;
- difficulty in staffing and managing widespread operations;
- differing labor regulations;
- differing protection of intellectual property;
- unexpected changes in regulatory requirements;
- geopolitical uncertainty or turmoil, terrorism and war; and
- impact of public health crises, including pandemics and epidemics, such as COVID-19, on the global economy.

We sell our products into many countries and we also source many components and materials for our products from and manufacture our products in various countries. Future tariffs and tariffs already implemented could have negative impact on our business, results of operations and financial condition. It may be time-consuming and expensive for us to alter our business operations in order to adapt to any such change. Further, additional tariffs, the scope and duration of which, if implemented, remains uncertain, which have been proposed.


- Differing interpretations of laws and regulations, including those related to tax and import/export
- Difficulty in staffing and managing widespread operations
- Differing labor regulations
- Differing protection of intellectual property
- Unexpected changes in regulatory requirements
- Geopolitical uncertainty or turmoil, terrorism and war
- Impact of public health crises, including pandemics and epidemics, such as COVID-19, on the global economy

## Tariffs and Trade Risks

- Future tariffs and tariffs already implemented could have negative impact on our business, results of operations and financial condition
- Time-consuming and expensive to alter business operations to adapt to changes
- Additional tariffs, the scope and duration of which remains uncertain, and potential escalation of a trade war and retaliatory measures could have a material adverse effect

## Centralized Accounting and Tax Processes

- Most accounting and tax processes centralized in India and Malaysia
- Changes in economical, political, health or other conditions in those countries may adversely affect operations, including impairing ability to pay suppliers and collect receivables
- Results of operations and liquidity may be adversely affected, and possible delays in reporting financial results

## Currency Fluctuations

- Significant expenses paid in local currencies
- Hedging programs reduce but do not eliminate impact of currency exchange rate movements
- Fluctuations in exchange rates, including those caused by currency controls, could impact business, operating results and financial condition
- Currency hedging programs involve third-party financial institutions as counterparties, whose weakening or failure may adversely affect hedging programs and financial condition

## Demand Fluctuations

- Demand depends on capital spending policies, research and development budgets, and government funding policies of customers
- Fluctuations in research and development budgets at customer organizations could significantly affect demand
- Timing and amount of revenue from customers relying on government or research funding may vary significantly due to changes in spending authorizations and budgetary priorities

## Inventory Management

- Failure to adjust purchases to reflect market fluctuations or accurately estimate customer demand could adversely affect income
- Difficulty in securing components and parts, leading to shortages, extended lead times, and increased prices
- Non-cancelable purchase commitments and advance payments to suppliers may impact ability to adjust inventory to declining market demands

## Talent Retention and Recruitment

- Future success depends on retaining and hiring key personnel
- Intense competition for certain highly technical specialties in geographic areas
- May become more difficult to hire and retain key employees

## Strategic Initiatives and Restructuring

- Strategic initiatives to adjust cost structure may distract management, slow product and service improvements, and limit ability to increase production
- Delays, unexpected costs, or failure to meet targeted improvements may diminish operational and financial benefits

## Mergers, Acquisitions, and Divestitures

- Financial results may differ from expectations due to difficulties in integrating and collaborating with acquired businesses
- Unexpected significant costs and expenses from transactions
- Potential need to record charges to earnings for impairment of goodwill, intangible assets, or strategic investments
- Successful divestitures depend on effectively transferring liabilities, contracts, facilities, and employees, and reducing fixed costs
- May still retain liabilities associated with divested businesses

## Industry Consolidation

- Consolidation in life sciences industry may result in existing competitors increasing market share through business combinations
- May not be able to compete successfully in increasingly consolidated industries

## Public Health Crises

- Public health crises, such as the COVID-19 pandemic, may adversely impact operations, sales, and delivery of products and services
- Supply chain disruptions and delays
- Difficulty accurately predicting the full extent and duration of the impact

## Internal Controls and Compliance

- Failure to maintain effective internal controls could lead to inaccurate financial reporting and loss of investor confidence
- Customers and the company are subject to various governmental regulations
- Compliance with or changes in regulations may cause significant expenses
- Failure to maintain satisfactory compliance could result in product recalls, manufacturing and distribution cessation, and civil or criminal penalties


- We devote significant resources and time to comply with the internal control over financial reporting requirements of the Sarbanes Oxley Act of 2002.
- We continue to enhance our controls, but cannot be certain that we will be able to prevent future significant deficiencies or material weaknesses.
- Inadequate internal controls could cause investors to lose confidence in our reported financial information, which could negatively affect investor confidence, stock price, and access to capital.

## Compliance with Government Regulations

- Our customers and we are subject to various significant international, federal, state, and local regulations, including in the areas of health and safety, packaging, product content, employment, labor, immigration, import/export controls, trade restrictions, and anti-competition.
- We are also subject to data privacy and security laws, regulations, and customer-imposed controls in numerous jurisdictions due to processing confidential, personal, sensitive, and/or patient health data.
- Global privacy laws, such as the EU's GDPR, Brazil's LGPD, the California Consumer Privacy Act, and China's PIPL and DSL, impose significant penalties for non-compliance.
- We must comply with complex foreign and U.S. laws, such as the U.S. Foreign Corrupt Practices Act and the U.K. Bribery Act, prohibiting corrupt payments to governmental officials.
- Violations of these laws and regulations could result in fines, penalties, criminal sanctions, suspension of government contracts, restrictions on our business, and damage to our brand and ability to attract and retain employees.
- We may be required to incur significant expenses to comply with these regulations or to remedy any violations.
- Failure to comply with government regulations could result in the cessation of our operations, product recalls, fines, and restrictions on our ability to carry on or expand our operations.

## Regulation by the FDA and Similar Agencies

- A number of our products and services are subject to regulation by the FDA and certain similar foreign regulatory agencies.
- Failure to comply with FDA regulations and other applicable regulatory requirements, or being perceived as potentially non-compliant, may result in warning letters, adverse publicity, investigations, fines, injunctions, penalties, import/export restrictions, production restrictions, license suspensions or revocations, product seizures or recalls, and inability to sell our products.
- The global regulatory environment has become increasingly stringent, such as the new EU In Vitro Diagnostic Regulation (EU IVDR) requirements.

## Regulation of Toxic Substances

- Some of our products and related consumables are used in conjunction with chemicals regulated by the EPA under the Toxic Substances Control Act (TSCA).
- Failure to comply with TSCA and similar laws in other countries regarding the manufacture, processing, distribution, and notification of these chemicals could result in civil penalties, criminal prosecution, and restrictions on distributing or marketing our products.

## Government Contracting Compliance

- We derive revenue from sales to governments, which are subject to various procurement laws, regulations, and contract provisions.
- Failure to comply could result in penalties, suspension of government contracts, debarment, termination of contracts, forfeiture of profits, and reduced ability to compete for new contracts.

## Risks from Improper Conduct by Employees, Agents, or Partners

- Our internal controls and compliance systems may not always protect us from acts of employees, agents, or business partners that violate laws, such as the Foreign Corrupt Practices Act and the U.K. Bribery Act.
- Such improper actions could damage our reputation and subject us to civil or criminal investigations, shareholder lawsuits, and significant penalties.
- We also rely on suppliers to adhere to our standards of conduct, and material violations could have a material effect on our business.

## Evolving Corporate Governance and ESG Expectations

- We are subject to changing rules and regulations from various governmental and self-regulatory organizations, as well as evolving investor expectations around corporate governance and environmental, social, and governance (ESG) practices and disclosures.
- Compliance with these evolving requirements and expectations, as well as any risk of non-compliance, could adversely impact us.
- Our ability to achieve our current and future ESG goals is uncertain and subject to numerous risks, including evolving regulatory requirements, stakeholder expectations, workforce diversity, supplier and partner alignment, cost considerations, and technology availability.

## Environmental Contamination Liabilities

- We may be subject to liability for environmental contamination from past and ongoing operations, even if we are indemnified for some of these liabilities.


Achieving our current and future ESG goals is uncertain and remains subject to numerous risks, including evolving regulatory requirements and stakeholder expectations, our ability to recruit and retain a diverse workforce, the availability of suppliers and other business partners that can meet our ESG expectations and standards, cost considerations and the development and availability of cost-effective technologies or resources that support our ESG goals.

## Environmental Contamination

Environmental contamination from past and ongoing operations could subject us to substantial liabilities. Certain properties we have previously owned or leased are undergoing remediation for subsurface contamination. Although we are indemnified for liability relating to the required remediation at some of those properties, we may be subject to liability if these indemnification obligations are not fulfilled. In other cases, we have agreed to indemnify the current owners of certain properties for liabilities related to contamination, including companies with which we have previously been affiliated such as HP, Inc., Hewlett-Packard Enterprise (formerly Hewlett-Packard Company) and Siemens Healthineers (formerly Varian Medical Systems, Inc.). Further, other properties we have previously owned or leased at which we have operated in the past, or for which we have otherwise contractually assumed or provided indemnities, certain actual or contingent environmental liabilities, may or do require remediation. While we are not aware of any material liabilities associated with any potential environmental contamination at any of those properties or facilities, we may be exposed to material liability if environmental contamination at material levels is found to exist. In addition, in connection with the acquisition of certain companies, we have assumed other costs and potential or contingent liabilities for environmental matters. Any significant costs or liabilities could have an adverse effect on results of operations.

## Environmental Laws and Regulations

We are subject to environmental laws and regulations that expose us to a number of risks and could result in significant liabilities and costs. Our current and historical manufacturing and research and development processes and facilities are subject to various foreign, federal, state and local environment protection and health and safety laws and regulations. As a result, we may become subject to liabilities for environmental contamination, and these liabilities may be substantial. Although our policy is to apply strict standards for environmental protection and health and safety at our sites inside and outside the United States, we may not be aware of all conditions that could subject us to liability. Further, in the event that any future climate change legislation would require that stricter standards be imposed by domestic or international environmental regulatory authorities, we may be required to make certain changes and adaptations to our manufacturing processes and facilities. We cannot predict how changes will affect our business operations or the cost of compliance to us, our customers or our suppliers. Failure to comply with these environmental protection and health and safety laws and regulations could result in civil, criminal, regulatory, administrative or contractual sanction, including fines, penalties or suspensions, restrictions on our operations and reputational damage. If we have any violations of, or incur liabilities pursuant to these laws or regulations, our financial condition and operating results could be adversely affected.

## Artificial Intelligence Risks

Issues in the development, deployment, and use of artificial intelligence technologies in our business operations, services and products may result in reputational harm, regulatory action, or legal liability, and any failure to adapt to such technological developments or industry trends could adversely affect the competitiveness of our business.

We are integrating artificial intelligence and machine learning technologies ("AI") into our business operations, products and services, while continuing to explore the opportunities that AI could bring to the company. The use of AI, particularly generative AI, presents opportunities as well as risks that could negatively impact the business. The development, deployment, and use of AI, including within the life sciences industry, is still in its early stages, where the use of insufficiently developed AI technologies and premature deployment practices could result in unintended outcomes that harm the business. AI technologies may be developed using inaccurate, incomplete, flawed or biased algorithms, training methodologies or data, which could result in competitive harm, regulatory penalties, legal liability, or brand or reputational harm. Further, a failure to timely and effectively use or deploy AI and integrate it into new product offerings and services could negatively impact our competitiveness, particularly ahead of evolving industry trends and evolving consumer demands. We may be unable to devote adequate financial resources to develop or acquire new AI technologies and systems in the future.

Use of AI to improve internal business operations, or in the development or provision of products or services, poses risks and challenges. AI can pose risks from an intellectual property, confidential data leakage, data protection, privacy perspective, as well as raise ethical concerns, compliance issues, and security risks. The input of confidential information or trade secrets into AI systems may result in the loss of intellectual property, proprietary rights, or attorney-client privilege in such information or trade secrets. The use of AI technologies for developing products or services may adversely affect or preclude the company's intellectual property rights in such products or services, or may expose the company to liability related to the infringement, misappropriation or other violation of third-party intellectual property. The use of AI technologies with personally identifiable information may also result in legal liability. Further, particularly given the nascent stage of the technology, the use of AI can lead to unintended consequences, including the generation of outputs that appear correct but are factually inaccurate, misleading, or that result in unintended biases and discriminatory outcomes, or are otherwise flawed, which could harm our reputation and business and expose us to risks related to such inaccuracies or errors in these outputs.

Moreover, AI is subject to a dynamic and rapidly evolving legal and regulatory environment, which, without appropriate review, governance and risk management, could expose the company to unforeseen legal or regulatory scrutiny and liabilities. As such, it remains uncertain how AI laws and regulations will impact our business or the associated cost or risks related to compliance therewith or with respect to embedding compliance mechanisms appropriately and effectively into our operations. The use of AI may be subject to new legal or regulatory requirements, the impact of which may be prohibitive or pose further risks from a legal or regulatory action perspective.

## Intellectual Property Risks

### Third-Party Intellectual Property Infringement

Third parties may claim that we are infringing their intellectual property, and we could suffer significant litigation or licensing expenses or be prevented from selling products or services. From time to time, third parties may claim that one or more of our products or services infringe their intellectual property rights. We analyze and take action in response to such claims on a case by case basis. Any dispute or litigation regarding patents or other intellectual property could be costly and time-consuming due to the complexity of our technology and the uncertainty of intellectual property litigation and could divert our management and key personnel from our business operations. A claim of intellectual property infringement could force us to enter into a costly or restrictive license agreement, which might not be available under acceptable terms or at all, could require us to redesign our products, which would be costly and time-consuming, and/or could subject us to significant damages or to an injunction against the development and sale of certain of our products or services. Our intellectual property portfolio may not be useful in asserting a counterclaim, or negotiating a license, in response to a claim of intellectual property infringement. In certain of our businesses, we rely on third-party intellectual property licenses, and we cannot ensure that these licenses will continue to be available to us in the future or can be expanded to cover new products on favorable terms or at all.

### Protecting Our Intellectual Property

Our success depends in large part on our proprietary technology, including technology we obtained through acquisitions. We rely on various intellectual property rights, including patents, copyrights, trademarks and trade secrets, as well as confidentiality provisions and licensing arrangements, to establish, maintain and enforce our proprietary rights. If we do not enforce our intellectual property rights successfully, our competitive position may suffer, which could harm our operating results.

Our pending patent, copyright and trademark registration applications may not be allowed, or competitors may challenge the validity or scope of our patents, copyrights or trademarks. In addition, our patents, copyrights, trademarks and other intellectual property rights may not provide us with a significant competitive advantage.

We may need to spend significant resources monitoring and enforcing our intellectual property rights, and we may not be aware of or able to detect or prove infringement by third parties. Our competitive position may be harmed if we cannot detect infringement and enforce our intellectual property rights quickly or at all. In some circumstances, we may choose to not pursue enforcement because an infringer has a dominant intellectual property position or for other business reasons. In addition, competitors might avoid infringement by designing around our intellectual property rights or by developing non-infringing competing technologies. Intellectual property rights and our ability to enforce them may be unavailable or limited in some countries, which could make it easier for competitors to capture market share and could result in lost revenues. Furthermore, some of our intellectual property is licensed to others which may allow them to compete with us using that intellectual property.

## Operational Risks

### Manufacturing Capacity

Our operating results may suffer if our manufacturing capacity does not match the demand for our products. Because we cannot immediately adapt our production capacity and related cost structures to rapidly changing market conditions, when demand does not meet our expectations, our manufacturing capacity may exceed our production requirements. If during an economic downturn we had excess manufacturing capacity which could occur due to our plans to expand certain manufacturing capacities, then our fixed costs associated with excess manufacturing capacity would adversely affect our gross margins and operating results. If, during a general market upturn or an upturn in one of our segments, we cannot increase our manufacturing capacity to meet product demand, we may not be able to fulfill orders in a timely manner which could lead to order cancellations, contract breaches or indemnification obligations. This inability could materially and adversely limit our ability to improve our results.

### Manufacturing Consolidation

If we are unable to successfully manage the consolidation and streamlining of our manufacturing operations, we may not achieve desired efficiencies, and our ability to deliver products to our customers could be disrupted. Although we utilize manufacturing facilities throughout the world, we have consolidated, and may further consolidate, our manufacturing operations to certain of our facilities to achieve efficiencies and gross margin improvements. Additionally, we typically consolidate the production of products from our acquisitions into our supply chain and manufacturing processes, which are technically complex and require expertise to operate. If we are unable to establish processes to efficiently and effectively produce high quality products in the consolidated locations, we may not achieve the anticipated synergies and production may be disrupted, which could adversely affect our business and operating results.

### Outsourcing Risks

Dependence on contract manufacturing and outsourcing other portions of our supply chain, including logistics and third-party package delivery services, may adversely affect our ability to bring products to market and damage our reputation. Dependence on outsourced information technology and other administrative functions may impair our ability to operate effectively.

As part of our efforts to streamline operations and to manage costs, we outsource aspects of our manufacturing processes and other functions and continue to evaluate additional outsourcing. If our contract manufacturers or other outsourcers fail to perform their obligations in a timely manner or at satisfactory quality levels, our ability to bring products to market and our reputation could suffer. For example, during a market upturn, our contract manufacturers may be unable to meet our demand requirements, which may preclude us from fulfilling our customers' orders on a timely basis. The ability of these manufacturers to perform is largely outside of our control. If one or more of the third-party package delivery or other logistics providers we use experiences a significant disruption in services or institutes a significant price increase, we may have to seek alternative providers, which could result in increased costs, and/or delay the delivery of our products. Additionally, changing or replacing our contract manufacturers, logistics providers or other outsourcers could cause disruptions or delays. In addition, we outsource significant portions of our information technology ("IT") and other administrative functions. Since IT is critical to our operations, any failure to perform on the part of our IT providers could impair our ability to operate effectively. In addition to the risks outlined above, problems with manufacturing or IT outsourcing could result in lower revenue and unexecuted efficiencies and impact our results of operations and our stock price.

### Catastrophic Loss

If we suffer a loss to our factories, facilities or distribution system due to catastrophe, our operations could be seriously harmed. Our factories, facilities and distribution system are subject to catastrophic loss due to fire, flood, terrorism, public health crises, increasing severity or frequency of extreme weather events, or other climate-change related events.


### Disruptions in our operations or supply chain could adversely affect our business

Our manufacturing operations and supply chain are subject to disruptions that could adversely affect our business, including:

- Disruptions or delays in the delivery of our products due to changes or problems with our contract manufacturers, logistics providers or other outsourcers.
- Failures or disruptions in our information technology ("IT") systems and other administrative functions that are outsourced.
- Catastrophic losses to our factories, facilities or distribution system due to fire, flood, terrorism, public health crises, extreme weather events, or other climate-related risks.

These disruptions could result in lower revenue, unexecuted efficiencies, and impact our financial results and stock price.

### Cybersecurity risks could disrupt our operations and adversely affect our business

We rely on several centralized IT systems to provide products and services, manage operations, and protect confidential information. These systems may be susceptible to damage, disruptions or shutdowns due to various threats. Significant disruptions or security breaches could result in the loss of sales, customers, and sensitive data, as well as increased compliance costs and reputational damage.

## Financial and Tax Risks

### Pension plan risks could adversely affect our results

Our retirement and post-retirement pension plans are subject to financial market risks, such as decreases in interest rates and investment losses. These can increase our funding obligations and adversely impact our results of operations and cash flows.

### Tax changes and examinations could impact our financial results

Changes in tax laws, unfavorable tax examinations, or additional tax liabilities could have a material adverse effect on our results. We also benefit from tax incentives in certain jurisdictions, which could be impacted if the incentives are not renewed or if we cannot satisfy the conditions.

### Debt levels and covenants could restrict our operations

We have significant outstanding debt and may incur additional debt in the future. This debt could increase our vulnerability to downturns, limit our flexibility, and require the dedication of cash flows to service the debt. Our credit facility and debt covenants also impose restrictions on our operations.

### Dividend payments are not guaranteed

While we have paid quarterly dividends since 2012, the timing, declaration, amount and payment of any future dividends are at the discretion of our Board of Directors and may be impacted by various factors.

### Cash investments and liquidity could be adversely affected

Disruptions in the banking industry and credit markets could impact the value of our cash investments or impair our liquidity, adversely affecting our operating results and financial condition.


We maintain cybersecurity policies that articulate Agilent's expectations and requirements regarding technology use, data privacy, risk management, and incident management. Regular exercises and assessments against recognized cybersecurity frameworks are conducted to improve the effectiveness of our processes. These are conducted by third party organizations in addition to internal audit teams. Cybersecurity is considered the responsibility of every Agilent employee, with regular education and best practice sharing to raise awareness of threats. Layered controls are implemented to prevent and detect cybersecurity threats, with policies and processes designed to provide timely notifications and compliance with legal requirements. These include controls to assess third party suppliers and their services.

## Governance and Oversight

Our cybersecurity program under the Chief Information Officer ("CIO") is led by our Chief Information Security Officer ("CISO"). The Board of Directors delegates oversight of cybersecurity risks to the Audit Committee, which receives updates from the CISO and CIO at least annually. Cybersecurity is integrated into the risk management process for the company through various mechanisms, including quarterly business reviews, annual budget planning, and linkage to the Enterprise Risk Management ("ERM") process.

As of the date of this report, we do not believe any risks from cybersecurity threats have materially affected Agilent, including our business strategy, results of operations, or financial condition. However, we can provide no assurance that there will not be any incidents in the future or that they will not materially affect us as outlined in Item 1A. Risk Factors.

## Item 2. Properties

As of October 31, 2024, we owned or leased a total of approximately 6.8 million square feet of space worldwide. Of that, we owned approximately 5.3 million square feet and leased the remaining 1.5 million square feet. Our sales and support facilities occupied a total of approximately 0.5 million square feet. Our manufacturing plants, R&D facilities and warehouse and administrative facilities occupied approximately 6.3 million square feet. All of our businesses share sales offices throughout the world.

### Life Sciences & Applied Markets Business

Our life sciences and applied markets business has manufacturing and R&D facilities in Australia, China, Germany, Italy, Japan, Malaysia, Netherlands, Singapore, United Kingdom and the United States.

### Diagnostics and Genomics Business

Our diagnostics and genomics business has manufacturing and R&D facilities in Belgium, Canada, China, Denmark, Germany, Malaysia and the United States.

### Agilent CrossLab Business

Our direct service delivery organization is regionally based and operating in 28 countries.

## Item 3. Legal Proceedings

We are involved in lawsuits, claims, investigations and proceedings, including, but not limited to, intellectual property, commercial, real estate, environmental and employment matters, which arise in the ordinary course of business. There are no matters pending that we currently believe are probable and reasonably possible of having a material impact to our business, consolidated financial condition, results of operations or cash flows.

## Item 4. Mine Safety Disclosures

Not applicable.

## PART II

### Item 5. Market for the Registrant's Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities

Our common stock is listed on the New York Stock Exchange with the ticker symbol "A". As of December 2, 2024, there were 16,806 common stockholders of record.

The information required by this item with respect to equity compensation plans is included under the caption "Equity Compensation Plans" in our Proxy Statement for the Annual Meeting of Stockholders to be held March 13, 2025, to be filed with the Securities and Exchange Commission pursuant to Regulation 14A, and is incorporated herein by reference.

#### STOCK PRICE PERFORMANCE GRAPH

The graph below shows the cumulative total stockholder return on our common stock with the cumulative total return of the S&P 500 Index and our peer group, consisting of all companies in the Health Care and Materials Indexes of the S&P 500, assuming an initial investment of $100 on October 31, 2019 and the reinvestment of all dividends.

Agilent's stock price performance shown in the following graph is not indicative of future stock price performance. The data for this performance graph was compiled for us by Standard and Poor's.

#### INDEXED RETURNS

| Base Period | Company Name / Index | 10/31/2019 | 10/31/2020 | 10/31/2021 | 10/31/2022 | 10/31/2023 | 10/31/2024 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Agilent Technologies | 100 | 135.93 | 210.90 | 186.45 | 140.29 | 178.06 |
| S&P 500 | 100 | 109.71 | 156.79 | 133.88 | 147.46 | 203.52 |
| Peer Group | 100 | 112.04 | 153.60 | 153.06 | 149.45 | 181.18 |

#### ISSUER PURCHASES OF EQUITY SECURITIES

| Period | Total Number of Shares of Common Stock Purchased(1) | Weighted Average Price Paid per Share of Common Stock(2) | Total Number of Shares of Common Stock Purchased as Part of Publicly Announced Plans or Programs(1) | Maximum Approximate Dollar Value of Shares of Common Stock that May Yet Be Purchased Under the Plans or Programs (in millions)(1) |
| --- | --- | --- | --- | --- |
| August 1, 2024 through August 31, 2024 | 804,464 | $140.19 | 804,464 | $596 |
| September 1, 2024 through September 30, 2024 | 1,240,953 | $137.98 | 1,240,953 | $425 |
| October 1, 2024 through October 31, 2024 | 366,113 | $139.28 | 366,113 | $374 |
| Total | 2,411,530 | $138.91 | 2,411,530 | |

### Item 6. [Reserved]

### Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations

The following discussion should be read in conjunction with the consolidated financial statements and notes thereto included elsewhere in this Annual Report on Form 10-K. This report contains forward-looking statements including, without limitation, statements regarding growth opportunities, including for and in our end markets, new product and service introductions, the position and strength of our businesses, products and services, market demand for and adoption of our products and solutions, the ability of our products and solutions to address customer needs and meet industry requirements, our focus on enhancing our customers' experience, delivering differentiated product solutions and driving productivity improvements, our investments, including in manufacturing infrastructure, research and development and expanding and improving our applications and solutions portfolios, expanding our position in developing countries and emerging markets, our contributions to our defined benefit plans, our hedging programs and other actions to offset the effects of foreign currency and interest rate movements, our future effective tax rate, unrecognized tax benefits, reimbursement incentives, our ability to satisfy our liquidity requirements, including through cash generated from operations, the potential impact of adopting new accounting pronouncements, indemnification obligations, our sales, our purchase commitments, our capital expenditures, the integration, effects and timing of our acquisitions and other transactions, expense reduction and other results from our restructuring programs and other cost saving initiatives, our stock repurchase program and dividends, macroeconomic and market conditions, the recovery and health of our end markets, seasonality, mix, future financial results, our operating margin, our geographical diversification, interest rates, inflationary pressures and local regulations and restrictions, that involve risks and uncertainties. Our actual results could differ materially from the results contemplated by these forward-looking statements due to various factors, including those discussed in Part I Item 1A and elsewhere in this Form 10-K.

## Overview and Executive Summary

Agilent Technologies Inc. ("we", "Agilent" or the "company"), incorporated in Delaware in May 1999, is a global leader in life sciences, diagnostics and applied chemical markets, providing application focused solutions that include instruments, software, services and consumables for the entire laboratory workflow.

### New Segment Structure

In the first quarter of fiscal year 2024, we announced a change in our operating segments to move our cell analysis business from our life sciences and applied markets segment to our diagnostics and genomics operating segment in order to further strengthen growth opportunities for both organizations. Following this reorganization, we continue to have three business segments comprised of life sciences and applied markets, diagnostics and genomics and Agilent CrossLab, each of which continues to comprise a reportable segment. We began reporting under this new structure with the Quarterly Report on Form 10-Q for the period ended January 31, 2024. All historical financial segment information has been recast to conform to this new presentation in our consolidated financial statements and accompanying notes. There was no change to our Agilent CrossLab business segment.

### Acquisition

On September 20, 2024, we acquired 100 percent of the stock of BIOVECTRA for total consideration of $915 million in cash. The acquisition expands our contract development and manufacturing organization. As a result of the acquisition, BIOVECTRA became a wholly-owned subsidiary of Agilent. The acquisition has been accounted for in accordance with the authoritative accounting guidance, and the results of BIOVECTRA are included in Agilent's consolidated financial statements from the date of acquisition.

### Senior Notes

#### 2027 Senior Notes

On September 9, 2024, we issued an aggregate principal amount of $600 million in senior notes ("2027 senior notes"). The 2027 senior notes were issued at 99.866% of their principal amount. The notes will mature on September 9, 2027, and bear interest at a fixed rate of 4.20% per annum. The interest is payable semi-annually on March 9th and September 9th of each year and payments will commence on March 9, 2025.

#### 2034 Senior Notes

On September 9, 2024, we issued an aggregate principal amount of $600 million in senior notes ("2034 senior notes"). The 2034 senior notes were issued at 99.638% of their principal amount. The 2034 senior notes will mature on September 9, 2034, and bear interest at a fixed rate of 4.75% per annum. The interest is payable semi-annually on March 9th and September 9th of each year and payments will commence on March 9, 2025.

### Actual Results

Agilent's net revenue of $6,510 million in 2024 decreased 5 percent when compared to 2023. Foreign currency movements for 2024 had no overall impact on revenue growth when compared to 2023. Net revenue declined in our life sciences and applied markets and diagnostics and genomics segments, mostly in the pharmaceutical market, due primarily to the overall pressures on our customers' capital expenditure spending which continued in 2024. Revenue declines were partially offset by revenue growth in our Agilent CrossLab segment. Revenue in the life sciences and applied markets business decreased 8 percent in 2024 when compared to 2023. Foreign currency movements had no overall impact on revenue growth in 2024 when compared to 2023. Revenue in the diagnostics and genomics business decreased 6 percent in 2024 when compared to 2023. Foreign currency movements had no overall impact on revenue growth in 2024 when compared to 2023. Revenue in the Agilent CrossLab business increased 5 percent in 2024 when compared to 2023. Foreign currency movements had no overall impact on revenue growth in 2024 when compared to 2023.

Agilent's net revenue of $6,833 million was slightly down in 2023 when compared to 2022. Foreign currency movements for 2023 had an overall unfavorable impact on revenue growth of 2 percentage points when compared to 2022. Net revenue declined in our life sciences and applied markets segment, in the pharmaceutical market and in the Asia Pacific region primarily related to weaker demand in China and an overall pressure on our customers' capital expenditures compared to 2022. The net revenue decline was partially offset by revenue growth from our other segments primarily in Agilent CrossLab. Revenue in the life sciences and applied markets business decreased 3 percent in 2023 when compared to 2022. Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022. Revenue in the diagnostics and genomics business decreased 1 percent in 2023 when compared to 2022. Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022. Revenue in the Agilent CrossLab business increased 8 percent in 2023 when compared to 2022. Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022.

Net income was $1,289 million in 2024 compared to net income of $1,240 million and $1,254 million in 2023 and 2022, respectively. Net income in 2024 was impacted by cost-saving initiatives and higher interest income. Net income in 2023 was impacted by the asset impairment charges primarily related to the exit of our Resolution Bioscience business and lower tax expense. Net income in 2022 was impacted by higher sales v

- Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022.
- Revenue in the diagnostics and genomics business decreased 1 percent in 2023 when compared to 2022.
- Revenue in the Agilent CrossLab business increased 8 percent in 2023 when compared to 2022.

## Net Income
- Net income was $1,289 million in 2024 compared to net income of $1,240 million and $1,254 million in 2023 and 2022, respectively.
- Net income in 2024 was impacted by cost-saving initiatives and higher interest income.
- Net income in 2023 was impacted by the asset impairment charges primarily related to the exit of our Resolution Bioscience business and lower tax expense.
- Net income in 2022 was impacted by higher sales volume partially offset by supply chain, logistics and inflationary pressures increasing our costs.

## Cash and Cash Equivalents
- As of October 31, 2024 and 2023, we had cash and cash equivalents balances of $1,329 million and $1,590 million, respectively.

## Share Repurchase Programs
### 2021 Repurchase Program
- During the year ended October 31, 2022, we repurchased and retired 8.4 million shares for $1,139 million under this authorization.
- During the year ended October 31, 2023, we repurchased and retired 661,739 shares for $99 million, excluding excise taxes, under this authorization.
- On March 1, 2023, the 2021 repurchase program was terminated and the remaining authorization of $339 million expired.

### 2023 Repurchase Program
- On January 9, 2023, we announced a new share repurchase program (the "2023 repurchase program") authorizing the purchase of up to $2.0 billion, excluding excise taxes, of our common stock.
- During the year ended October 31, 2023, we repurchased and retired 3.9 million shares for $476 million, excluding excise taxes, under this authorization.
- During the year ended October 31, 2024, we repurchased and retired 8.4 million shares for $1,150 million, excluding excise taxes, under this authorization.
- As of October 31, 2024, we had remaining authorization to repurchase up to approximately $374 million of our common stock under the 2023 repurchase program.

### 2024 Repurchase Program
- On May 29, 2024, we announced a new share repurchase program (the "2024 repurchase program") authorizing the purchase of up to $2.0 billion, excluding excise taxes, of our common stock.
- The 2024 repurchase program became effective on August 1, 2024 and will commence upon the termination of our 2023 repurchase program.

## Dividends
- During the year ended October 31, 2024, cash dividends of $0.944 per share, or $274 million were declared and paid.
- During the year ended October 31, 2023, cash dividends of $0.900 per share, or $265 million were declared and paid.
- During the year ended October 31, 2022, cash dividends of $0.840 per share, or $250 million were declared and paid.
- On November 20, 2024, we declared a quarterly dividend of $0.248 per share of common stock, or approximately $71 million which will be paid on January 22, 2025, to shareholders of record as of the close of business on December 31, 2024.

## Looking Forward
- Our primary focus remains on enhancing our customers' experience, delivering differentiated product solutions and driving productivity improvements.
- We anticipate a gradual and steady recovery in the short-term, despite constrained customer capital budgets.
- We remain optimistic about the long-term health of our key end markets.
- We will continue to mitigate the impact of inflationary pressures through targeted pricing strategies and various other cost-saving initiatives.

## Critical Accounting Policies and Estimates
- Our critical accounting policies are those that affect our financial statements materially and involve difficult, subjective or complex judgments by management.
- These policies include revenue recognition, inventory valuation, retirement and post-retirement plan assumptions, valuation of goodwill and purchased intangible assets, restructuring and accounting for income taxes.


## Discount Rate

The discount rate is used to determine the present value of future benefit payments at the measurement date - October 31 for both U.S. and non-U.S. plans. For 2024 and 2023, the U.S. discount rates were based on the results of matching expected plan benefit payments with cash flows from a hypothetically constructed bond portfolio. In 2024, discount rates for the U.S. defined benefit plans and post-retirement benefit plans decreased compared to the previous year due to the decrease in the corporate bond rates. For 2024 and 2023, the discount rates for non-U.S. defined benefit plans were generally based on published rates for high quality corporate bonds and in 2024, mostly decreased compared to the previous year. If we had changed our discount rate by 1 percent, the impact would have been approximately $1 million on U.S. defined benefit plans and post-retirement benefit plans expense and $11 million on non-U.S. defined benefit plans expense for the year ended October 31, 2024. Lower discount rates usually increase present values of the pension benefit obligation and subsequent year pension expense; higher discount rates usually decrease present values of the pension benefit obligation and subsequent year pension expense.

## Amortization of Gains and Losses

The company uses alternate methods of amortization as allowed by the authoritative guidance which amortizes the actuarial gains and losses on a consistent basis for the years presented. For U.S. defined benefit plans, gains and losses are amortized over the average future lifetime of participants using the corridor method. For most non-U.S. defined benefit plans and U.S. post-retirement benefit plans, gains and losses are amortized over the average remaining future service period or remaining lifetime of participants depending upon the plan, using a separate layer for each year's gains and losses.

## Asset Allocation

In the U.S., target asset allocations for our retirement and post-retirement benefit plans were approximately 50 percent to equities and approximately 50 percent to fixed income investments as of October 31, 2024. Our Deferred Profit-Sharing Plan target asset allocation is approximately 60 percent to equities and approximately 40 percent to fixed income investments. Approximately 1 percent of the retirement and post-retirement plans consists of limited partnerships. Outside the U.S., our target asset allocation (excluding annuity contracts in the U.K.) ranges from zero percent to 60 percent to equities, from 38 percent to 100 percent to fixed income investments, and from zero to 25 percent to real estate, depending on the plan. All plans' assets are broadly diversified. Due to fluctuations in equity and bond markets, our actual allocations of plan assets at October 31, 2024, may differ from the target allocation. Our policy is to bring the actual allocation in line with the target allocation.

## Expected Long-Term Return on Plan Assets

The expected long-term return on plan assets is estimated using current and expected asset allocations as well as historical and expected returns. Plan assets are valued at fair value. If we had changed our estimated return on assets by 1 percent, the impact would have been $4 million on U.S. defined benefit plans and post-retirement benefit plans expense and $8 million on non-U.S. defined benefit plans expense for the year ended October 31, 2024. The total net periodic pension and post-retirement benefit costs recorded were a $9 million benefit in 2024, $6 million expense in 2023 and $2 million benefit in 2022. These costs included a loss on settlement of $2 million, $4 million and $4 million, for the years ended October 31, 2024, 2023 and 2022, respectively.

# Goodwill and Purchased Intangible Assets

## Goodwill Impairment

We assess our goodwill and purchased intangible assets for impairment annually or whenever events or changes in circumstances indicate that the carrying value may not be recoverable. Under the authoritative guidance, we have the option to perform a qualitative assessment to determine whether further impairment testing is necessary. The accounting standard gives an entity the option to first assess qualitative factors to determine whether performing the quantitative test is necessary. If an entity believes, as a result of its qualitative assessment, that it is more-likely-than-not (i.e., greater than 50% chance) that the fair value of a reporting unit is less than its carrying amount, the quantitative impairment test will be required. Otherwise, no further testing will be required.

At the beginning of fiscal year 2024, in connection with the change in our segment reporting, we assessed goodwill impairment for our three reporting units which consisted of our three segments: life sciences and applied markets, diagnostics and genomics and Agilent CrossLab. We performed a quantitative test for goodwill impairment of the three reporting units as of November 1, 2023, due to the change in our segment structure. As of November 1, 2023, there was no impairment of goodwill.

In fiscal year 2024, we again assessed goodwill impairment for our three reporting units which consisted of our three segments: life sciences and applied markets, diagnostics and genomics and Agilent CrossLab. We performed a qualitative test for goodwill impairment of the three reporting units as of September 30, 2024, our annual impairment test date. Based on the results of our qualitative testing, there was no impairment of goodwill as of September 30, 2024. Each quarter we review the events and circumstances to determine if goodwill impairment is indicated. There was no impairment of goodwill during the years ended October 31, 2024, 2023 and 2022.

## Purchased Intangible Assets

Purchased intangible assets consist primarily of acquired developed technologies, proprietary know-how, trademarks, and customer relationships and are amortized using the best estimate of the asset's useful life that reflects the pattern in which the economic benefits are consumed or used up or a straight-line method ranging from 6 months to 15 years. Our determination of the fair value of the intangible assets acquired involves the use of significant estimates and assumptions.  Specifically, our determination of the fair value of the developed product technology and in-process research and development ("IPR&D") acquired involves significant estimates and assumptions related to revenue growth rates and discount rates. Our determination of the fair value of customer relationships acquired involves significant estimates and assumptions related to revenue growth rates, discount rates, and customer attrition rates. Our determination of the fair value of the trade name acquired involves the use of significant estimates and assumptions related to revenue growth rates, royalty rates and discount rates. We value backlog using the discounted cash flows based on the estimated revenue from pending orders. We value license agreements based on the expected future cash receipts from license agreements, discounted to present value over the term of the agreement. We believe that the fair value assigned to the assets acquired and liabilities assumed are based on reasonable assumptions and estimates that marketplace participants would use. Actual results could differ materially from these estimates. IPR&D is initially capitalized at fair value as an intangible asset with an indefinite life and assessed for impairment thereafter. When the IPR&D project is complete, it is reclassified as an amortizable purchased intangible asset and is amortized over its estimated useful life. If an IPR&D project is abandoned, we will record a charge for the value of the related intangible asset to our consolidated statement of operations in the period it is abandoned.

During fiscal year 2024, we recorded an impairment of in-process research and development of $6 million in research and development in the consolidated statement of operations related to a project in our life sciences and applied markets segment.  There were no impairments of indefinite-lived intangible assets during fiscal years 2023 and 2022.

# Restructuring

The main components of our restructuring plan are related to workforce reductions, consolidation of excess leased facilities and site closures. Workforce reduction charges are accrued when payment of benefits becomes probable that the employees are entitled to the severance and the amounts can be estimated. Consolidation of facilities costs primarily consists of accelerated depreciation of right-of-use assets classified as held and used. In accordance with the accounting guidance, it was determined that certain assets had been abandoned, and an assessment was made of the remaining useful lives and potential alternative uses. If the amounts and timing of cash flows from restructuring activities are significantly different from what we have estimated, the actual amounts of restructuring and other related charges could be materially different, either higher or lower, than those we have recorded.

# Accounting for Income Taxes

We must make certain estimates and judgments in determining income tax expense for financial statement purposes. These estimates and judgments occur in the calculation of tax credits, benefits and deductions, and in the calculation of certain tax assets and liabilities which arise from differences in the timing of recognition of revenue and expense for tax and financial statement purposes, as well as interest and penalties related to uncertain tax positions. Significant changes to these estimates may result in an increase or decrease to our tax provision in a subsequent period. On a quarterly basis, we provide for income taxes based upon an estimated annual effective tax rate.  The effective tax rate is highly dependent upon the geographic composition of worldwide earnings, tax regulations governing each region, availability of tax credits and the effectiveness of our tax planning strategies.  We monitor the changes in many factors and adjust our effective income tax rate on a timely basis.  If actual results differ from these estimates, this could have a mater

- Significant management judgment is required in determining income taxes
- Estimates and judgments occur in the calculation of tax credits, benefits and deductions, and in the calculation of certain tax assets and liabilities
- Significant changes to these estimates may result in an increase or decrease to the tax provision in a subsequent period
- The effective tax rate is highly dependent upon the geographic composition of worldwide earnings, tax regulations, availability of tax credits, and the effectiveness of tax planning strategies
- Management monitors changes in many factors and adjusts the effective income tax rate on a timely basis
- Significant judgment is also required in determining whether deferred tax assets will be realized in full or in part
- The calculation of tax liabilities involves dealing with uncertainties in the application of complex tax law and regulations in multiple jurisdictions

## Adoption of New Pronouncements
- See Note 2, "New Accounting Pronouncements," for a description of new accounting pronouncements

## Restructuring and Other Related Costs

### Summary of Restructuring Plans
- In fiscal years 2024 and 2023, the company announced restructuring plans to reduce costs and expenses in response to macroeconomic conditions
- The restructuring plans impact all three business segments
- The costs were not allocated to the business segments, but each segment will benefit from the future cost savings

### Fiscal Year 2024 Plan ("FY24 Plan")
- Initiated in the third quarter of fiscal year 2024 to further reduce costs and expenses
- Includes a reduction of total headcount by approximately 500 regular employees, representing 3% of the global workforce
- Recorded $72 million in restructuring expenses in fiscal year 2024
- Expected to result in the reduction of approximately $100 million in annual cost of sales and operating expenses

### Fiscal Year 2023 Plan ("FY23 Plan")
- Initiated in the fourth quarter of fiscal year 2023 to reduce costs and expenses
- Included a reduction of total headcount by approximately 400 regular employees, representing 2% of the global workforce, and the consolidation of excess facilities
- Recorded $46 million in restructuring expenses in 2023 and $4 million in 2024
- Expected to result in the reduction of approximately $80 million in annual cost of sales and operating expenses

## Foreign Currency
- Revenues, costs and expenses, monetary assets and liabilities, and equity are exposed to changes in foreign currency exchange rates
- Foreign currency movements had no overall impact on revenue growth in 2024 and an unfavorable impact of 2 percentage points in 2023
- The company hedges revenues, expenses and balance sheet exposures that are not denominated in the functional currencies of its subsidiaries on a short-term and anticipated basis

## Results from Operations

### Net Revenue
- Total net revenue decreased 5% in 2024 compared to 2023, with no overall impact from foreign currency movements
- Total net revenue was slightly down in 2023 compared to 2022, with an unfavorable impact of 2 percentage points from foreign currency movements
- Product revenue decreased 7% in 2024 and 3% in 2023, primarily due to declines in the pharmaceutical market
- Services and other revenue increased 3% in 2024 and 7% in 2023, reflecting growth in contract repair and preventative maintenance services

### Net Revenue By Segment
- Detailed net revenue information for the company's three business segments (Life Sciences and Applied Markets, Diagnostics and Genomics, and Agilent CrossLab) is provided for fiscal years 2024, 2023, and 2022.


Services and other revenue reflected strong growth from contract repair and preventative maintenance services partly offset by declines in installation services related to the decline of the product revenues.

Services and other revenue increased 7 percent in 2023 as compared to 2022. Service revenue increases reflected strong growth from contract repair services, consultative services, per incident repair and maintenance services, and relocation services in all key end markets.

## Net Revenue By Segment

Years Ended October 31,
2024 over 2023 Change
2023 over 2022 Change
2024
2023
2022
(in millions)

Net revenue by segment:
Life sciences and applied markets
$
3,215
$
3,510
$
3,630
(8)%
(3)%
Diagnostics and genomics
$
1,651
$
1,755
$
1,766
(6)%
(1)%
Agilent CrossLab
$
1,644
$
1,568
$
1,452
5%
8%
Total net revenue
$
6,510
$
6,833
$
6,848
(5)%
—

Revenue in the life sciences and applied markets business decreased 8 percent in 2024 when compared to 2023. Foreign currency movements had no overall impact on revenue growth in 2024 when compared to 2023. For the year ended October 31, 2024, revenue declined in all of our end markets. We saw a significant decline in revenue in the pharmaceutical, chemical and advanced materials, food and academia and government markets when compared to 2023. Revenue in the life sciences and applied markets business decreased 3 percent in 2023 when compared to 2022. Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022. For the year ended October 31, 2023, we saw a significant decline in revenue in the pharmaceutical and the diagnostics and clinical markets partially offset by strong growth in the academia and government market when compared to 2022.

Revenue in the diagnostics and genomics business decreased 6 percent in 2024 when compared to 2023. Foreign currency movements had no overall impact on revenue growth in 2024 when compared to 2023. In 2024, we saw a significant decline in revenue in the pharmaceutical market due to lower sales in our nucleic acid solutions, cell analysis and genomics businesses when compared to 2023. Revenue in the diagnostics and genomics business decreased 1 percent in 2023 when compared to 2022. Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022. Revenue declined in the academia and government and pharmaceutical markets partially offset by revenue growth in the diagnostics and clinical market when compared to 2022.

Revenue in the Agilent CrossLab business increased 5 percent in 2024 when compared to 2023. Foreign currency movements had no overall impact on revenue growth in 2024 when compared to 2023. For the year ended October 31, 2024, we saw revenue growth across all of our end markets led by strong revenue growth in the pharmaceutical, diagnostics and clinical and environmental and forensics markets when compared to 2023.

Revenue generated by Agilent CrossLab increased 8 percent in 2023 when compared to 2022. Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022. For the year ended October 31, 2023, we saw revenue growth across all of our end markets led by strong revenue growth in the pharmaceutical, academia and government, diagnostics and clinical and chemical and advanced materials markets when compared to 2022.

## Costs and Expenses

Years Ended October 31,
2024 over 2023 Change
2023 over 2022 Change
2024
2023
2022
(in millions, except margin data)

Gross margin on products
56.7%
51.9%
56.8%
5 ppts.
(5) ppts.
Gross margin on services and other
48.3%
47.3%
46.8%
1 ppt.
1 ppt.
Total gross margin
54.3%
50.7%
54.4%
4 ppts.
(4) ppts.
Research and development
$
479
$
481
$
467
—
3%
Selling, general and administrative
$
1,568
$
1,634
$
1,637
(4)%
—
Operating margin
22.9%
19.8%
23.6%
3 ppts.
(4) ppts.

Total gross margin for the year ended October 31, 2024 increased 4 percentage points when compared to 2023. Total gross margin as well as gross margin on products for 2024 improved from the prior year as 2023 had asset impairment charges of $253 million primarily related to the exit of our Resolution Bioscience business. In addition, total gross margin was favorably impacted by targeted price increases, lower shipping costs and intangible amortization expense partially offset by lower sales volume, higher share-based compensation expense, higher wages and restructuring charges. Total gross margin for the year ended October 31, 2023 decreased 4 percentage points when compared to 2022. Total gross margin as well as gross margin on products for the year ended October 31, 2023 was significantly impacted by asset impairment charges of $253 million primarily related to the exit of our Resolution Bioscience business. Excluding these asset impairment charges, total gross margin for the year ended October 31, 2023 was relatively flat when compared to 2022. Total gross margin was also impacted by targeted price increases, lower shipping and logistics costs, variable pay expenses and intangible amortization expense offset by the unfavorable impact of currency movements, higher wages, restructuring and other related costs and inventory charges.

Gross inventory charges were $45 million in 2024, $40 million in 2023 and $24 million in 2022. Sales of previously written down inventory were $16 million in 2024, $9 million in 2023 and $11 million in 2022.

Research and development expenses for the year ended October 31, 2024 were flat when compared to 2023. Research and development expenses slightly decreased due to lower salary expense related to workforce reduction activities mostly offset by restructuring charges and an impairment of in-process research and development when compared to 2023. Research and development expenses for the year ended October 31, 2023 increased 3 percent when compared to 2022. Research and development expenses increased due to higher wages, program costs in our life sciences and applied markets and diagnostics and genomics businesses and restructuring and other related costs partially offset by the lower variable pay expenses and favorable impact of currency movements.

Selling, general and administrative expenses decreased 4 percent in 2024 when compared to 2023. Selling, general and administrative expenses decreased due to lower intangible amortization expenses, transformational initiatives, advertising expenses, variable pay and salary expense related to workforce reduction activities partially offset by higher restructuring charges and share-based compensation expense. Selling, general and administrative expenses were flat in 2023 compared to 2022. Selling, general and administrative expenses in 2022 included a decrease in expenses of $25 million related to the change in the fair value of contingent consideration. Excluding this amount, selling, general and administrative expenses decreased 2 percent when compared to 2022. The decrease was due to lower variable pay, intangible amortization expense, sales commissions and the favorable impact of currency movements partially offset by higher wages, restructuring and other related costs and asset impairment charges primarily related to the exit of our Resolution Bioscience business.

Total operating margin for the year ended October 31, 2024 increased 3 percentage points when compared to 2023. Total operating margin for the year ended October 31, 2024 increased mostly due to lower impairment charges in 2024 compared to 2023 partially offset by restructuring charges. Total operating margin for the year ended October 31, 2023, decreased 4 percentage points when compared to 2022. Total operating margin for the year ended October 31, 2023 decreased mainly due to asset impairment charges primarily related to the exit of our Resolution Bioscience business and restructuring and other related costs.

Interest income for the years ended October 31, 2024, 2023 and 2022 was $80 million, $51 million and $9 million, respectively. The increase in interest income in 2024 and 2023 was primarily due to higher cash balances and increases in interest rates related to our cash and cash equivalents.

Interest expense for the years ended October 31, 2024, 2023 and 2022 was $96 million, $95 million and $84 million, respectively, and primarily relates to the interest charged on our senior notes, term loan, credit facilities, commercial paper and the amortization of the deferred loss recorded upon termination of the forward starting interest rate swap contracts partially offset by the amortization of deferred gains recorded upon termination of interest rate swap contracts. The increase in interest expense from 2022 is primarily related to higher interest rates on short-term commercial paper and the variable rate on the term loan facility.

Our headcount was approximately 17,900 at October 31, 2024 and 18,100 at October 31, 2023.

### Other income (expense), net

For the year ended October 31, 2024, other income (expense), net of $49 million income includes $8 million of income related to foreign currency translation reclassified out of accumulated comprehensive income (loss) and $12 million income related to the provision of site service costs to, and lease income from, Keysight Technologies, Inc. ("Keysight"). The costs associated with these services are reported within income from operations. Other income (expense), net also includes $25 million income related to the defined benefit retirement and post-retirement benefit plans (interest cost, expected return on assets, amortization of net actuarial (gain) loss, prior service credits and settlement loss).

For the year ended October 31, 2023, other income (expense), net of $33 million income includes $43 million income related to the net gain on the divestiture of our Resolution Bioscience business and $12 million income related to the provision of site service costs to, and lease income from, Keysight. The costs associated with these services are reported within income from operations. Other income (expense), net also includes $10 million income related to the defined benefit retirement and post-retirement benefit plans (interest cost, expected return on assets, amortization of net actuarial (gain) loss, prior service credits and settlement loss) partially offset by the net loss on the fair value of equity securities of approximately $41 million.

For the year ended October 31, 2022, other income (expense), net of $39 million expense includes the net loss on the fair value of equity securities of approximately $67 million and a $9 million loss on extinguishment of debt partially offset by income of $25 million income related to the defined benefit retirement and post-retirement benefit plans and $11 million related to the provision of site service costs to, and lease income from, Keysight. The costs associated with these services are reported within income from operations.

## Income Taxes

Years Ended October 31,
2024
2023
2022
(in millions)
Provision (benefit) for income taxes
$
232
$
99
$
250

For 2024, our income tax expense was $232 million with an effective tax rate of 15.3 percent. For the year ended October 31, 2024, our effective tax rate and the resulting provision for income taxes were impacted by the tax benefit of $47 million related to foreign-derived intangible income.

For 2023, our income tax expense was $99 million with an effective tax rate of 7.4 percent. For the year ended October 31, 2023, our effective tax rate and the resulting provision for income taxes were impacted by the federal tax benefit of $104 million related to the realized loss on the divestiture of a business. The income taxes for the year ended October 31, 2023, also include the tax benefit of $41 million related to foreign-derived intangible income along with the tax benefit of $30 million related to the release of tax reserves in the U.S. due to the settlement of the audit with the Internal Revenue Service ("IRS") for tax years 2018 and 2019.

For 2022, our income tax expense was $250 million with an effective tax rate of 16.6 percent. For the year ended October 31, 2022, our effective tax rate and the resulting provision for income taxes were impacted by the tax benefit of $46 million related to foreign-derived intangible income.

We have negotiated a tax holiday in Singapore. The tax holiday provides a lower rate of taxation on certain classes of income and requires various thresholds of investments and employment or specific types of income. The tax holiday in Singapore was renegotiated and extended through 2030. As a result of the incentive, the impact of the tax holiday decreased income taxes by $84 million, $54 million, and $53 million in 2024, 2023, and 2022, respectively. The benefit of the tax holiday on net income per share (diluted) was approximately $0.29, $0.18, and $0.18 in 2024, 2023 and 2022, respectively.

With these jurisdictions and the U.S., it is reasonably possible that some tax audits may be completed over the next twelve months. However, management is not able to provide a reasonably reliable estimate of the timing of any other future tax payments or change in unrecognized tax benefits, if any.

The calculation of our tax liabilities involves dealing with uncertainties in the application of complex tax law and regulations in a multitude of jurisdictions. Although the guidance on the accounting for uncertainty in income taxes prescribes the use of a recognition and measurement model, the determination of whether an uncertain tax position has met those thresholds will continue to require significant judgment by management. In accordance with the guidance on the accounting for uncertainty in income taxes, for all U.S. and other tax jurisdictions, we recognize potential liabilities for anticipated tax audit issues based on our estimate of whether, and the extent to which, additional taxes and interest will be due. The ultimate resolution of tax uncertainties may differ from what is currently estimated, which could result in a material impact on income tax expense. If our estimate of income tax liabilities proves to be less than the ultimate assessment, a further charge to expense would be required. If events occur and the payment of these amounts ultimately proves to be unnecessary, the reversal of the liabilities would result in tax benefits being recognized in the period when we determine the liabilities are no longer necessary. We include interest and penalties related to unrecognized tax benefits within the provision for income taxes on the consolidated statements of operations.


## Life Sciences and Applied Markets

Our life sciences and applied markets business provides application-focused solutions that include instruments, consumables and software that enable customers to identify, quantify and analyze the physical and biological properties of substances and products, as well as enable customers in the clinical and life sciences research areas to interrogate samples at the molecular and cellular level. Key product categories include: liquid chromatography ("LC") systems and components; liquid chromatography mass spectrometry ("LCMS") systems; gas chromatography ("GC") systems and components; gas chromatography mass spectrometry ("GCMS") systems; inductively coupled plasma mass spectrometry ("ICP-MS") instruments; atomic absorption ("AA") instruments; microwave plasma-atomic emission spectrometry ("MP-AES") instruments; inductively coupled plasma optical emission spectrometry ("ICP-OES") instruments; raman spectroscopy; laboratory software for sample tracking; information management and analytics; laboratory automation and robotic systems; dissolution testing; vacuum pumps and measurement technologies. Our consumables portfolio is designed to improve customer outcomes. Most of the portfolio is vendor neutral, meaning Agilent can serve and supply customers regardless of their instrument purchase choices. Solutions range from chemistries to supplies. Key product categories in consumables include GC and LC columns, sample preparation products, custom chemistries, and a large selection of laboratory instrument supplies.

### Net Revenue

| Years Ended October 31 | 2024 over 2023 Change | 2023 over 2022 Change |
| --- | --- | --- |
| 2024 | 2023 | 2022 |
| (in millions) |
| $3,215 | $3,510 | $3,630 | (8)% | (3)% |

Life sciences and applied markets business revenue in 2024 decreased 8 percent compared to 2023. Foreign currency movements had no overall impact on revenue growth in 2024 when compared to the same period last year. Geographically, revenue decreased 9 percent in the Americas with no currency impact, decreased 5 percent in Europe with a 1 percentage point favorable currency impact and decreased 10 percent in Asia Pacific with a 1 percentage point unfavorable currency impact.

The revenue decline in the Americas was driven by weakness in our liquid chromatography, liquid chromatography mass spectrometry, gas chromatography mass spectrometry and gas chromatography businesses partially offset by strength in the consumables business when compared to 2023. The revenue decline in Europe was driven by weakness in our liquid chromatography, gas chromatography, automation and gas chromatography mass spectrometry businesses partially offset by strength in the consumables business when compared to 2023. The revenue decline in Asia Pacific was driven by lower demand in China within our liquid chromatography, liquid chromatography mass spectrometry, gas chromatography mass spectrometry and spectroscopy businesses when compared to 2023.

All end market revenue declined in 2024.

Revenue in the pharmaceutical market declined significantly due to weakness in our liquid chromatography, liquid chromatography mass spectrometry and gas chromatography businesses when compared to 2023. Revenue in the chemicals and advanced materials market declined significantly due to weakness in our liquid chromatography, gas chromatography and liquid chromatography mass spectrometry businesses partially offset by strength in our consumables business when compared to 2023. Revenue in the food market declined significantly due to weakness in our liquid chromatography, gas chromatography mass spectrometry and spectroscopy businesses partially offset by strength in our consumables business when compared to 2023. Revenue in the academia and government market declined significantly due to weakness in our liquid chromatography and liquid chromatography mass spectrometry businesses when compared to 2023.

Life sciences and applied markets business revenue in 2023 decreased 3 percent compared to 2022. Foreign currency movements for 2023 had an overall unfavorable impact on revenue growth of 2 percentage points when compared to 2022. Geographically, revenue decreased 1 percent in the Americas with no currency impact, increased 1 percent in Europe with a 1 percentage point unfavorable currency impact and decreased 7 percent in Asia Pacific with a 3 percentage point unfavorable currency impact. The revenue decline in the Americas was driven by weakness in our liquid chromatography mass spectrometry and liquid chromatography businesses partially offset by strength in the consumables business when compared to 2022. The revenue growth in Europe was driven by strength in our liquid chromatography business partially offset by weakness in the liquid chromatography mass spectrometry business when compared to 2022. The revenue decline in Asia Pacific was driven by China with weakness in our liquid chromatography, gas chromatography and gas chromatography mass spectrometry businesses partially offset by strength in the spectroscopy business when compared to 2022.

End market revenue performance in 2023 was mixed as the pharmaceutical and diagnostics and clinical markets declined significantly, environmental and forensics market declined modestly, chemicals and advance materials and food markets remained relatively flat while the academia and government market delivered strong growth when compared to 2022. Revenue decline in the pharmaceutical market was driven by weakness in our liquid chromatography, liquid chromatography mass spectrometry and gas chromatography businesses when compared to 2022. Revenue decline in the diagnostics and clinical market was driven by weakness in our liquid chromatography mass spectrometry business partially offset by strength in our consumables business when compared to 2022. Revenue decline in the environmental and forensics market was driven by weakness in our gas chromatography and spectroscopy businesses partially offset by strength in our liquid chromatography business when compared to 2022. Revenue growth in the academia and government end market was mainly driven by strength in the liquid chromatography, spectroscopy, gas chromatography and liquid chromatography mass spectrometry businesses when compared to 2022.

Looking forward, we anticipate continued and steady market recovery and are optimistic about our long-term growth opportunities in the life sciences and applied markets as our broad portfolio of products and solutions are well suited to address customer needs. We will continue to invest in expanding and improving our applications and solutions portfolio.

### Gross Margin and Operating Margin

| Years Ended October 31 | 2024 over 2023 Change | 2023 over 2022 Change |
| --- | --- | --- |
| 2024 | 2023 | 2022 |
| (in millions, except margin data) |
| Total gross margin | 59.7% | 60.3% | 60.1% | (1) ppt. | — |
| Research and development | $255 | $262 | $257 | (3)% | 2% |
| Selling, general and administrative | $788 | $805 | $829 | (2)% | (3)% |
| Operating margin | 27.3% | 29.9% | 30.2% | (3) ppts. | — |
| Income from operations | $877 | $1,049 | $1,097 | (16)% | (4)% |

Gross margin decreased 1 percentage point in 2024 compared to 2023. Gross margin was impacted by lower sales volume, unfavorable impact of currency movements and higher warranty costs which were partially offset by lower salary expense related to workforce reduction activities and shipping costs when compared to 2023. Gross margin was flat in 2023 compared to 2022. Gross margin was impacted by lower revenue, the unfavorable impact of currency movements, higher wages and warranty costs which were fully offset by targeted price increases, lower variable pay, logistics and materials cost.

Research and development expenses decreased 3 percent in 2024 when compared to 2023. Research and development expenses decreased due to lower salary expense related to workforce reduction activities, lower consumables costs and depreciation expenses partially offset by the unfavorable impact of currency movements when compared to 2023. Research and development expenses increased 2 percent in 2023 when compared to 2022. Research and development expenses increased due to higher wages and program investments in our software and informatics business partially offset by lower variable pay.

Selling, general and administrative expenses decreased 2 percent in 2024 compared to 2023. Selling, general and administrative expenses decreased due to lower salary expense related to workforce reduction activities, variable pay and marketing costs when compared to 2023. Selling, general and administrative expenses decreased 3 percent in 2023 compared to 2022. Selling, general and administrative expenses decreased due to lower variable pay, sales commissions, and marketing expenses, and the favorable impact of currency movements when compared to 2022.

Operating margin decreased 3 percentage points in 2024 compared to 2023. Operating margin was impacted by lower sales volume and the unfavorable impact of currency movements partially offset by lower salary expense related to workforce reduction activities, and shipping costs when compared to 2023. Operating margin was flat in 2023 compared to 2022. Operating margin was impacted by lower sales volume and the unfavorable impact of currency movements offset by lower variable pay and logistics costs.

### Income from Operations

Income from operations in 2024 decreased by $172 million or 16 percent when compared to 2023 on a revenue decrease of $295 million. Income from operations in 2023 decreased by $48 million or 4 percent when compared to 2022 on a revenue decrease of $120 million.

## Diagnostics and Genomics

Our diagnostics and genomics business includes the cell analysis, advanced manufacturing partnerships contract manufacturing and research and development, pathology, companion diagnostics, reagent partnership, genomics and biomolecular analysis businesses.

Our diagnostics and genomics business is comprised of seven areas of activity providing active pharmaceutical ingredients ("APIs") for oligo-based therapeutics as well as solutions that include reagents, instruments, software and consumables, which enable customers in the clinical and life sciences research areas to interrogate samples at the cellular and molecular level. First, our cell analysis business includes instruments, reagents, software, and labware associated with unique live-cell analysis platforms in addition to mainstream flow cytometers, plate-readers, and plate washers/dispensers which are used across a broad range of applications. Second, our advanced manufacturing partnerships business is a contract and development manufacturing organization that provides services related to and the production of synthesized oligonucleotides under pharmaceutical good manufacturing practices ("GMP") conditions for use as API in a class of drugs that utilize nucleic acid molecules for disease therapy. Together, our BIOVECTRA and nucleic acid solutions businesses offer a broad range of contract and development manufacturing services to our customers. They provide clinical-to-commercial scale production capabilities focused mainly on mRNA manufacturing. Third, our pathology solutions business is focused on product offerings for cancer diagnostics and anatomic pathology workflows. The broad portfolio of offerings includes immunohistochemistry ("IHC"), in situ hybridization ("ISH"), hematoxylin and eosin ("H&E") staining and special staining. Fourth, we also collaborate with a number of major pharmaceutical companies to develop new potential tissue pharmacodiagnostics, also known as companion diagnostics, which may be used to identify patients most likely to benefit from a specific targeted therapy. Fifth, the reagent partnership business provides clinical flow cytometry reagents for routine cancer diagnostics. This business also provides bulk antibodies as raw materials and associated assay development services to in vitro diagnostics ("IVD") manufacturers, biotechnology and pharmaceutical companies. Sixth, our genomics business includes arrays and next generation sequencing ("NGS"). This business also includes solutions that enable clinical labs to identify DNA variants associated with genetic disease and help direct cancer therapy. Finally, our biomolecular analysis business provides complete workflow solutions, including instruments, consumables and software, for quality control analysis of nucleic acid samples. Samples are analyzed using quantitative and qualitative techniques to ensure accuracy in further genomics analysis techniques including NGS, utilized in clinical and life science research applications.

### Net Revenue

| Years Ended October 31 | 2024 over 2023 Change | 2023 over 2022 Change |
| --- | --- | --- |
| 2024 | 2023 | 2022 |
| (in millions) |
| $1,651 | $1,755 | $1,766 | (6)% | (1)% |

Diagnostics and genomics business revenue decreased 6 percent in 2024 compared to 2023.


### Years Ended October 31

| Metric | 2024 | 2023 | 2022 | 2024 over 2023 Change | 2023 over 2022 Change |
| --- | --- | --- | --- | --- | --- |
| Net revenue | $1,651 | $1,755 | $1,766 | (6)% | (1)% |

Diagnostics and genomics business revenue decreased 6 percent in 2024 compared to 2023. Foreign currency movements for 2024 had no overall impact on revenue growth when compared to the same period last year. Geographically, revenue decreased 10 percent in the Americas with no currency impact, increased 4 percent in Europe with a 1 percentage point favorable currency impact and decreased 11 percent in Asia Pacific with a 2 percentage point unfavorable currency impact. The revenue decline in the Americas was primarily driven by our cell analysis, genomics and nucleic acid solutions businesses. Revenue increased in Europe due to strong performance in our pathology, genomics and biomolecular analysis businesses partially offset by a decline in our cell analysis business. The revenue decline in Asia Pacific was driven by our cell analysis business partially offset by increased revenue in our pathology business.

In 2024, revenue performance in the pharmaceutical market declined significantly due to our cell analysis business which was impacted by the continuing slow availability of the customer capital budgets and by unfavorable mix in our nucleic acid solutions business when compared to the same period last year. We also saw a modest revenue increase in the diagnostics and clinical markets primarily from our pathology business which was partially offset by a decline in our genomics business. Revenue in the academia and government markets declined due to our cell analysis business.

Diagnostics and genomics business revenue in 2023 decreased 1 percent compared to 2022. Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022. Geographically, revenue decreased 1 percent in the Americas with no currency impact, increased 3 percent in Europe with a 1 percentage point unfavorable currency impact and decreased 6 percent in Asia Pacific with a 5 percentage point unfavorable currency impact. The decrease in the Americas was driven by a decline in our genomics and cell analysis businesses which was partially offset by strong growth in our nucleic acid solutions, reagent partnership and pathology businesses. The increase in Europe was driven by growth in our pathology, reagent partnership and cell analysis businesses. The revenue decline in Asia Pacific was driven by our biomolecular analysis and genomics businesses and an overall weakness in China.

In 2023, the decline in revenue in the pharmaceutical market was led by our cell analysis, biomolecular analysis and genomics businesses which was partially offset by strong revenue growth in our nucleic acid solutions business. We saw moderate revenue growth in the diagnostics and clinical markets led by our pathology, reagent partnership and cell analysis businesses when compared to 2022. Revenue in the academia and government markets declined due to softness in our cell analysis and genomics businesses.

Looking forward, despite the challenging market conditions, we are optimistic about our long-term growth opportunities in our end markets and continue to invest in expanding and improving our applications and solutions portfolio. We remain positive about our growth in our end markets as our product portfolio around OMNIS and PD-L1 assays continues to gain strength with our customers in clinical oncology applications, and our next generation sequencing related solutions continue to be adopted. Market demand in the advanced manufacturing partnerships business related to therapeutic oligo programs continues, and we are well positioned to serve more of the market demand. We will also continue to invest in research and development and seek to expand our position in developing countries and emerging markets.

## Gross Margin and Operating Margin

| Metric | 2024 | 2023 | 2022 | 2024 over 2023 Change | 2023 over 2022 Change |
| --- | --- | --- | --- | --- | --- |
| Total gross margin | 52.4% | 53.4% | 55.0% | (1) ppt. | (2) ppts. |
| Research and development | $161 | $177 | $174 | (9)% | 2% |
| Selling, general and administrative | $385 | $397 | $407 | (3)% | (3)% |
| Operating margin | 19.4% | 20.7% | 22.1% | (1) ppt. | (1) ppt. |
| Income from operations | $320 | $363 | $390 | (12)% | (7)% |

Gross margin decreased 1 percentage point in 2024 when compared to 2023. Gross margin was impacted by lower sales volume and higher infrastructure costs which were partially offset by lower salary expense related to workforce reduction activities and expenses attributed to business exit activities. Gross margin decreased 2 percentage points in 2023 when compared to 2022. Gross margin decreased due to volume decline in our cell analysis and genomics businesses resulting in unfavorable business mix. Gross margin was also impacted by the unfavorable impact of currency movements, higher wages and infrastructure costs partially offset by lower variable pay.

Research and development expenses decreased 9 percent in 2024 when compared to 2023. Research and development expenses decreased primarily due to lower expenses attributed to business exit activities and salary expenses related to workforce reduction activities. Research and development expenses increased 2 percent in 2023 when compared to 2022. Research and development expenses increased due to higher wages, additional expenses related to a recent acquisition and higher infrastructure costs.

Selling, general and administrative expenses decreased 3 percent in 2024 when compared to 2023. Selling, general and administrative expenses decreased due to lower expenses attributed to business exit activities and lower salary expense related to workforce reduction activities partially offset by higher infrastructure costs. Selling, general and administrative expenses decreased 3 percent in 2023 when compared to 2022. Selling, general and administrative expenses decreased due to lower variable pay, the favorable impact of currency movements and lower infrastructure costs.

Operating margin decreased 1 percentage point in 2024 when compared to 2023. Operating margin decreased due to lower revenue, higher infrastructure costs and higher wages partially offset by lower salary expense related to workforce reduction activities and expenses attributed to business exit activities. Operating margin decreased 1 percentage point in 2023 when compared to 2022. The decrease in operating margin resulted from lower gross margins, the unfavorable impact of currency movements and additional costs related to a recent acquisition partially offset by lower variable pay.

## Income from Operations

Income from operations in 2024 decreased by $43 million or 12 percent when compared to 2023 on a revenue decrease of $104 million. Income from operations in 2023 decreased by $27 million or 7 percent when compared to 2022 on a revenue decrease of $11 million.

## Agilent CrossLab

The Agilent CrossLab business spans the entire lab with its extensive services portfolio, which is designed to improve customer outcomes and represents a broad range of offerings designed to serve customer needs across end-markets and regardless of instrument manufacturer.

The services portfolio includes repairs, parts, maintenance, installations, training, compliance support, software as a service, asset management, consulting and various other custom services to support the customers' laboratory operations. Custom services are tailored to meet the specific application needs of various industries and to keep instruments fully operational and compliant with the respective industry requirements.

### Net Revenue

| Metric | 2024 | 2023 | 2022 | 2024 over 2023 Change | 2023 over 2022 Change |
| --- | --- | --- | --- | --- | --- |
| Total net revenue | $1,644 | $1,568 | $1,452 | 5% | 8% |

Agilent CrossLab business revenue increased 5 percent in 2024 when compared to 2023. Foreign currency movements for 2024 had no overall impact on revenue growth when compared to 2023. Geographically, revenue increased 6 percent in the Americas with no currency impact, increased 8 percent in Europe with a 2 percentage point favorable currency impact and was flat in Asia Pacific with a 2 percentage point unfavorable currency impact. During the year ended October 31, 2024, revenue in all three regions reflected consistent high demand for repair and maintenance services across the entire portfolio. In Americas and Europe, revenue growth was partially offset by weakness in installation revenue. In the Asia Pacific region the weakness in installation revenue offset the revenue growth seen from repair and maintenance services.

Agilent CrossLab business revenue increased 8 percent in 2023 when compared to 2022. Foreign currency movements had an overall unfavorable impact on revenue growth of 2 percentage points in 2023 when compared to 2022. Geographically, revenue increased 12 percent in the Americas with a 1 percentage point favorable currency impact, increased 10 percent in Europe with no currency impact and increased 3 percent in Asia Pacific with a 5 percentage point unfavorable currency impact. During the year ended October 31, 2023, revenue growth in all three regions was driven by contract repair services, per-incident repair services and consultative services, with installation related service in China partially offsetting the overall growth in Asia Pacific.

In 2024, we saw strong revenue growth in the environmental and forensics, diagnostics and clinical and pharmaceutical markets, mainly driven by our spectroscopy, gas chromatography and liquid chromatography businesses, when compared to the same period last year.

In 2023, we saw strong revenue growth in the pharmaceutical, academia and government, diagnostics and clinical and chemical and advanced materials markets, mainly driven by our spectroscopy, liquid and gas chromatography mass spectrometry and liquid chromatography businesses when compared to the same period the previous year.

Looking forward, Agilent CrossLab services are well positioned to continue their success in our key end markets by supporting a growing installed base of instruments. Digital and remote capabilities will continue to be a key factor in improving the service quality and the customers' experience. Geographically, the business is well diversified across all regions to take advantage of local market opportunities and to hedge against weakness in any one region.

### Gross Margin and Operating Margin

| Metric | 2024 | 2023 | 2022 | 2024 over 2023 Change | 2023 over 2022 Change |
| --- | --- | --- | --- | --- | --- |
| Total gross margin | 50.9% | 49.3% | 47.6% | 2 ppts. | 2 ppts. |
| Research and development | $33 | $33 | $32 | (1)% | 2% |
| Selling, general and administrative | $281 | $276 | $288 | 2% | (4)% |
| Operating margin | 31.9% | 29.5% | 25.5% | 2 ppts. | 4 ppts. |
| Income from operations | $524 | $463 | $370 | 13% | 25% |

Gross margin increased 2 percentage points in 2024 when compared to 2023. Gross margin was impacted by targeted price increases and well-controlled variable service delivery costs, offset by higher wages.

Gross margin increased 2 percentage points in 2023 when compared to 2022. Gross margin was impacted by higher sales volume, targeted price increases and lower variable pay that improved margins, which were partially offset by higher wages, service delivery costs for logistics and parts and the unfavorable impact of currency movements.

Research and development expenses decreased 1 percent in 2024 when compared to 2023. Research and development expenses decreased due to lower salary expense related to workforce reduction activities.

Research and development expenses increased 2 percent in 2023 when compared to 2022. Research and development expenses increased due to higher wages, partially offset by lower variable pay and the favorable impact of currency movements.

Selling, general and administrative expenses increased 2 percent in 2024 when compared to 2023. Selling, general and administrative expenses increased due to higher commissions partially offset by lower travel expenses and other discretionary spending and salary expense related to workforce reduction activities.

Selling, general and administrative expenses decreased 4 percent in 2023 when compared to 2022. The decrease was primarily due to lower variable pay and a favorable impact of currency movements partially offset by higher wages.

Operating margin increased 2 percentage points in 2024 when compared to 2023. Operating margin increased mostly driven by targeted price increases, lower service delivery costs and salary expense related to workforce reduction activities.

Operating margin increased 4 percentage points in 2023 when compared to 2022. Operating margin increased mostly due to higher sales volume, targeted price increases and lower variable pay that improved margins in addition to a reduction in expenses.

## Income from Operations

Income from operations in 2024 increased by $61 million or 13 percent when compared to 2023 on a revenue increase of $76 million. Income from operations in 2023 increased by $93 million or 25 percent when compared to 2022 on a revenue increase of $116 million.

## Financial Condition

### Liquidity and Capital Resources

We believe our cash and cash equivalents, cash generated from operations, and ability to access capital markets and credit lines will satisfy, for at least the next twelve months and beyond, our liquidity requirements, both globally and domestically, including the following: working capital needs, capital expenditures, business acquisitions, stock repurchases, cash dividends, contractual obligations, commitments, principal and interest payments on debt, and other liquidity requirements associated with our operations.

Our financial position as of October 31, 2024 consisted of cash and cash equivalents...


We believe our cash and cash equivalents, cash generated from operations, and ability to access capital markets and credit lines will satisfy, for at least the next twelve months and beyond, our liquidity requirements, both globally and domestically, including the following: working capital needs, capital expenditures, business acquisitions, stock repurchases, cash dividends, contractual obligations, commitments, principal and interest payments on debt, and other liquidity requirements associated with our operations.

Our financial position as of October 31, 2024 consisted of cash and cash equivalents of $1,329 million as compared to $1,590 million as of October 31, 2023.

We may, from time to time, retire certain outstanding debt of ours through open market cash purchases, privately-negotiated transactions or otherwise. Such transactions, if any, will depend on prevailing market conditions, our liquidity requirements, contractual restrictions and other factors.

### Net Cash Provided by Operating Activities

Net cash provided by operating activities was $1,751 million in 2024 compared to net cash provided of $1,772 million in 2023 and net cash provided of $1,312 million in 2022. Net cash paid for income taxes was approximately $314 million in 2024 compared to income taxes paid of $199 million in 2023 and $279 million, in 2022. For the years ended October 31, 2024, 2023 and 2022, other assets and liabilities used cash of $49 million, provided cash of $47 million and used cash of $8 million, respectively.

In 2024, accounts receivable provided cash of $7 million, compared to cash provided of $132 million in 2023, and cash used of $321 million in 2022. Days' sales outstanding as of October 31, were 70 days in 2024, 69 days in 2023 and 68 days in 2022. In 2024, the decrease in cash provided by accounts receivable was primarily due to lower collectible sales compared to 2023. In 2023 cash provided in accounts receivable was driven by stronger collections compared to 2022. The change in accounts payable provided cash of $103 million in 2024, used cash of $171 million in 2023 and provided cash of $121 million in 2022. In 2024 the change was mainly due to less expenditures for direct materials as we continued optimizing our inventory levels and to timing of payments. Cash provided by inventory was $34 million in 2024 compared to cash used of $33 million in 2023 and cash used of $248 million in 2022. Inventory days on-hand decreased to 111 days in 2024 compared to 120 days in 2023 and 112 days in 2022.

The employee compensation and benefits liability used cash of $12 million in 2024 compared to cash used of $91 million in 2023 and cash used of $22 million in 2022. In 2024, the change was largely due to a decrease in variable and incentive pay compared to 2023. In 2023, the change was largely due to a lower accrual for variable pay compared to 2022. We paid approximately $105 million in 2024 under our variable and incentive pay programs compared to $185 million in 2023 and $201 million in 2022.

We made no contributions to our U.S defined benefit plans in 2024, 2023 and 2022. We contributed $20 million in 2024 and $21 million in 2023 and $17 million in 2022 to our non-U.S. defined benefit plans, respectively. We did not contribute to our U.S. post-retirement benefit plans in 2024, 2023 and 2022. Our non-U.S. defined benefit plans are generally funded ratably throughout the year. Our annual contributions are highly dependent on the relative performance of our assets versus our projected liabilities, among other factors. We do not expect to contribute to our U.S. plans and U.S. post-retirement benefit plans during 2025. We expect to contribute $19 million to our non-U.S. defined benefit plans during 2025.

### Net Cash Used in Investing Activities

Net cash used in investing activities was $1,258 million in 2024 compared to net cash used of $310 million in 2023 and net cash used of $338 million in 2022.

Investments in property, plant and equipment were $378 million in 2024, $298 million in 2023 and $291 million in 2022. Our anticipated capital expenditures for fiscal year 2025 will be approximately $450 million. These continued investments in property plant and equipment are primarily due to the planned expansion of our manufacturing capacity for production of nucleic acid based therapeutics in Frederick, Colorado. Some of our investment may be eligible to qualify for reimbursement incentives, which will not fully be known until the expansion is substantially complete.

In 2024, we invested $862 million primarily for our acquisition of BIOVECTRA and one other acquisition compared to $51 million for two acquisitions in 2023 and $52 million for our acquisition of Polymer Standards Service and advanced artificial intelligence technology in 2022. In 2023, proceeds from the divestiture of our Resolution Bioscience business were $50 million.

### Net Cash Used in Financing Activities

Net cash used in financing activities was $752 million in 2024 compared to net cash used of $930 million in 2023 and net cash used of $1,372 million in 2022. Net cash from financing activities consisted primarily of cash flows associated with the issuances and repurchases of common stock, payments of cash dividends, borrowings and repayments under credit facilities and commercial paper, issuances and repayments of long term debt and payments of contingent consideration.

#### Treasury Stock Repurchases

In 2024, we repurchased and retired 8.4 million shares for $1,150 million, excluding excise tax liability of approximately $10 million compared to repurchases in 2023 of 4.6 million shares for $575 million, excluding excise tax liability of approximately $3 million which was paid in 2024 and 8.4 million shares for $1,139 million, in 2022.

The activity of our repurchases and remaining authorization by repurchase program follows:

| Year | Shares | Cost | Remaining Authorization |
| ---- | ------ | ---- | ----------------------- |
| 2024 | 8.4    | $1,150 million | $2,000 million |
| 2023 | 4.6    | $575 million | $374 million |
| 2022 | 8.4    | $1,139 million | - |

#### Dividends

For the years ended October 31, 2024, 2023 and 2022, cash dividends of $274 million, $265 million and $250 million were paid on the company's outstanding common stock, respectively.

On November 20, 2024, we declared a quarterly dividend of $0.248 per share of common stock, or approximately $71 million which will be paid on January 22, 2025 to shareholders of record as of the close of business on December 31, 2024. The timing and amounts of any future dividends are subject to determination and approval by our board of directors.

## Short-term Debt

### Credit Facilities

On June 7, 2023, we entered into a new credit agreement with a group of financial institutions which provides for a $1.5 billion five-year unsecured credit facility that will expire on June 7, 2028, and an incremental revolving credit facility in an aggregate amount of up to $750 million. During 2024, we made no borrowings or repayments under these credit facilities compared to $360 million borrowed and repaid in 2023 and no borrowings or repayments in 2022.

On June 2, 2023, we entered into an Uncommitted Money Market Line Credit agreement with Societe Generale which provides for an aggregate borrowing capacity of $300 million. During 2024, we borrowed and repaid $215 million under this credit facility compared to $61 million borrowed and repaid in 2023. As of October 31, 2024, we had no borrowings outstanding under the credit facility.

### Commercial Paper

Under our U.S. commercial paper program, we may issue and sell unsecured, short-term promissory notes in the aggregate principal amount not to exceed $1.5 billion with up to 397-day maturities. During 2024, we borrowed $1.19 billion and repaid $1.15 billion compared to $1.67 billion borrowed and $1.70 billion repaid in 2023, and $1.29 billion borrowed and $1.26 billion repaid in 2022.

As of October 31, 2024, we had $40 million borrowings outstanding under our U.S. commercial paper program and had a weighted average annual interest rate of 4.92 percent.

### Other Loans

In September 2024, we completed the BIOVECTRA acquisition and assumed two interest-free loans from the Strategic Innovation Fund ("SIF") in the amount of $20 million with $2 million recorded at fair value in short-term debt. We also assumed two interest-free loans with the Atlantic Canada Opportunities Agency ("ACOA") in the amount of $4 million with $3 million recorded at fair value in short-term debt.

## Long-term Debt

In 2024, proceeds from the issuance of long-term debt of $1,197 million related to the issuance of our 2027 and 2034 senior notes. Repayments of long-term debt of $600 million related to the full payment of the outstanding principal amount of our term loan.

In September 2024, we completed the BIOVECTRA acquisition and assumed two interest-free loans from the Strategic Innovation Fund ("SIF") in the amount of $20 million with $18 million recorded at fair value in long-term debt. We also assumed two interest-free loans with the Atlantic Canada Opportunities Agency ("ACOA") in the amount of $4 million with $1 million recorded at fair value in long-term debt.

### Senior Notes

| Senior Notes | Year Issued | Principal Amount ($M) | Interest Rate | Interest payable | Maturity Date |
| ------------ | ----------- | --------------------- | ------------- | ---------------- | ------------- |
| 2026 Senior Notes | 2016 | $300 | 3.05% | semi-annually | September, 2026 |
| 2027 Senior Notes | 2024 | $600 | 4.20% | semi-annually | September, 2027 |
| 2029 Senior Notes | 2019 | $500 | 2.75% | semi-annually | September, 2029 |
| 2030 Senior Notes | 2020 | $500 | 2.10% | semi-annually | June, 2030 |
| 2031 Senior Notes | 2021 | $850 | 2.30% | semi-annually | March, 2031 |
| 2034 Senior Notes | 2024 | $600 | 4.75% | semi-annually | September, 2034 |

### Contingent Consideration Payment

During the year ended October 31, 2023, we paid a total of $72 million in contingent consideration payments, of which $4 million is included as an outflow in cash from operations. We paid $65 million related to the achievement of a certain technical milestone associated with our acquisition of Resolution Bioscience and $7 million related to other acquisitions.

## Off Balance Sheet Arrangements and Other

Our liquidity is affected by many factors, some of which are based on normal ongoing operations of our business and some of which arise from fluctuations related to global economics and markets. Our cash balances are generated and held in many locations throughout the world. Local government regulations may restrict our ability to move cash balances to meet cash needs under certain circumstances. We do not currently expect such regulations and restrictions to impact our ability to pay vendors and conduct operations throughout our global organization.

## Contractual Commitments

The following table summarizes our total contractual obligations at October 31, 2024, for Agilent operations and excludes amounts recorded in our consolidated balance sheet (in millions):

| Contractual Obligations | Less than one year | One to three years | Three to five years | More than five years |
| ---------------------- | ------------------ | ------------------ | ------------------- | -------------------- |
| Commitments to contract manufacturers and suppliers | $601 | $40 | $0 | $0 |
| Other purchase commitments | $128 | $8 | $0 | $0 |
| Total | $729 | $48 | $0 | $0 |


### Opinions on the Financial Statements and Internal Control over Financial Reporting

We have audited the accompanying consolidated balance sheets of Agilent Technologies, Inc. and its subsidiaries (the "Company") as of October 31, 2024 and 2023, and the related consolidated statements of operations, of comprehensive income, of equity and of cash flows for each of the three years in the period ended October 31, 2024, including the related notes and schedule of valuation and qualifying accounts for each of the three years in the period ended October 31, 2024 appearing under Item 15(a)(2) (collectively referred to as the "consolidated financial statements"). We also have audited the Company's internal control over financial reporting as of October 31, 2024, based on criteria established in *Internal Control - Integrated Framework* (2013) issued by the Committee of Sponsoring Organizations of the Treadway Commission (COSO).

In our opinion, the consolidated financial statements referred to above present fairly, in all material respects, the financial position of the Company as of October 31, 2024 and 2023, and the results of its operations and its cash flows for each of the three years in the period ended October 31, 2024 in conformity with accounting principles generally accepted in the United States of America. Also in our opinion, the Company maintained, in all material respects, effective internal control over financial reporting as of October 31, 2024, based on criteria established in *Internal Control - Integrated Framework* (2013) issued by the COSO.

### Basis for Opinions

The Company's management is responsible for these consolidated financial statements, for maintaining effective internal control over financial reporting, and for its assessment of the effectiveness of internal control over financial reporting, included in Management's Report on Internal Control over Financial Reporting appearing under Item 9A. Our responsibility is to express opinions on the Company's consolidated financial statements and on the Company's internal control over financial reporting based on our audits. We are a public accounting firm registered with the Public Company Accounting Oversight Board (United States) (PCAOB) and are required to be independent with respect to the Company in accordance with the U.S. federal securities laws and the applicable rules and regulations of the Securities and Exchange Commission and the PCAOB.

We conducted our audits in accordance with the standards of the PCAOB. Those standards require that we plan and perform the audits to obtain reasonable assurance about whether the consolidated financial statements are free of material misstatement, whether due to error or fraud, and whether effective internal control over financial reporting was maintained in all material respects.

Our audits of the consolidated financial statements included performing procedures to assess the risks of material misstatement of the consolidated financial statements, whether due to error or fraud, and performing procedures that respond to those risks. Such procedures included examining, on a test basis, evidence regarding the amounts and disclosures in the consolidated financial statements. Our audits also included evaluating the accounting principles used and significant estimates made by management, as well as evaluating the overall presentation of the consolidated financial statements. Our audit of internal control over financial reporting included obtaining an understanding of internal control over financial reporting, assessing the risk that a material weakness exists, and testing and evaluating the design and operating effectiveness of internal control based on the assessed risk. Our audits also included performing such other procedures as we considered necessary in the circumstances. We believe that our audits provide a reasonable basis for our opinions.

As described in Management's Report on Internal Control Over Financial Reporting, management has excluded BIOVECTRA from its assessment of internal control over financial reporting as of October 31, 2024 because it was acquired by the Company in a purchase business combination during 2024. We have also excluded BIOVECTRA from our audit of internal control over financial reporting. BIOVECTRA is a wholly-owned subsidiary whose total assets and total revenues excluded from management's assessment and our audit of internal control over financial reporting represent 3% and less than 1%, respectively, of the related consolidated financial statement amounts as of and for the year ended October 31, 2024.

### Definition and Limitations of Internal Control over Financial Reporting

A company's internal control over financial reporting is a process designed to provide reasonable assurance regarding the reliability of financial reporting and the preparation of financial statements for external purposes in accordance with generally accepted accounting principles. A company's internal control over financial reporting includes those policies and procedures that (i) pertain to the maintenance of records that, in reasonable detail, accurately and fairly reflect the transactions and dispositions of the assets of the company; (ii) provide reasonable assurance that transactions are recorded as necessary to permit preparation of financial statements in accordance with generally accepted accounting principles, and that receipts and expenditures of the company are being made only in accordance with authorizations of management and directors of the company; and (iii) provide reasonable assurance regarding prevention or timely detection of unauthorized acquisition, use, or disposition of the company's assets that could have a material effect on the financial statements.

Because of its inherent limitations, internal control over financial reporting may not prevent or detect misstatements. Also, projections of any evaluation of effectiveness to future periods are subject to the risk that controls may become inadequate because of changes in conditions, or that the degree of compliance with the policies or procedures may deteriorate.

### Critical Audit Matters

The critical audit matter communicated below is a matter arising from the current period audit of the consolidated financial statements that was communicated or required to be communicated to the audit committee and that (i) relates to accounts or disclosures that are material to the consolidated financial statements and (ii) involved our especially challenging, subjective, or complex judgments. The communication of critical audit matters does not alter in any way our opinion on the consolidated financial statements, taken as a whole, and we are not, by communicating the critical audit matter below, providing a separate opinion on the critical audit matter or on the accounts or disclosures to which it relates.

#### Acquisition of BIOVECTRA – Valuation of Certain Customer Relationships

As described in Notes 1, 3 and 11 to the consolidated financial statements, on September 


As described in Notes 1, 3 and 11 to the consolidated financial statements, on September 20, 2024, the Company acquired 100 percent of the stock of BIOVECTRA for total consideration paid of $915 million in cash. As of October 31, 2024, gross carrying amount of customer relationships includes approximately $165 million related to BIOVECTRA which was valued by management using the multi-period excess earnings method under the income approach which values the customer relationships by discounting the direct cash flow expected to be generated by the customers. Of the customer relationships related to BIOVECTRA, the majority relates to certain customer relationships. Management's determination of the fair value of customer relationships acquired involved significant estimates and assumptions related to revenue growth rates, discount rates, and customer attrition rates.

The principal considerations for our determination that performing procedures relating to the valuation of certain customer relationships acquired in the acquisition of BIOVECTRA is a critical audit matter are (i) the significant judgment by management when estimating the fair value of certain customer relationships acquired; (ii) a high degree of auditor judgment, subjectivity, and effort in performing procedures and evaluating management's significant assumption related to revenue growth rates; and (iii) the audit effort involved the use of professionals with specialized skill and knowledge.

Addressing the matter involved performing procedures and evaluating audit evidence in connection with forming our overall opinion on the consolidated financial statements. These procedures included testing the effectiveness of controls relating to the acquisition accounting, including controls over management's valuation of certain customer relationships acquired. These procedures also included, among others (i) reading the purchase agreement; (ii) testing management's process for estimating the fair value of certain customer relationships acquired; (iii) evaluating the appropriateness of the multi-period excess earnings method used by management; (iv) testing the completeness and accuracy of the underlying data used in the multi-period excess earnings method; and (v) evaluating the reasonableness of the significant assumption used by management related to revenue growth rates. Evaluating management's assumption related to the revenue growth rates involved considering (i) the current performance of the BIOVECTRA business and (ii) whether the assumption was consistent with evidence obtained in other areas of the audit. Professionals with specialized skill and knowledge were used to assist in evaluating the appropriateness of the multi-period excess earnings method.

## Consolidated Statement of Operations

Years Ended October 31,
| (in millions, except per share data) | 2024 | 2023 | 2022 |
| --- | --- | --- | --- |
| Net revenue: |
| Products | $4,672 | $5,051 | $5,187 |
| Services and other | 1,838 | 1,782 | 1,661 |
| Total net revenue | 6,510 | 6,833 | 6,848 |
| Costs and expenses: |
| Cost of products | 2,024 | 2,428 | 2,242 |
| Cost of services and other | 951 | 940 | 884 |
| Total costs | 2,975 | 3,368 | 3,126 |
| Research and development | 479 | 481 | 467 |
| Selling, general and administrative | 1,568 | 1,634 | 1,637 |
| Total costs and expenses | 5,022 | 5,483 | 5,230 |
| Income from operations | 1,488 | 1,350 | 1,618 |
| Interest income | 80 | 51 | 9 |
| Interest expense | (96) | (95) | (84) |
| Other income (expense), net | 49 | 33 | (39) |
| Income before taxes | 1,521 | 1,339 | 1,504 |
| Provision for income taxes | 232 | 99 | 250 |
| Net income | $1,289 | $1,240 | $1,254 |
| Net income per share: |
| Basic | $4.44 | $4.22 | $4.19 |
| Diluted | $4.43 | $4.19 | $4.18 |
| Weighted average shares used in computing net income per share: |
| Basic | 290 | 294 | 299 |
| Diluted | 291 | 296 | 300 |

## Consolidated Statement of Comprehensive Income

(in millions)
| Years Ended October 31, | 2024 | 2023 | 2022 |
| --- | --- | --- | --- |
| Net income | $1,289 | $1,240 | $1,254 |
| Other comprehensive income (loss): |
| Unrealized gain (loss) on derivative instruments, net of tax expense (benefit) of $(2), $(1) and $13 | (7) | (3) | 43 |
| Amounts reclassified into earnings related to derivative instruments, net of tax expense (benefit) of $(1), $0 and $(8) | (1) | — | (26) |
| Foreign currency translation, net of tax expense (benefit) of $3, $(1) and $(12) | (22) | 34 | (150) |
| Net defined benefit pension cost and post retirement plan costs: |
| Change in actuarial net gain (loss), net of tax expense (benefit) of $0, $(5) and $9 | 53 | (10) | 69 |
| Change in net prior service benefit, net of tax expense (benefit) of $0, $0 and $0 | (1) | (1) | (1) |
| Other comprehensive income (loss) | 22 | 20 | (65) |
| Total comprehensive income | $1,311 | $1,260 | $1,189 |

## Consolidated Balance Sheet

October 31,
| (in millions, except par value and share data) | 2024 | 2023 |
| --- | --- | --- |
| ASSETS |
| Current assets: |
| Cash and cash equivalents | $1,329 | $1,590 |
| Accounts receivable, net | 1,324 | 1,291 |
| Inventory | 972 | 1,031 |
| Other current assets | 334 | 274 |
| Total current assets | 3,959 | 4,186 |
| Property, plant and equipment, net | 1,778 | 1,270 |
| Goodwill | 4,477 | 3,960 |
| Other intangible assets, net | 547 | 475 |
| Long-term investments | 175 | 164 |
| Other assets | 910 | 708 |
| Total assets | $11,846 | $10,763 |
| LIABILITIES AND EQUITY |
| Current liabilities: |
| Accounts payable | $540 | $418 |
| Employee compensation and benefits | 368 | 371 |
| Deferred revenue | 544 | 505 |
| Short-term debt | 45 | — |
| Other accrued liabilities | 398 | 309 |
| Total current liabilities | 1,895 | 1,603 |
| Long-term debt | 3,345 | 2,735 |
| Retirement and post-retirement benefits | 130 | 103 |
| Other long-term liabilities | 578 | 477 |
| Total liabilities | 5,948 | 4,918 |
| Stockholders' equity: |
| Preferred stock; $0.01 par value; 125,000,000 shares authorized; none issued and outstanding | — | — |
| Common stock; $0.01 par value; 2,000,000,000 shares authorized; 285,193,011 shares at October 31, 2024 and 292,123,241 shares at October 31, 2023 issued and outstanding | 3 | 3 |
| Additional paid-in-capital | 5,450 | 5,387 |
| Retained earnings | 750 | 782 |
| Accumulated other comprehensive loss | (305) | (327) |
| Total stockholders' equity | 5,898 | 5,845 |
| Total liabilities and stockholders' equity | $11,846 | $10,763 |

## Consolidated Statement of Cash Flows

Years Ended October 31,
| (in millions) | 2024 | 2023 | 2022 |
| --- | --- | --- | --- |
| Cash flows from operating activities: |
| Net income | $1,289 | $1,240 | $1,254 |
| Adjustments to reconcile net income to net cash provided by operating activities: |
| Depreciation and amortization | 257 | 271 | 317 |
| Share-based compensation | 129 | 111 | 125 |
| Deferred taxes expense (benefit) | (64) | (56) | 8 |
| Excess and obsolete inventory related charges | 45 | 40 | 24 |
| Net (gain) loss on equity securities | (6) | 41 | 67 |
| Asset impairment charges | 19 | 277 | — |
| Change in fair value of contingent consideration | — | 1 | (25) |
| Loss on extinguishment of debt | — | — | 9 |
| Net gain on divestiture of business | — | (43) | — |
| Other non-cash (income) expense, net | (1) | 6 | 11 |
| Changes in assets and liabilities: |
| Accounts receivable, net | 7 | 132 | (321) |
| Inventory | 34 | (33) | (248) |
| Accounts payable | 103 | (171) | 121 |
| Employee compensation and benefits | (12) | (91) | (22) |
| Other assets and liabilities | (49) | 47 | (8) |
| Net cash provided by operating activities | 1,751 | 1,772 | 1,312 |
| Cash flows from investing activities: |
| Payments to acquire property, plant and equipment | (378) | (298) | (291) |
| Proceeds from the sale of equity securities | — | 5 | 22 |
| Payments to acquire equity securities | (5) | (8) | (13) |
| Proceeds from convertible note | — | 4 | — |
| Payment in exchange for convertible note | (13) | (12) | (4) |
| Proceeds from divestiture of business | — | 50 | — |
| Payments to acquire businesses and intangible assets, net of cash acquired | (862) | (51) | (52) |
| Net cash used in investing activities | (1,258) | (310) | (338) |
| Cash flows from financing activities: |
| Proceeds from the issuance of common stock under employee stock plans | 77 | 67 | 58 |
| Payment of taxes related to net share settlement of equity awards | (30) | (54) | (67) |
| Payments for repurchase of common stock | (1,150) | (575) | (1,139) |
| Payment of excise taxes related to repurchases of common stock | (3) | — | — |
| Payment of dividends | (274) | (265) | (250) |
| Proceeds from issuance of long-term debt | 1,197 | — | 600 |
| Repayment of long-term debt | (600) | — | (609) |
| Payments of debt issuance costs | (9) | — | — |
| Net proceeds from (repayments of) short term debt | 40 | (35) | 35 |
| Payment for contingent consideration | — | (68) | — |
| Net cash used in financing activities | (752) | (930) | (1,372) |
| Effect of exchange rate movements | (2) | 5 | (36) |
| Net increase (decrease) in cash, cash equivalents and restricted cash | (261) | 537 | (434) |
| Cash, cash equivalents and restricted cash at beginning of year | 1,593 | 1,056 | 1,490 |
| Cash, cash equivalents and restricted cash at end of year | $1,332 | $1,593 | $1,056 |
| Supplemental cash flow information: |
| Income tax payments, net of refunds received | $314 | $199 | $279 |
| Interest payments, net of capitalized interest | $80 | $89 | $85 |
| Net change in property, plant and equipment included in accounts payable and accrued liabilities-increase (decrease) | $9 | $4 | $26 |
| Excise tax on share repurchases, accrued but not paid | $10 | $3 | $— |

## Consolidated Statement of Equity

| Common Stock | Accumulated Other Comprehensive Loss |
| --- | --- | --- | --- | --- |
| Number of Shares | Par Value | Additional Paid-in Capital | Retained Earnings | Total Stockholders' Equity |
| (in millions, except number of shares in thousands) |
| Balance as of October 31, 2021 | 302,208 | $3 | $5,320 | $348 | $(282) | $5,389 |
| Components of comprehensive income, net of tax: |
| Net income | — | — | — | 1,254 | — | 1,254 |
| Other comprehensive income (loss) | — | — | — | — | (65) | (65) |
| Total comprehensive income | 1,189 |
| Cash dividends declared ($0.840 per common share) | — | — | — | (250) | — | (250) |
| Share-based awards issued, net of tax of $67 | 1,419 | — | (9) | — | — | (9) |
| Repurchase of common stock | (8,368) | — | (111) | (1,028) | — | (1,139) |
| Share-based compensation | — | — | 125 | — | — | 125 |
| Balance as of October 31, 2022 | 295,259 | $3 | $5,325 | $324 | $(347) | $5,305 |
| Components of comprehensive income, net of tax: |
| Net income | — | — | — | 1,240 | — | 1,240 |
| Other comprehensive income (loss) | — | — | — | — | 20 | 20 |
| Total comprehensive income | 1,260 |
| Cash dividends declared ($0.900 per common share) | — | — | — | (265) | — | (265) |
| Share-based awards issued, net of tax of $54 | 1,473 | — | 13 | — | — | 13 |
| Repurchase of common stock, including excise taxes | (4,609) | — | (62) | (517) | — | (579) |
| Share-based compensation | — | — | 111 | — | — | 111 |
| Balance as of October 31, 2023 | 292,123 | $3 | $5,387 | $782 | $(327) | $5,845 |
| Components of comprehensive income, net of tax: |
| Net income | — | — | — | 1,289 | — | 1,289 |
| Other comprehensive income (loss) | — | — | — | — | 22 | 22 |
| Total comprehensive income | 1,311 |
| Cash dividends declared ($0.944 per common share) | — | — | — | (274) | — | (274) |
| Share-based awards issued, net of tax of $30 | 1,473 | — | 47 | — | — | 47 |
| Repurchase of common stock, including excise taxes | (8,403) | — | (113) | (1,047) | — | (1,160) |
| Share-based compensation | — | — | 129 | — | — | 


## Announced Exit and Subsequent Divestiture of Resolution Bioscience Business

During the third quarter of fiscal year 2023, we made the decision to exit the Resolution Bioscience business within our diagnostics and genomics segment and recorded a long-lived asset impairment charge of $270 million. In the fourth quarter of fiscal year 2023, we received an unsolicited offer and entered into an agreement to divest the Resolution Bioscience business for $50 million. As a result, we recorded a gain on the divestiture of $43 million in other income (expense), net in the consolidated statement of operations, which included an adjustment to goodwill of $13 million.

### Basis of Presentation

The accompanying consolidated financial statements have been prepared by us pursuant to the rules and regulations of the U.S. Securities and Exchange Commission ("SEC") and are in conformity with U.S. generally accepted accounting principles ("GAAP"). Our fiscal year end is October 31. Unless otherwise stated, all years and dates refer to our fiscal year.

### Principles of Consolidation

The consolidated financial statements include the accounts of the company and our wholly- and majority-owned subsidiaries. All intercompany accounts and transactions have been eliminated.

### Use of Estimates

The preparation of financial statements in accordance with U.S. GAAP requires management to make estimates and assumptions that affect the amounts reported in our consolidated financial statements and accompanying notes. Management bases its estimates on historical experience and various other assumptions believed to be reasonable. Although these estimates are based on management's best knowledge of current events and actions that may impact the company in the future, actual results may be different from the estimates. Our critical accounting policies are those that affect our financial statements materially and involve difficult, subjective or complex judgments by management. Those policies are revenue recognition, valuation of goodwill and purchased intangible assets, inventory valuation, retirement and post-retirement plan assumptions, restructuring and accounting for income taxes.

### Restructuring

The main components of our restructuring plan are related to workforce reductions, consolidation of excess leased facilities and site closures. Workforce reduction charges are accrued when payment of benefits becomes probable that the employees are entitled to the severance and the amounts can be estimated. Consolidation of facilities costs primarily consists of accelerated depreciation of right-of-use assets classified as held and used. In accordance with the accounting guidance, it was determined that certain assets had been abandoned, and an assessment was made of the remaining useful lives and potential alternative uses. If the amounts and timing of cash flows from restructuring activities are significantly different from what we have estimated, the actual amounts of restructuring and other related charges could be materially different, either higher or lower, than those we have recorded.

### Risks and Uncertainties

We are subject to risks common to companies in the analytical instrument industry, such as global economic and financial market conditions, fluctuations in foreign currency exchange rates and fluctuations in customer demand, among others.

### Revenue Recognition

We enter into contracts to sell products, services or combinations of products and services. Products may include hardware or software and services may include one-time service events or services performed over time.

We derive revenue primarily from the sale of analytical and diagnostics products and services. A performance obligation is a promise in a contract to transfer a distinct product or service to a customer and is the unit of account under Accounting Standard Codification Topic 606, Revenue from Contracts with Customers ("ASC 606''). See also Note 4, "Revenue" for additional information on revenue recognition.

Revenue is recognized when control of the promised products or services is transferred to our customers and the performance obligation is fulfilled in an amount that reflects the consideration that we expect to be entitled in exchange for those products or services, the transaction price. For equipment, consumables, and most software licenses, control transfers to the customer at a point in time. We use present right to payment, legal title, physical possession of the asset, and risks and rewards of ownership as indicators to determine the transfer of control to the customer. For products that transfer control over time, revenue is recognized as the performance obligation is satisfied. Product over time revenue is assessed against the following criteria: the performance creates an asset that the customer controls as the asset is created; the asset has no alternative use; and we have an enforceable right to payment. Where acceptance is not a formality, the customer must have documented their acceptance of the product or service. For products that include installation, if the installation meets the criteria to be considered a separate performance obligation, product revenue is recognized when control has passed to the customer, and recognition of installation revenue occurs once completed. Product revenue, including sales to resellers and distributors is reduced for provisions for warranties, returns, and other adjustments in the period the related sales are recorded.

Service revenue includes extended warranty, customer and software support including: Software as a Service, post contract support, consulting including companion diagnostics, and training and education. Instrument service contracts and software maintenance contracts are typically annual contracts, which are billed at the beginning of the contract or maintenance period. Revenue for these contracts is recognized on a straight-line basis to revenue over the service period, as a time-based measure of progress best reflects our performance in satisfying this obligation. There are no deferred costs associated with the service contract, as the cost of the service is recorded when the service is performed. Service calls not included in a support contract are recognized to revenue at the time a service is performed.

We have sales from standalone software. These arrangements typically include software licenses and maintenance contracts, both of which we have determined are distinct performance obligations. We determine the amount of the transaction price to allocate to the license and maintenance contract based on the relative standalone selling price of each performance obligation. Software license revenue is recognized at the point in time when control has been transferred to the customer. The revenue allocated to the software maintenance contract is recognized on a straight-line basis over the maintenance period, which is the contractual term of the contract, as a time-based measure of progress best reflects our performance in satisfying this obligation. Unspecified rights to software upgrades are typically sold as part of the maintenance contract on a when-and-if-available basis.

Our multiple-element arrangements are generally comprised of a combination of instruments, installation or other start-up services, and/or software, and/or support or services. Hardware and software elements are typically delivered at the same time and revenue is recognized when control passes to the customer. Service revenue is deferred and recognized over the contractual period or as services are rendered and accepted by the customer. Our arrangements generally do not include any provisions for cancellation, termination, or refunds that would significantly impact recognized revenue.

For contracts with multiple performance obligations, we allocate the consideration to which we expect to be entitled to each performance obligation based on relative standalone selling prices and recognize the related revenue when or as control of each individual performance obligation is transferred to customers. We estimate the standalone selling price by calculating the average historical selling price of our products and services per geographic region for each performance obligation. Standalone selling prices are determined for each distinct good or service in the contract, and then we allocate the transaction price in proportion to those standalone selling prices by performance obligations.

A portion of our revenue relates to lease arrangements. Standalone lease arrangements are outside the scope of ASC 606 and are therefore accounted for in accordance with ASC 842, Leases. Each of these contracts is evaluated as a lease arrangement, either as an operating lease or a sales-type finance lease using the current lease classification guidance.

### Deferred Revenue

Contract liabilities (deferred revenue) primarily relate to multiple element arrangements for which billing has occurred but transfer of control of all elements (performance obligations) to the customer has either partially or not occurred at the balance sheet date. This includes cash received from customers for products and related installation and services in advance of the transfer of control. Contract liabilities are classified as either in current liabilities in deferred revenue or long-term in other long-term liabilities in the consolidated balance sheet based on the timing of when we expect to complete our performance obligation.

### Sales Taxes

Sales taxes collected from customers and remitted to governmental authorities are not included in our revenue.

### Shipping and Handling Costs

Our shipping and handling costs charged to customers are included in net revenue, and the associated expense is recorded in cost of products for all periods presented.

### Research and Development

Costs related to research, design and development of our products are charged to research and development expense as they are incurred.

### Advertising

Advertising costs are generally expensed as incurred and amounted to $49 million in 2024, $54 million in 2023 and $66 million in 2022.

### Taxes on Income

Income tax expense or benefit is based on income or loss before taxes. Deferred tax assets and liabilities are recognized principally for the expected tax consequences of temporary differences between the tax bases of assets and liabilities and their reported amounts. See Note 6, "Income Taxes" for more information.

### Net Income Per Share

Basic net income per share is computed by dividing net income - the numerator - by the weighted average number of common shares outstanding - the denominator - during the period excluding the dilutive effect of stock options and other employee stock plans. Diluted net income per share gives effect to all potential common shares outstanding during the period unless the effect is anti-dilutive. The dilutive effect of share-based awards is reflected in diluted net income per share by application of the treasury stock method, which includes consideration of unamortized share-based compensation expense and the dilutive effect of in-the-money options and non-vested restricted stock units. Under the treasury stock method, the amount the employee must pay for exercising stock options and unamortized share-based compensation expense are assumed proceeds to be used to repurchase hypothetical shares. See Note 7, "Net Income Per Share".

### Cash, Cash Equivalents and Short-Term Investments

We classify investments as cash equivalents if their original or remaining maturity is three months or less at the date of purchase. Cash equivalents are stated at cost, which approximates fair value.

As of October 31, 2024, approximately $1,313 million of our cash and cash equivalents is held outside of the U.S. by our foreign subsidiaries. Our cash and cash equivalents mainly consist of short-term deposits held at major global financial institutions, institutional money market funds, and similar short duration instruments with original maturities of 90 days or less. We continuously monitor the creditworthiness of the financial institutions and institutional money market funds in which we invest our funds.

We classify equity investments as short-term investments based on their nature and our intent and ability to exit within a year or less. As of October 31, 2024, we had no short-term investments.

### Restricted Cash and Restricted Cash Equivalents

Restricted cash and restricted cash equivalents are included with cash and cash equivalents when reconciling the beginning-of-period and end-of-period total amounts shown on the statement of cash flows. A reconciliation of cash, cash equivalents and restricted cash to the consolidated balance sheet follows:

| October 31 | 2024 | 2023 | 2022 |
| --- | --- | --- | --- |
| (in millions) |  |  |  |
| Cash and cash equivalents | $1,329 | $1,590 | $1,053 |
| Restricted cash included in other assets | 3 | 3 | 3 |
| Total cash, cash equivalents and restricted cash | $1,332 | $1,593 | $1,056 |

### Accounts Receivable, net

Trade accounts receivable are recorded at the invoiced amount and do not bear interest. Such accounts receivable have been reduced by an allowance for doubtful accounts, which is our best estimate of the amount of probable credit losses in our existing accounts receivable. We determine the allowance based on customer specific experience and the aging of such receivables, among other factors. The allowance for doubtful accounts as of October 31, 2024 and 2023 was not material. We do not have any off-balance-sheet credit exposure related to our customers. Accounts receivable are also recorded net of estimated product returns which are not material.

### Concentration of Credit Risk

Financial instruments that potentially subject Agilent to significant concentration of credit risk include money market fund investments, equity investments with readily determinable fair value securities, time deposits and demand deposit balances. These investments are categorized as cash and cash equivalents or short-term investments. In addition, Agilent has credit risk from derivative financial instruments used in hedging activities and accounts receivable. We invest in a variety of financial instruments and limit the amount of credit exposure with any one financial institution. We have a comprehensive credit policy in place and credit exposure is monitored on an ongoing basis.

Credit risk with respect to our accounts receivable is diversified due to the large number of entities comprising our cust

### Concentration of Credit Risk

Financial instruments that potentially subject Agilent to significant concentration of credit risk include money market fund investments, equity investments with readily determinable fair value securities, time deposits and demand deposit balances. These investments are categorized as cash and cash equivalents or short-term investments. In addition, Agilent has credit risk from derivative financial instruments used in hedging activities and accounts receivable. We invest in a variety of financial instruments and limit the amount of credit exposure with any one financial institution. We have a comprehensive credit policy in place and credit exposure is monitored on an ongoing basis.

Credit risk with respect to our accounts receivable is diversified due to the large number of entities comprising our customer base and their dispersion across many different industries and geographies. Credit evaluations are performed on customers requiring credit over a certain amount, and we sell the majority of our products through our direct sales force. Credit risk is mitigated through collateral such as letter of credit, bank guarantees or payment terms like cash in advance.

No single customer accounted for more than 10 percent of accounts receivable as of October 31, 2024, or 2023.

### Inventory

Inventory is valued at standard cost, which approximates actual cost computed on a first-in, first-out basis, not in excess of market value. We assess the valuation of our inventory on a periodic basis and make adjustments to the value for estimated excess and obsolete inventory based on estimates and assumptions about future demand, economic conditions and actual usage, which require management judgment. The excess balance determined by this analysis becomes the basis for our excess inventory charge. Our excess inventory review process includes analysis of inventory levels, sales trends and forecasts, managing product rollovers and working with manufacturing to maximize recovery of excess inventory and to estimate and record reserves for excess, slow-moving and obsolete inventory.

### Property, Plant and Equipment

Property, plant and equipment are stated at cost less accumulated depreciation. Additions, improvements and major renewals are capitalized; maintenance, repairs and minor renewals are expensed as incurred. When assets are retired or disposed of, the assets and related accumulated depreciation and amortization are removed from our general ledger, and the resulting gain or loss is reflected in the consolidated statement of operations. Buildings and improvements are depreciated over the lesser of their useful lives or the remaining term of the lease and machinery and equipment over 3 years to 10 years. We use the straight-line method to depreciate assets.

### Capitalized Software

We capitalize certain internal and external costs incurred to acquire or create internal use software. Capitalized software is included in property, plant and equipment and is depreciated over 3 years to 5 years once development is complete.

### Leases

We determine whether an arrangement is, or contains, a lease at inception. We record the present value of operating lease payments as right-of-use ("ROU") assets and lease liabilities on the consolidated balance sheet. Where we are the lessee, ROU assets represent the company's right to use an underlying asset for the lease term, and lease liabilities represent an obligation to make lease payments based on the present value of lease payments over the lease term. Classification of operating lease liabilities as either current or non-current is based on the expected timing of payments due under our obligations. As most of our leases do not provide an implicit interest rate, we use our incremental borrowing rate based on the information available at commencement date in determining the present value of lease payments. The ROU asset also consists of any lease incentives received. The lease terms used to calculate the ROU asset and related lease liability include options to extend or terminate the lease when it is reasonably certain that we will exercise that option. Leases with an initial term of twelve months or less are not recorded on the consolidated balance sheet and lease expense for these leases is recognized on a straight-line basis over the lease term. Lease expense for operating leases with an initial term of more than twelve months is recognized on a straight-line basis over the lease term as an operating expense. We have lease agreements which require payments for lease and non-lease components. We have elected to account for these payments as a single lease component.

A portion of our revenue relates to lease arrangements where Agilent is the lessor. Standalone lease arrangements are outside the scope of Accounting Standard Codification ("ASC") Topic 606, Revenue Contracts with Customers, and are therefore accounted for in accordance with ASC Topic 842, Leases. Each of these contracts is evaluated as a lease arrangement, either as an operating lease or a sales-type finance lease using the current lease classification guidance. In a lease arrangement that is a multiple-element arrangement, the revenue associated with the lease component is treated under the lease accounting standard ASC 842, whereas the revenue associated with the non-lease component is recognized in accordance with the ASC 606 revenue standard.

See also Note 10, "Leases" for additional information about our leases.

### Acquisitions

Agilent accounts for the acquisition of a business using the acquisition method of accounting, and we allocate the fair value of the purchase price to the tangible assets acquired, liabilities assumed, and intangible assets acquired, including in-process research and development ("IPR&D"), based on their estimated fair values. The excess value of the cost of an acquired business over the fair value of the assets acquired and liabilities assumed is recognized as goodwill. The fair value of IPR&D is initially capitalized as an intangible asset with an indefinite life. When an IPR&D project is completed, the IPR&D is reclassified as an amortizable purchased intangible asset and amortized to costs of revenues over the asset's estimated useful life.

Our determination of the fair value of the intangible assets acquired involves the use of significant estimates and assumptions. Specifically, our determination of the fair value of the developed product technology and IPR&D acquired involve significant estimates and assumptions related to revenue growth rates and discount rates. Our determination of the fair value of customer relationships acquired involved significant estimates and assumptions related to revenue growth rates, discount rates, and customer attrition rates. Our determination of the fair value of the trade name acquired involved the use of significant estimates and assumptions related to revenue growth rates, royalty rates and discount rates. We value backlog using the discounted cash flows based on the estimated revenue from pending orders. We value license agreements based on the expected future cash receipts from license agreements, discounted to present value over the term of the agreement. We believe that the fair value assigned to the assets acquired and liabilities assumed are based on reasonable assumptions and estimates that marketplace participants would use. Actual results could differ materially from these estimates.

### Goodwill and Purchased Intangible Assets

We assess our goodwill and purchased intangible assets for impairment annually or whenever events or changes in circumstances indicate that the carrying value may not be recoverable. Under the authoritative guidance, we have the option to perform a qualitative assessment to determine whether further impairment testing is necessary. The accounting standard gives an entity the option to first assess qualitative factors to determine whether performing the quantitative test is necessary. If an entity believes, as a result of its qualitative assessment, that it is more-likely-than-not (i.e., greater than 50% chance) that the fair value of a reporting unit is less than its carrying amount, the quantitative impairment test will be required. Otherwise, no further testing will be required.

The guidance includes examples of events and circumstances that might indicate that a reporting unit's fair value is less than its carrying amount. These include macro-economic conditions such as deterioration in the entity's operating environment or industry or market considerations; entity-specific events such as increasing costs, declining financial performance, or loss of key personnel; or other events such as an expectation that a reporting unit will be sold or a sustained decrease in the stock price on either an absolute basis or relative to peers.

If it is determined, as a result of the qualitative assessment, that it is more-likely-than-not that the fair value of a reporting unit is less than its carrying amount, then we are required to perform a quantitative impairment test on goodwill to identify and measure the amount of a goodwill impairment loss to be recognized. A goodwill impairment loss, if any, is measured as the amount by which a reporting unit's carrying value, including goodwill, exceeds its fair value, not to exceed the carrying amount of goodwill. As defined in the authoritative guidance, a reporting unit is an operating segment, or one level below an operating segment. We aggregate components of an operating segment that have similar economic characteristics into our reporting units.

In fiscal year 2024, in connection with the change in our segment reporting, we assessed goodwill impairment for our three reporting units which consisted of our three segments: life sciences and applied markets, diagnostics and genomics and Agilent CrossLab. We performed a quantitative test for goodwill impairment of the three reporting units as of November 1, 2023, due to the change in our segment structure. As of November 1, 2023, there was no impairment of goodwill.

In fiscal year 2024, we assessed goodwill impairment for our three reporting units which consisted of three segments: life sciences and applied markets, diagnostics and genomics and Agilent CrossLab. We performed a qualitative test for goodwill impairment of the three reporting units as of September 30, 2024, our annual impairment test date. Based on the results of our qualitative testing, there was no impairment of goodwill as of September 30, 2024. Each quarter we review the events and circumstances to determine if goodwill impairment is indicated. There was no impairment of goodwill during the years ended October 31, 2024, 2023 and 2022.

Purchased intangible assets consist primarily of acquired developed technologies, proprietary know-how, trademarks, and customer relationships and are amortized using the best estimate of the asset's useful life that reflect the pattern in which the economic benefits are consumed or used up or a straight-line method ranging from 2 years to 15 years. IPR&D is initially capitalized at fair value as an intangible asset with an indefinite life and assessed for impairment thereafter. When the IPR&D project is complete, it is reclassified as an amortizable purchased intangible asset and is amortized over its estimated useful life. If an IPR&D project is abandoned, Agilent will record a charge for the value of the related intangible asset to Agilent's consolidated statement of operations in the period it is abandoned.

Our indefinite-lived intangible assets are IPR&D intangible assets. The accounting guidance allows a qualitative approach for testing indefinite-lived intangible assets for impairment, similar to the issued impairment testing guidance for goodwill and allows the option to first assess qualitative factors (events and circumstances) that could have affected the significant inputs used in determining the fair value of the indefinite-lived intangible asset to determine whether it is more-likely-than-not (i.e., greater than 50% chance) that the indefinite-lived intangible asset is impaired. An organization may choose to bypass the qualitative assessment for any indefinite-lived intangible asset in any period and proceed directly to calculating its fair value. As of October 31, 2024, we do not have any indefinite-lived intangible assets.

During fiscal year 2024, we recorded an impairment of in-process research and development of $6 million in research and development in the consolidated statement of operations related to a project in our life sciences and applied markets segment. There were no impairments of indefinite-lived intangible assets during fiscal years 2023 and 2022.

### Impairment of Long-Lived Assets

We continually monitor events and changes in circumstances that could indicate carrying amounts of long-lived assets, including intangible assets, may not be recoverable. When such events or changes in circumstances occur, we assess the recoverability of long-lived assets by determining whether the carrying value of such assets will be recovered through undiscounted expected future cash flows. If the total of the undiscounted future cash flows is less than the carrying amount of those assets, we recognize an impairment loss based on the excess of the carrying amount over the fair value of the assets.

During the year ended October 31, 2024, we recorded an impairment charge of long-lived assets including indefinite-lived intangible assets of $19 million. During the year ended October 31, 2023, we recorded an impairment charge of long-lived assets including intangible assets of $277 primarily related to the exit of our Resolution Bioscience business. During the year ended October 31, 2022, there were no impairments of long-lived assets.

### Variable Interest Entities

We make a determination upon entering into an arrangement whether an entity in which we have made an investment is considered a Variable Interest Entity ("VIE"). We evaluate our investments in privately held companies on an ongoing basis. We have determined that as of October 31, 2024 and 2023, there were no VIEs required to be consolidated in our c


- During the year ended October 31, 2024, we recorded an impairment charge of long-lived assets including indefinite-lived intangible assets of $19 million.
- During the year ended October 31, 2023, we recorded an impairment charge of long-lived assets including intangible assets of $277 million, primarily related to the exit of our Resolution Bioscience business.
- During the year ended October 31, 2022, there were no impairments of long-lived assets.

## Variable Interest Entities

- As of October 31, 2024 and 2023, the total carrying value of investments and loans in privately held companies considered as VIEs was $79 million and $82 million respectively.
- The maximum exposure is equal to the carrying value because we do not have future funding commitments.

## Investments

- Equity investments without readily determinable fair value consist of non-marketable equity securities.
- Equity investments with readily determinable fair value consist of marketable equity securities.
- There are no equity investments with readily determinable fair value at October 31, 2024 and 2023.
- Other investments with readily determinable fair value consist of shares we own in a special fund.
- Trading securities are reported at fair value, with gains or losses resulting from changes in fair value recognized currently in net income.
- Equity method investments are reported at the amount of the company's initial investment and adjusted each period for the company's share of the investee's income or loss and dividend paid.

## Fair Value of Financial Instruments

- The fair value of our senior notes was $3,083 million with a carrying value of $3,326 million as of October 31, 2024.
- The fair value of our senior notes was $1,747 million with a carrying value of $2,135 million as of October 31, 2023.

## Warranty

- Our standard warranty terms typically extend for one year from the date of delivery.
- Estimated warranty charges are recorded within cost of products at the time products are sold.

## Employee Compensation and Benefits

- The total amount of accrued vacation benefit was $116 million and $120 million as of October 31, 2024, and 2023, respectively.

## Retirement and Post-Retirement Plans

- We have various defined benefit and defined contribution retirement plans.
- We sponsor post-retirement health care benefits for our eligible U.S. employees.

## Retirement of Treasury Shares

- Upon the formal retirement of treasury shares, we deduct the par value of the retired treasury shares from common stock and allocate the excess of cost over par as a deduction to additional paid-in capital and the remaining excess as a deduction to retained earnings.

## Share-Based Compensation

- We accounted for share-based awards made to our employees and directors using the estimated grant date fair value method of accounting.
- We recorded compensation expense for all share-based awards of $130 million in 2024, $112 million in 2023 and $126 million in 2022.

## Derivative Instruments

- We enter into foreign exchange hedging contracts, primarily forward contracts, interest rate swaps and interest rate locks to manage financial exposures resulting from changes in foreign currency exchange rates and interest rates.
- Derivative instruments are subject to master netting arrangements and are disclosed gross in the balance sheet.

## Foreign Currency Translation

- For those subsidiaries that operate in a local currency functional environment, all assets and liabilities are translated into U.S. dollars using current exchange rates at the balance sheet date.
- For those subsidiaries that operate in a U.S. dollar functional environment, foreign currency assets and liabilities are remeasured into U.S. dollars at current exchange rates except for non-monetary assets and capital accounts which are remeasured at historical exchange rates.

## New Accounting Pronouncements Not Yet Adopted

- In November 2023, the FASB issued guidance to improve segment reporting through enhanced disclosure requirements of significant segment expenses.
- In December 2023, the FASB issued guidance to expand the disclosure requirements for income taxes.
- In November 2024, the FASB issued guidance requiring new income statement disclosures to provide disaggregated information for certain types of costs and expenses included in each income statement line.

## Acquisition

- On September 20, 2024, we acquired 100 percent of the stock of BIOVECTRA for total consideration paid of $915 million in cash.


Other amendments to GAAP in the U.S. that have been issued by the FASB or other standards-setting bodies that do not require adoption until a future date are not expected to have a material impact on our consolidated financial statements upon adoption.

## Acquisition of BIOVECTRA

On September 20, 2024, we acquired 100 percent of the stock of BIOVECTRA for total consideration paid of $915 million in cash. The acquisition expands our contract development and manufacturing organization. As a result of the acquisition, BIOVECTRA became a wholly-owned subsidiary of Agilent. Accordingly, the results of BIOVECTRA are included in Agilent's consolidated financial statements from the acquisition date.

The BIOVECTRA acquisition was accounted for in accordance with the authoritative accounting guidance. The acquired assets and assumed liabilities were recorded at their estimated fair values. We determined the estimated fair values with the assistance of appraisals or valuations performed by third party specialists, discounted cash flow analyses, and estimates made by management. We expect to realize revenue synergies, leverage and expand the existing sales channels and product development resources, and utilize the assembled workforce. These factors, among others, contributed to a purchase price in excess of the estimated fair value of BIOVECTRA's net identifiable assets acquired (see summary of net assets below), and, as a result, we have recorded goodwill in connection with this transaction.

Goodwill acquired was allocated to our operating segments and reporting units as a part of the purchase price allocation. All goodwill was allocated to the diagnostics and genomics reporting unit.

Our acquisition of BIOVECTRA is treated as a stock acquisition and therefore is not deductible for United States federal tax purposes.

The following table summarizes the allocation of the purchase price to the estimated fair values of the assets acquired and liabilities assumed on the closing date of September 20, 2024 (in millions):

| Item | Amount (in millions) |
| --- | --- |
| Cash and cash equivalents | $56 |
| Accounts receivable | $36 |
| Inventories | $25 |
| Other current assets | $2 |
| Property, plant and equipment | $276 |
| Intangible assets | $183 |
| Goodwill | $526 |
| Total assets acquired | $1,104 |
| Accounts payable | $(10) |
| Other accrued liabilities | $(20) |
| Deferred revenue | $(70) |
| Deferred tax liability | $(45) |
| Other liabilities | $(19) |
| Debt | $(25) |
| Net assets acquired | $915 |

Pro forma results of operations and the revenue and net income subsequent to the acquisition date for BIOVECTRA have not been presented because the effects of the acquisition were not material to our financial results.

## Revenue

The following table presents the company's total revenue and segment revenue disaggregated by geographical region:

| Segment | Americas | Europe | Asia Pacific | Total |
| --- | --- | --- | --- | --- |
| Year Ended October 31, 2024: |  |  |  |  |
| Life Sciences and Applied Markets | $995 | $811 | $1,409 | $3,215 |
| Agilent CrossLab | $674 | $452 | $518 | $1,644 |
| Diagnostics and Genomics | $904 | $507 | $240 | $1,651 |
| Total | $2,573 | $1,770 | $2,167 | $6,510 |

The following table presents the company's total revenue disaggregated by end markets and by revenue type:

| Item | 2024 | 2023 | 2022 |
| --- | --- | --- | --- |
| Revenue by End Markets (in millions) |  |  |  |
| Pharmaceutical and Biopharmaceutical | $2,242 | $2,433 | $2,515 |
| Chemicals and Advanced Materials | $1,495 | $1,543 | $1,521 |
| Diagnostics and Clinical | $964 | $966 | $963 |
| Food | $592 | $628 | $617 |
| Academia and Government | $567 | $601 | $576 |
| Environmental and Forensics | $650 | $662 | $656 |
| Total | $6,510 | $6,833 | $6,848 |
| Revenue by Type (in millions) |  |  |  |
| Instrumentation | $2,354 | $2,742 | $2,907 |
| Non-instrumentation and other | $4,156 | $4,091 | $3,941 |
| Total | $6,510 | $6,833 | $6,848 |

## Contract Balances

### Contract Assets

Contract assets (unbilled accounts receivable) primarily relate to the company's right to consideration for work completed but not billed at the reporting date. The unbilled receivables are reclassified to trade receivables when billed to customers. Contract assets are generally classified as current assets and are included in "Accounts receivable, net" in the consolidated balance sheet. The balances of contract assets as of October 31, 2024 and 2023, were $247 million and $252 million, respectively.

### Contract Liabilities

The following table provides information about contract liabilities (deferred revenue) and the significant changes in the balances during the years ended October 31, 2023 and 2024:

| Item | Amount (in millions) |
| --- | --- |
| Ending balance as of October 31, 2022 | $557 |
| Net revenue deferred in the period | $488 |
| Revenue recognized that was included in the contract liability balance at the beginning of the period | $(409) |
| Change in deferrals from customer cash advances, net of revenue recognized | $(28) |
| Currency translation and other adjustments | $8 |
| Ending balance as of October 31, 2023 | $616 |
| Net revenue deferred in the period | $469 |
| Revenue recognized that was included in the contract liability balance at the beginning of the period | $(448) |
| Change in deferrals from customer cash advances, net of revenue recognized | $(9) |
| Contract liabilities acquired in business combinations | $70 |
| Currency translation and other adjustments | $3 |
| Ending balance as of October 31, 2024 | $701 |

## Share-based Compensation

Agilent accounts for share-based awards in accordance with the provisions of the accounting guidance which requires the measurement and recognition of compensation expense for all share-based payment awards made to our employees and directors including restricted stock units, employee stock options, employee stock purchases made under our employee stock purchase plan and performance share awards granted to selected members of our senior management under the long-term performance plan ("LTPP") based on estimated fair values.

### Description of Share-Based Plans

#### Employee Stock Purchase Plan

Effective May 1, 2020, we adopted the 2020 Employee Stock Purchase Plan ("ESPP") which replaced our previous Employee Stock Purchase Plan. The ESPP allows eligible employees to contribute up to 10 percent of their base compensation to purchase shares of our common stock at 85 percent of the closing market price at purchase date. There are 31 million shares authorized for issuance in connection with the ESPP.

#### Incentive Compensation Plans

On November 15, 2017 and March 21, 2018, the Board of Directors and the stockholders, respectively, approved the Agilent Technologies, Inc. 2018 Stock Plan (the "2018 Plan") which amends, including renaming and extending the term of, the Agilent Technologies, Inc. 2009 Stock Plan (the "2009 Plan"). The 2018 Plan provides for the grant of awards in the form of stock options, stock appreciation rights ("SARs"), restricted stock, restricted stock units ("RSUs"), performance shares and performance units with performance-based conditions on vesting or exercisability, and cash awards. The 2018 Plan has a term of ten years. As of October 31, 2024, 17,135,988 shares were available for future awards under the 2018 Plan.

### Impact of Share-based Compensation Awards

The impact on our results for share-based compensation was as follows:

| Item | 2024 | 2023 | 2022 |
| --- | --- | --- | --- |
| Cost of products and services | $41 | $34 | $30 |
| Research and development | $16 | $13 | $14 |
| Selling, general and administrative | $73 | $65 | $82 |
| Total share-based compensation expense | $130 | $112 | $126 |

At October 31, 2024 and 2023, no share-based compensation was capitalized within inventory.

### Share-Based Compensation

For all share-based awards, we have recognized compensation expense using a straight-line amortization method. As the guidance requires that share-based compensation expense should be based on awards that are ultimately expected to vest, estimated share-based compensation has been reduced for estimated forfeitures.

The impact on our results for share-based compensation was as follows:

Years Ended October 31,
2024 | 2023 | 2022
--- | --- | ---
(in millions) |  |  
Cost of products and services | $41 | $34 | $30
Research and development | 16 | 13 | 14
Selling, general and administrative | 73 | 65 | 82
Total share-based compensation expense | $130 | $112 | $126

At October 31, 2024 and 2023, no share-based compensation was capitalized within inventory.

#### Valuation Assumptions

The fair value of share-based awards for our employee stock option awards was estimated using the Black-Scholes option pricing model. Shares granted under the LTPP (TSR) were valued using a Monte Carlo simulation model.

The Monte Carlo simulation fair value model requires the use of highly subjective and complex assumptions, including the price volatility of the underlying stock. For the volatility of our LTPP (TSR) grants, we used our own historical stock price volatility.

The ESPP allows eligible employees to purchase shares of our common stock at 85 percent of the price at purchase and uses the purchase date to establish the fair market value.

We use historical volatility to estimate the expected stock price volatility assumption for employee stock option awards. In reaching the conclusion, we have considered many factors including the extent to which our options are currently traded and our ability to find traded options in the current market with similar terms and prices to the options we are valuing. In estimating the expected life of our options granted, we considered the historical option exercise behavior of our executives, which we believe is representative of future behavior.

The estimated fair value of restricted stock units and LTPP (EPS) awards is determined based on the market price of our common stock on the date of grant adjusted for expected dividend yield. The compensation cost for LTPP (EPS) reflects the cost of awards that are probable to vest at the end of the performance period.

All LTPP awards granted to our senior management employees have a one-year post-vest holding restriction. The estimated discount associated with post-vest holding restrictions is calculated using the Finnerty model. The model calculates the potential lost value if the employees were able to sell the shares during the lack of marketability period, instead of being required to hold the shares. The model used the same historical stock price volatility and dividend yield assumption used for the Monte Carlo simulation model and an expected dividend yield to compute the discount.

The following assumptions were used to estimate the fair value of awards granted.

Years Ended October 31,
2024 | 2023 | 2022
--- | --- | ---
Stock Option Plan: |  |  
Weighted average risk-free interest rate | 4.4% | 3.9% | 1.5%
Dividend yield | 0.8% | 0.6% | 0.5%
Weighted average volatility | 29% | 28% | 26%
Expected life | 5.5 years | 5.5 years | 5.5 years
LTPP: |  |  
Volatility of Agilent shares | 28% | 31% | 29%
Volatility of selected peer-company shares | 16%-70% | 22%-84% | 23%-81%
Pair-wise correlation with selected peers | 30% | 42% | 41%
Post-vest restriction discount for all executive awards | 6.4% | 7.1% | 6.5%

### Share-Based Payment Award Activity

#### Employee Stock Options

The following table summarizes employee stock option award activity of our employees and directors for 2024.

Options | Outstanding | Weighted Average Exercise Price
--- | --- | ---
(in thousands) |  |  
Outstanding at October 31, 2023 | 1,080 | $118
Granted | 335 | $126
Exercised | (300) | $64
Cancelled | (110) | $140
Outstanding at October 31, 2024 | 1,005 | $134

The options outstanding and exercisable for equity share-based payment awards at October 31, 2024 were as follow:

Options Outstanding | Options Exercisable
--- | ---
Range of Exercise Prices | Number Outstanding | Weighted Average Remaining Contractual Life | Weighted Average Exercise Price | Aggregate Intrinsic Value | Number Exercisable | Weighted Average Remaining Contractual Life | Weighted Average Exercise Price | Aggregate Intrinsic Value
(in thousands) | (in years) |  | (in thousands) | (in thousands) | (in years) |  | (in thousands)
$100.00 - $110.00 | 242 | 6.0 | $110 | 4,951 | 187 | 6.0 | $110 | 3,832
$110.01 - $150.00 | 559 | 8.6 | $134 | 1,930 | 95 | 8.0 | $143 | 139
$150.01 & Over | 204 | 7.0 | $161 | — | 120 | 7.0 | $161 | —
 | 1,005 | 7.7 | $134 | $6,881 | 402 | 6.8 | $133 | $3,971

The aggregate intrinsic value in the table above represents the total pre-tax intrinsic value, based on the company's closing stock price of $130.31 at October 31, 2024, which would have been received by award holders had all award holders exercised their awards that were in-the-money as of that date. The total number of in-the-money awards exercisable at October 31, 2024 was approximately 0.2 million.

The following table summarizes the aggregate intrinsic value of options exercised and the fair value of options granted in 2024, 2023 and 2022:

 | Aggregate Intrinsic Value | Weighted Average Exercise Price | Per Share Value Using Black-Scholes Model
--- | --- | --- | ---
(in thousands) |  |  
Options exercised in fiscal 2022 | $10,765 | $38 | $39
Options exercised in fiscal 2023 | $25,303 | $41 | $47
Options exercised in fiscal 2024 | $22,762 | $64 | $41

As of October 31, 2024, the unrecognized share-based compensation cost for outstanding stock option awards, net of expected forfeitures, was $10 million. The amount of cash received from the exercise of share-based awards granted was $77 million in 2024, $67 million in 2023 and $58 million in 2022.

#### Non-Vested Awards

The following table summarizes non-vested award activity in 2024 primarily for our LTPP and restricted stock unit awards.

Shares | Weighted Average Grant Price
--- | ---
(in thousands) |  
Non-vested at October 31, 2023 | 1,889 | $136
Granted | 1,200 | $126
Vested | (855) | $121
Forfeited | (129) | $139
Change in LTPP shares in the year due to exceeding performance targets | 31 | $111
Non-vested at October 31, 2024 | 2,136 | $136

As of October 31, 2024, the unrecognized share-based compensation cost for non-vested restricted stock awards net of expected forfeitures was approximately $129 million which is expected to be amortized over a weighted average period of 2.1 years. The total fair value of restricted stock awards vested was $103 million for 2024, $99 million for 2023 and $89 million for 2022.

### Income Taxes

The domestic and foreign components of income before taxes are:

Years Ended October 31,
2024 | 2023 | 2022
--- | --- | ---
(in millions) |  |  
U.S. operations | $391 | $614 | $858
Non-U.S. operations | 1,130 | 725 | 646
Total income before taxes | $1,521 | $1,339 | $1,504

The provision for income taxes is comprised of:

Years Ended October 31,
2024 | 2023 | 2022
--- | --- | ---
(in millions) |  |  
U.S. federal taxes: |  |  
Current | $182 | $117 | $173
Deferred | $(104) | $(84) | $(28)
Non-U.S. taxes: |  |  
Current | 87 | 26 | 47
Deferred | 60 | 38 | 35
State taxes, net of federal benefit: |  |  
Current | 27 | 12 | 22
Deferred | $(20) | $(10) | 1
Total provision for income taxes | $232 | $99 | $250

The differences between the U.S. federal statutory income tax rate and our effective tax rate are:

Years Ended October 31,
2024 | 2023 | 2022
--- | --- | ---
(in millions) |  |  
Profit before tax times statutory rate | $319 | $281 | $316
State income taxes, net of federal benefit | 7 | 2 | 23
Non-U.S. income taxed at different rates | $(14) | 20 | $(18)
Change in unrecognized tax benefits | $(8) | $(35) | $(6)
Foreign-derived intangible income deduction | $(47) | $(41) | $(46)
Realized loss on divestiture of business | — | $(104) | —
Excess tax benefits from stock-based compensation | $(4) | $(14) | $(19)
Other, net | $(21) | $(10) | —
Provision (benefit) for income taxes | $232 | $99 | $250
Effective tax rate | 15.3% | 7.4% | 16.6%

We have negotiated a tax holiday in Singapore. The tax holiday provides a lower rate of taxation on certain classes of income and requires various thresholds of investments and employment or specific types of income. The tax holiday in Singapore was renegotiated and extended through 2030. As a result of the incentive, the impact of the tax holiday decreased income taxes by $84 million, $54 million, and $53 million in 2024, 2023, and 2022, respectively. The benefit of the tax holiday on net income per share (diluted) was approximately $0.29, $0.18, and $0.18 in 2024, 2023 and 2022, respectively.

The significant components of deferred tax assets and deferred tax liabilities included on the consolidated balance sheet are:

Years Ended October 31,
2024 | 2023
--- | ---
(in millions) |  
Deferred Tax Assets |  
Intangibles | $20 | $102
Employee benefits, other than retirement | 31 | 36
Net operating loss, capital loss, and credit carryforwards | 184 | 152
Deferred revenue | 98 | 16
Share-based compensation | 25 | 24
Capitalized R&D | 93 | 41
Lease obligations | 39 | 37
Other | 35 | 42
Deferred tax assets | $525 | $450
Tax valuation allowance | $(113) | $(112)
Deferred tax assets, net of valuation allowance | $412 | $338
Deferred Tax Liabilities |  
Property, plant and equipment | $(62) | $(26)
Pension benefits and retiree medical benefits | $(41) | $(25)
Right-of-use asset | $(39) | $(37)
Other | $(4) | $(4)
Deferred tax liabilities | $(146) | $(92)
Net deferred tax assets (liabilities) | $266 | $246

#### Uncertain Tax Positions

The aggregate changes in the balances of our gross unrecognized tax benefits including all federal, state and foreign tax jurisdictions are as follows:

2024 | 2023 | 2022
--- | --- | ---
(in millions) |  |  
Balance, beginning of year | $98 | $123 | $133
Additions for tax positions related to the current year | 6 | 5 | 5
Additions for tax positions from prior years | 3 | 3 | —
Reductions for tax positions from prior years | $(1) | $(27) | $(9)
Statute of limitations expirations | $(9) | $(6) | $(6)
Balance, end of year | $97 | $98 | $123

As of October 31, 2024, we had $114 million of unrecognized tax benefits, including interest and penalties of which $91 million, if recognized, would affect our effective tax rate.

Interest and penalties accrued as of October 31, 2024 and 2023 were $17 million and $16 million, respectively. We recognized tax expense of $1 million in 2024, tax benefit of $5 million in 2023, and tax benefit of $2 million in 2022, for interest and penalties related to unrecognized tax benefits.

In the U.S., tax years remain open back to the year 2021 for federal income tax purposes and 2020 for significant states. In other major jurisdictions where we conduct business, the tax years generally remain open back to the year 2014.

With these jurisdictions and the U.S., it is reasonably possible that some tax audits may be completed over the next twelve months. However, management is n

### 7. NET INCOME PER SHARE

Years Ended October 31,
2024 | 2023 | 2022
--- | --- | ---
(in millions)
Numerator:
Net income | $1,289 | $1,240 | $1,254
Denominators:
Basic weighted average shares | 290 | 294 | 299
Potential common shares — stock options and other employee stock plans | 1 | 2 | 1
Diluted weighted average shares | 291 | 296 | 300

### 8. INVENTORY

October 31,
2024 | 2023
--- | ---
(in millions)
Finished goods | $523 | $570
Purchased parts and fabricated assemblies | 449 | 461
Inventory | $972 | $1,031

### 9. PROPERTY, PLANT AND EQUIPMENT, NET

October 31,
2024 | 2023
--- | ---
(in millions)
Land | $69 | $60
Buildings and leasehold improvements | 1,786 | 1,409
Machinery and equipment | 960 | 749
Software | 267 | 275
Total property, plant and equipment | 3,082 | 2,493
Accumulated depreciation and amortization | (1,304) | (1,223)
Property, plant and equipment, net | $1,778 | $1,270

### 10. LEASES

Year Ended October 31,
2024 | 2023 | 2022
--- | --- | ---
(in millions)
Operating lease cost | $58 | $68 | $59
Short-term lease cost | — | 2 | 2
Variable lease cost | 15 | 16 | 15
Sublease income | (17) | (16) | (14)
Total lease cost | $56 | 70 | 62

October 31,
Financial Statement Line Item | 2024 | 2023
--- | --- | ---
(in millions, except lease term and discount rate)
Assets:
Operating lease:
Right of use asset | Other assets | $177 | $154
Liabilities:
Current
Operating lease liabilities | Other accrued liabilities | $42 | $46
Long-term
Operating lease liabilities | Other long-term liabilities | $142 | $118
Weighted average remaining lease term (in years)
Operating leases | 8.2 years | 8.3 years
Weighted average discount rate
Operating leases | 3.7% | 3.3%

### 11. GOODWILL AND OTHER INTANGIBLE ASSETS

Life Sciences and Applied Markets | Diagnostics and Genomics | Agilent CrossLab | Total
--- | --- | --- | ---
(in millions)
Goodwill as of October 31, 2022 | $1,575 | $2,121 | $256 | $3,952
Foreign currency translation impact | 4 | 1 | 1 | 6
Goodwill arising from acquisitions and adjustments | — | 15 | — | 15
Goodwill adjustment related to divestiture of business | — | (13) | — | (13)
Goodwill as of October 31, 2023 | $1,579 | $2,124 | $257 | $3,960
Foreign currency translation impact | 4 | (15) | 2 | (9)
Goodwill arising from acquisitions and adjustments | — | 526 | — | 526
Goodwill as of October 31, 2024 | $1,583 | $2,635 | $259 | $4,477

Other Intangible Assets | Gross Carrying Amount | Accumulated Amortization | Net Book Value
--- | --- | --- | ---
(in millions)
As of October 31, 2023:
Purchased technology | $1,467 | $1,093 | $374
Trademark/Trade name | 196 | 163 | 33
Customer relationships | 149 | 112 | 37
Third-party technology and licenses | 34 | 13 | 21
Total amortizable intangible assets | $1,846 | $1,381 | $465
In-Process R&D | 10 | — | 10
Total | $1,856 | $1,381 | $475
As of October 31, 2024:
Purchased technology | $1,484 | $1,169 | $315
Backlog | 9 | — | 9
Trademark/Trade name | 199 | 174 | 25
Customer relationships | 291 | 107 | 184
Third-party technology and licenses | 33 | 19 | 14
Total amortizable intangible assets | $2,016 | $1,469 | $547
In-Process R&D | — | — | —
Total | $2,016 | $1,469 | $547

### 12. Investments

The following table summarizes the company's equity investments as of October 31, 2024 and 2023 (net book value):

October 31,
2024 | 2023
--- | ---
(in millions) |
Long-Term |
Equity investments - without readily determinable fair value | $101 | $102
Other investments - with readily determinable fair value | 31 | 26
Trading securities | 43 | 36
Total long-term investments | $175 | $164

Equity investments without readily determinable fair value (RDFV) consist of non-marketable equity securities issued by private companies and include VIEs. These investments are accounted for using the measurement alternative at cost adjusting for impairments and observable price changes (orderly transactions for the identical or a similar security from the same issuer). The adjustments are included in net income in the period in which they occur. Other investments with RDFV consist of shares we own in a special fund and are reported at fair value, with gains or losses resulting from changes in fair value included in net income.

Trading securities, which are comprised of mutual funds, bonds and other similar instruments, other investments and deferred compensation liability are reported at fair value, with gains or losses resulting from changes in fair value recognized currently in net income.

Our investments without RDFV and marketable equity securities with RDFV are subject to periodic impairment review. The impairment analysis requires significant judgment to identify events or circumstances that would likely have a significant adverse effect on the future value of the investment.

Gains and losses reflected in other income (expense), net for our equity investments with RDFV and equity investments without RDFV are summarized below:

Years Ended October 31,
2024 | 2023 | 2022
--- | --- | ---
(in millions) |
Net gain (loss) recognized during the period on equity securities | $6 | $(41) | $(67)
Less: Net gain (loss) on equity securities sold during the period | — | (15) | 11
Unrealized gain (loss) on equity securities held as of the end of the period | $6 | $(26) | $(78)

In 2024, unrealized gains on our equity securities without RDFV were $1 million. In 2023, unrealized losses on our equity securities without RDFV were $26 million. In 2022, unrealized gains on our equity securities without RDFV were $6 million.

In 2024, net unrealized gains on our trading securities were $10 million. In 2023, net unrealized gains were $2 million on our trading securities. In 2022, net unrealized losses were $7 million on our trading securities.

In 2024, we recorded $11 million impairment of investments. There were no impairments of investments in 2023 and 2022.

### 13. Fair Value Measurements

The authoritative guidance defines fair value as the price that would be received from selling an asset or paid to transfer a liability in an orderly transaction between market participants at the measurement date. When determining the fair value measurements for assets and liabilities required or permitted to be recorded at fair value, we consider the principal or most advantageous market and assumptions that market participants would use when pricing the asset or liability.

#### Fair Value Hierarchy

The guidance establishes a fair value hierarchy that prioritizes the use of inputs used in valuation techniques into three levels. A financial instrument's categorization within the fair value hierarchy is based upon the lowest level of input that is significant to the fair value measurement. There are three levels of inputs that may be used to measure fair value:

##### Level 1
- applies to assets or liabilities for which there are quoted prices in active markets for identical assets or liabilities.

##### Level 2
- applies to assets or liabilities for which there are inputs other than quoted prices included within level 1 that are observable, either directly or indirectly, for the asset or liability such as: quoted prices for similar assets or liabilities in active markets; quoted prices for identical or similar assets or liabilities in less active markets; or other inputs that can be derived principally from, or corroborated by, observable market data.

##### Level 3
- applies to assets or liabilities for which there are unobservable inputs to the valuation methodology that are significant to the measurement of the fair value of the assets or liabilities.

#### Financial Assets and Liabilities Measured at Fair Value on a Recurring Basis

Financial assets and liabilities measured at fair value on a recurring basis as of October 31, 2024 were as follows:

Fair Value Measurement
at October 31, 2024 Using
October 31,
2024 | Quoted Prices
in Active
Markets for
Identical Assets
(Level 1) | Significant
Other
Observable
Inputs
(Level 2) | Significant
Unobservable
Inputs
(Level 3)
--- | --- | --- | ---
(in millions) |
Assets: |
Short-term |
Cash equivalents (money market funds) | $800 | $800 | $— | $—
Derivative instruments (foreign exchange contracts) | 14 | — | 14 | —
Long-term |
Trading securities | 43 | 43 | — | —
Other investments | 31 | — | 31 | —
Total assets measured at fair value | $888 | $843 | $45 | $—
Liabilities: |
Short-term |
Derivative instruments (foreign exchange contracts) | $12 | $— | $12 | $—
Long-term |
Deferred compensation liability | 43 | — | 43 | —
Total liabilities measured at fair value | $55 | $— | $55 | $—

Financial assets and liabilities measured at fair value on a recurring basis as of October 31, 2023 were as follows:

Fair Value Measurement
at October 31, 2023 Using
October 31,
2023 | Quoted Prices
in Active
Markets for
Identical Assets
(Level 1) | Significant
Other
Observable
Inputs
(Level 2) | Significant
Unobservable
Inputs
(Level 3)
--- | --- | --- | ---
(in millions) |
Assets: |
Short-term |
Cash equivalents (money market funds) | $994 | $994 | $— | $—
Derivative instruments (foreign exchange contracts) | 19 | — | 19 | —
Long-term |
Trading securities | 36 | 36 | — | —
Other investments | 26 | — | 26 | —
Total assets measured at fair value | $1,075 | $1,030 | $45 | $—
Liabilities: |
Short-term |
Derivative instruments (foreign exchange contracts) | $2 | $— | $2 | $—
Contingent consideration | 1 | — | — | 1
Long-term |
Deferred compensation liability | 36 | — | 36 | —
Total liabilities measured at fair value | $39 | $— | $38 | $1

Our money market funds and trading securities are generally valued using quoted market prices and therefore are classified within level 1 of the fair value hierarchy. Our derivative financial instruments are classified within level 2, as there is not an active market for each hedge contract, but the inputs used to calculate the value of the instruments are tied to active markets. Our deferred compensation liability is classified as level 2 because, although the values are not directly based on quoted market prices, the inputs used in the calculations are observable.

Trading securities, which are comprised of mutual funds, bonds and other similar instruments, other investments and deferred compensation liability are reported at fair value, with gains or losses resulting from changes in fair value recognized currently in net income. Certain derivative instruments are reported at fair value, with unrealized gains and losses, net of tax, included in accumulated other comprehensive income (loss) within stockholders' equity. Realized gains and losses from the sale of these instruments are recorded in net income.

Other investments represent shares we own in a special fund that targets underlying investments of approximately 40 percent in debt securities and 60 percent in equity securities. These shares have been classified as level 2 because, although the shares of the fund are not traded on any active stock exchange, each of the individual underlying securities are or can be derived from and hence we have a readily determinable value for the underlying securities, from which we are able to determine the fair market value for the special fund itself.

#### Contingent Consideration

The contingent consideration liability was our only Level 3 asset or liability. A summary of the Level 3 activity follows:

Contingent Consideration
(in millions)
Balance at October 31, 2022 | $67
Additions to contingent consideration | 5
Payments | (72)
Change in fair value (included within selling, general and administrative expenses) | 1
Balance at October 31, 2023 | $1
Change in fair value (included within selling, general and administrative expenses) | (1)
Balance at October 31, 2024 | $—

During fiscal year 2023, we made contingent consideration payments totaling $72 million related to the achievement of certain technical milestones associated with our acquisition of Resolution Bioscience and another acquisition.

Resolution Bioscience. In the third quarter of fiscal year 2023, we decided to exit the Resolution Bioscience business and subsequently divested our interest in the business in the fourth quarter of fiscal year 2023. We project that there are no potential future milestone payments related to the Resolution Bioscience business.

#### Assets and Liabilities Measured at Fair Value on a Non-Recurring Basis

##### Long-Lived Assets

For assets measured at fair value on a non-recurring basis, the following table summarizes the impairments included in net income for the years ended October 31, 2024, 2023 and 2022:

Years Ended
October 31,
2024 | 2023 | 2022
--- | --- | ---
(in millions) |
Long-lived assets held and used | $19 | $277 | $—

For the year ended October 31, 2024, long-lived assets held and used with a carrying amount of $19 million were written down to fair value of zero resulting in an impairment charge of $19 million. For the year ended October 31, 2023, long-lived assets held and used with a carrying amount of $277 million were written down to fair value of zero, resulting in an impairment charge of $277 million primarily related to the exit of our Resolution Bioscience business in our diagnostics and genomics segment. For the year ended October 31, 2022, there were no impairments of long-lived assets held and used.

Fair values for the impaired long-lived assets during 2023 were measured using level 3 inputs. To determine the fair value of long-lived assets in 2023, we primarily used an estimate of undiscounted future cash flows expected over the life of the primary asset. Since the carrying value was greater than the undiscounted cash flow, the loss was measured by the excess of the carrying amount of the asset over its fair value of zero.

##### Non-Marketable Equity Securities

For the year ended October 31, 2024, the unrealized gain (loss) on our non-marketable equity securities without readily determinable fair values was comprised of $2 million of upward adjustments, $1 million of downward adjustments and an impairment of $11 million which were included in net income as adjustments to the carrying value.

For the year ended October 31, 2023, the unrealized gain (loss) on our non-marketable equity securities without readily determinable fair values was comprised of only downward adjustments of $26 million which were included in net income as adjustments to the carrying value.

For the year ended October 31, 2022, the unrealized gain (loss) on our non-marketable equity securities without readily determinable fair values was comprised of only upward adjustments of $6 million which were included in net income as adjustments to the carrying value.

As of October 31, 2024, the cumulative net gain (loss) on our non-marketable equity securities without readily determinable fair values was comprised of $40 million upward adjustments, $11 million impairment loss and $30 million downward adjustments, and the carrying amount was $101 million.

As of October 31, 2023, the cumulative net gain (loss) on our non-marketable equity securities without readily determinable fair values was comprised of a $38 million upward adjustments and $29 million downward adjustments, and the carrying amount was $102 million.

Fair values for the non-marketable securities included in long-term investments on the consolidated balance sheet were measured using Level 3 inputs.


We are exposed to foreign currency exchange rate fluctuations and interest rate changes in the normal course of our business. As part of our risk management strategy, we use derivative instruments, primarily forward contracts and purchased options to hedge economic and/or accounting exposures resulting from changes in foreign currency exchange rates.

### Cash Flow Hedges

We enter into foreign exchange contracts to hedge our forecasted operational cash flow exposures resulting from changes in foreign currency exchange rates. These foreign exchange contracts, carried at fair value, have maturities between one and twelve months. These derivative instruments are designated and qualify as cash flow hedges under the criteria prescribed in the authoritative guidance and are assessed for effectiveness against the underlying exposure every reporting period. For open contracts as of October 31, 2024, changes in the time value of the foreign exchange contract are excluded from the assessment of hedge effectiveness and are recognized in cost of sales over the life of the foreign exchange contract. The changes in fair value of the effective portion of the derivative instrument are recognized in accumulated other comprehensive income (loss). Amounts associated with cash flow hedges are reclassified to cost of sales in the consolidated statement of operations when the forecasted transaction occurs. If it becomes probable that the forecasted transaction will not occur, the hedge relationship will be de-designated and amounts accumulated in other comprehensive income (loss) will be reclassified to other income (expense), net in the current period. Changes in the fair value of the ineffective portion of derivative instruments are recognized in other income (expense), net in the consolidated statement of operations in the current period. We record the premium paid (time value) of an option on the date of purchase as an asset. For options designated as cash flow hedges, changes in the time value are excluded from the assessment of hedge effectiveness and are recognized in cost of sales over the life of the option contract. For the years ended October 31, 2024, 2023 and 2022, ineffectiveness and gains and losses recognized in other income (expense), net due to de-designation of cash flow hedge contracts were not significant.

In February 2016, we executed three forward-starting pay fixed/receive variable interest rate swaps for the notional amount of $300 million in connection with future interest payments to be made on our 2026 senior notes issued on September 15, 2016. These derivative instruments were designated and qualified as cash flow hedges under the criteria prescribed in the authoritative guidance. The swap arrangements were terminated on September 15, 2016 with a payment of $10 million, and we recognized this as a deferred loss in accumulated other comprehensive income (loss) which is being amortized to interest expense over the life of the 2026 senior notes. The remaining loss to be amortized related to the interest rate swap agreements at October 31, 2024 was $2 million.

In August 2019, we executed treasury lock agreements for $250 million in connection with future interest payments to be made on our 2029 senior notes issued on September 16, 2019. We designated the treasury lock as a cash flow hedge. The treasury lock contracts were terminated on September 6, 2019 and we recognized a deferred loss of $6 million in accumulated other comprehensive income (loss) which is being amortized to interest expense over the life of the 2029 senior notes. The remaining loss to be amortized related to the treasury lock agreements at October 31, 2024 was $3 million.

### Net Investment Hedges

We enter into foreign exchange contracts to hedge net investments in foreign operations to mitigate the risk of adverse movements in exchange rates. These foreign exchange contracts are carried at fair value and are designated and qualify as net investment hedges under the criteria prescribed in the authoritative guidance. Changes in fair value of the effective portion of the derivative instrument are recognized in accumulated other comprehensive income (loss)- translation adjustment and are assessed for effectiveness against the underlying exposure every reporting period. If the company's net investment changes during the year, the hedge relationship will be assessed and de-designated if the hedge notional amount is outside of prescribed tolerance with a gain/loss reclassified from other comprehensive income (loss) to other income (expense) in the current period. For the years ended October 31, 2024, 2023 and 2022, ineffectiveness and the resultant effect of any gains or losses recognized in other income (expense) due to de-designation of the hedge contracts were not significant.

### Other Hedges

Additionally, we enter into foreign exchange contracts to hedge monetary assets and liabilities that are denominated in currencies other than the functional currency of our subsidiaries. These foreign exchange contracts are carried at fair value and do not qualify for hedge accounting treatment and are not designated as hedging instruments. Changes in value of the derivative instruments are recognized in other income (expense), net in the consolidated statement of operations, in the current period, along with the offsetting foreign currency gain or loss on the underlying assets or liabilities.

Our use of derivative instruments exposes us to credit risk to the extent that the counterparties may be unable to meet the terms of the agreement. We do, however, seek to mitigate such risks by limiting our counterparties to major financial institutions which are selected based on their credit ratings and other factors. We have established policies and procedures for mitigating credit risk that include establishing counterparty credit limits, monitoring credit exposures, and continually assessing the creditworthiness of counterparties.

A number of our derivative agreements contain threshold limits to the net liability position with counterparties and are dependent on our corporate credit rating determined by the major credit rating agencies. The counterparties to the derivative instruments may request collateralization, in accordance with derivative agreements, on derivative instruments in net liability positions.

The aggregate fair value of all derivative instruments with credit-risk-related contingent features that were in a net liability position as of October 31, 2024, was $7 million. The credit-risk-related contingent features underlying these agreements had not been triggered as of October 31, 2024.

The number of open foreign exchange forward contracts and aggregated notional amounts by designation as of October 31, 2024 were as follows:

| Number of Open Forward Contracts | Aggregate Notional Amount USD Buy/(Sell) ($ in millions) |
| --- | --- |
| Derivatives designated as hedging instruments: |  |
| Cash Flow Hedges Foreign exchange forward contracts | 321 $(-542) |
| Net Investment Hedges Foreign exchange forward contracts | 3 $(-33) |
| Derivatives not designated as hedging instruments: |  |
| Foreign exchange forward contracts | 201 $(-19) |

Derivative instruments are subject to master netting arrangements and are disclosed gross in the balance sheet in accordance with the authoritative guidance.

The gross fair values and balance sheet location of derivative instruments held in the consolidated balance sheet as of October 31, 2024 and 2023 were as follows:

| Fair Values of Derivative Instruments | Asset Derivatives |  | Liability Derivatives |  |
| --- | --- | --- | --- | --- |
|  | Fair Value | Fair Value | Fair Value | Fair Value |
|  | October 31, 2024 | October 31, 2023 | October 31, 2024 | October 31, 2023 |
|  | (in millions) |
| Derivatives designated as hedging instruments: |  |  |  |  |
| Cash flow hedges |  |  |  |  |
| Foreign exchange contracts | $4 | $15 | $2 | $1 |
| Net investment hedges |  |  |  |  |
| Foreign exchange contracts | $- | $1 | $- | $- |
| Derivatives not designated as hedging instruments: |  |  |  |  |
| Foreign exchange contracts | $10 | $3 | $10 | $1 |
| Total derivatives | $14 | $19 | $12 | $2 |

The effects of derivative instruments for foreign exchange contracts designated as hedging instruments and not designated as hedging instruments in our consolidated statement of operations were as follows:

| Years Ended October 31, | 2024 | 2023 | 2022 |
| --- | --- | --- | --- |
|  | (in millions) |
| Derivatives designated as hedging instruments: |  |  |  |
| Cash flow hedges |  |  |  |
| Foreign exchange contracts: |  |  |  |
| Gain (loss) recognized in accumulated other comprehensive income (loss) | $(9) | $(4) | $56 |
| Gain (loss) reclassified from accumulated other comprehensive income (loss) into cost of sales | $4 | $2 | $36 |
| Gain (loss) reclassified from accumulated other comprehensive income (loss) into interest expense | $(2) | $(2) | $(2) |
| Gain on time value of forward contracts recorded in cost of sales | $7 | $7 | $- |
| Net investment hedges |  |  |  |
| Foreign exchange contracts: |  |  |  |
| Gain (loss) recognized in accumulated other comprehensive income (loss) - translation adjustment | $- | $(1) | $5 |
| Derivatives not designated as hedging instruments: |  |  |  |
| Gain (loss) recognized in other income (expense), net | $2 | $3 | $10 |

At October 31, 2024 the total amount of existing net gain that is expected to be reclassified from accumulated other comprehensive income (loss) is $9 million. Within the next twelve months it is estimated that $1 million of loss included within the net amount of accumulated other comprehensive income (loss) will be reclassified to cost of sales in respect of cash flow hedges.

## 15. Retirement Plans and Post Retirement Pension Plans

### General

We have various defined benefit and defined contribution retirement plans. Additionally, we sponsor post-retirement health care benefits for our eligible U.S. employees.

Agilent provides defined benefits to U.S. employees who meet eligibility criteria under the Agilent Technologies, Inc. Retirement Plan (the "RP"). Effective November 1, 2014, Agilent's U.S. RP was closed to new entrants including new employees, new transfers to the U.S. payroll and rehires. As of April 30, 2016, benefits under the RP were frozen. Any pension benefit earned in the U.S. Plans through April 30, 2016, remained fully vested and is payable on termination, retirement, death, or permanent disability, based on an eligible participant's years of credited service, age and other criteria. There are no additional benefit accruals after April 30, 2016.

For eligible service through October 31, 1993, the benefit payable under the Agilent RP is reduced by any amounts due to the eligible employee under the Agilent Technologies, Inc. Deferred Profit-Sharing Plan (the "DPSP"), which is a defined contribution plan that was frozen and closed to new participants as of November 1993.

As of October 31, 2024 and 2023, the fair value of plan assets of the DPSP was $74 million and $81 million, respectively. Note that the projected benefit obligation for the DPSP equals the fair value of plan assets.

Agilent also maintains a Supplemental Benefit Retirement Plan ("SBRP") in the U.S., which is an unfunded non-qualified defined benefit plan to provide supplemental retirement benefits to certain employees that would be provided under the RP but for limitations imposed by the Internal Revenue Code. The RP and the SBRP comprise the "U.S. defined benefit plans" in the tables below.

Eligible employees outside the U.S. generally receive retirement benefits under various retirement plans based upon factors such as years of service and/or employee compensation levels. Eligibility is generally determined in accordance with local statutory requirements.

### Post-Retirement Medical Benefit Plans

In addition to receiving retirement benefits, Agilent U.S. employees who meet eligibility requirements as of their termination date may participate in the Agilent Technologies, Inc. Health Plan for Retirees. As of January 1, 2020, the Health Plan for Retirees is comprised solely of insured pre-65 HMOs as the self-funded Pre-Medicare Medical Plan was eliminated effective December 31, 2019. The Health Plan for Retirees was closed to new retiree entrants after December 31, 2020.

If eligible, a retiree may receive a fixed amount (different fixed amounts for different groups) under the Agilent Technologies, Inc. Retiree Medical Account Plan ("RMA") or a fixed monthly amount under the Agilent Technologies, Inc. Reimbursement Arrangement Plan ("ARA").

Any new employee hired on or after November 1, 2014, will not be eligible to participate in the post-retirement medical benefit plans upon retiring.

### 401(k) and Other Defined Contribution Plans

Eligible Agilent U.S. employees may participate in the Agilent Technologies, Inc. 401(k) Plan. We match an employee's contributions (both pre-tax and Roth) up to a maximum of 6 percent of an employee's annual eligible compensation, subject to the annual regulatory limit. All matching contributions vest immediately. Effective May 1, 2016 until April 30, 2022, we provided an additional transitional company contribution for certain eligible employees equal to 3 percent, 4 percent or 5 percent of an employee's annual eligible compensation due to the RP benefits.


- Eligible Agilent U.S. employees may participate in the Agilent Technologies, Inc. 401(k) Plan.
- We match an employee's contributions (both pre-tax and Roth) up to a maximum of 6 percent of an employee's annual eligible compensation, subject to the annual regulatory limit. All matching contributions vest immediately.
- Effective May 1, 2016 until April 30, 2022, we provided an additional transitional company contribution for certain eligible employees equal to 3 percent, 4 percent or 5 percent of an employee's annual eligible compensation due to the RP benefits being frozen.
- The maximum employee contribution to the 401(k) Plan is 50 percent of an employee's annual eligible compensation, subject to regulatory limitations.
- We also sponsor and make contributions to various other defined contribution plans that cover employees outside of the U.S.

Our defined contribution plan expenses included in income from operations were as follows:

Years Ended October 31,
| 2024 | 2023 | 2022 |
| --- | --- | --- |
| Contributions to the 401(k) Plan | $46 | $47 | $46 |
| Contributions to plans outside the U.S. | $51 | $51 | $47 |
| Total defined contribution plan expense | $97 | $98 | $93 |

## Components of net periodic benefit cost (income)

- The service cost component is recorded in cost of sales and operating expenses in the consolidated statement of operations.
- All other cost components are recorded in other income (expense), net in the consolidated statement of operations.
- For U.S. defined benefit plans, gains and losses are amortized over the average future lifetime of participants using the corridor method.
- For most Non-U.S. defined benefit plans and U.S. Post-Retirement Benefit Plans, gains and losses are amortized over the average remaining future service period or remaining lifetime of participants depending upon the plan, using a separate layer for each year's gains and losses.

For the years ended October 31, 2024, 2023 and 2022, components of net periodic benefit cost and other amounts recognized in other comprehensive income were comprised of:

| Pensions | U.S. Post-Retirement Benefit Plans |
| --- | --- |
| U.S. Defined Benefit Plans | Non-U.S. Defined Benefit Plans |
| 2024 | 2023 | 2022 | 2024 | 2023 | 2022 | 2024 | 2023 | 2022 |
| Net periodic benefit cost (income) |
| Service cost - benefits earned during the period | $- | $- | $- | $15 | $16 | $22 | $1 | $- | $1 |
| Interest cost on benefit obligation | $21 | $21 | $14 | $26 | $24 | $9 | $4 | $4 | $2 |
| Expected return on plan assets | $(21) | $(19) | $(27) | $(37) | $(36) | $(43) | $(4) | $(4) | $(6) |
| Amortization of net actuarial (gain) loss | $2 | $- | $- | $(16) | $(2) | $25 | $(1) | $(1) | $(2) |
| Amortization of prior service benefit | $- | $- | $- | $- | $- | $- | $(1) | $(1) | $(1) |
| Total net periodic benefit cost (income) | $2 | $2 | $(13) | $(12) | $2 | $13 | $(1) | $(2) | $(6) |
| Settlement loss | $2 | $4 | $4 | $- | $- | $- | $- | $- | $- |
| Other changes in plan assets and benefit obligations recognized in other comprehensive (income) loss |
| Net actuarial (gain) loss | $(31) | $22 | $16 | $(24) | $(13) | $(83) | $(11) | $9 | $15 |
| Amortization of net actuarial (gain) loss | $(2) | $- | $- | $16 | $2 | $(25) | $1 | $1 | $2 |
| Amortization of prior service benefit | $- | $- | $- | $- | $- | $- | $1 | $1 | $1 |
| Loss due to settlement | $(2) | $(4) | $(4) | $- | $- | $- | $- | $- | $- |
| Foreign currency | $- | $- | $- | $2 | $2 | $11 | $- | $- | $- |
| Total recognized in other comprehensive (income) loss | $(35) | $18 | $12 | $(6) | $(9) | $(97) | $(9) | $11 | $18 |
| Total recognized in net periodic benefit cost (benefit) and other comprehensive (income) loss | $(31) | $24 | $3 | $(18) | $(7) | $(84) | $(10) | $9 | $12 |

## Funded Status

As of October 31, 2024 and 2023, the funded status of the defined benefit and post-retirement benefit plans was:

| U.S. Defined Benefit Plans | Non-U.S. Defined Benefit Plans | U.S. Post-Retirement Benefit Plans |
| --- | --- | --- |
| 2024 | 2023 | 2024 | 2023 | 2024 | 2023 |
| Change in fair value of plan assets: |
| Fair value — beginning of year | $359 | $396 | $791 | $748 | $76 | $85 |
| Actual return on plan assets | $88 | $(8) | $119 | $14 | $16 | $(2) |
| Employer contributions | $- | $- | $20 | $21 | $- | $- |
| Participants' contributions | $- | $- | $2 | $1 | $- | $- |
| Benefits paid | $(10) | $(8) | $(36) | $(35) | $(6) | $(7) |
| Settlements | $(23) | $(21) | $- | $- | $- | $- |
| Currency impact | $- | $- | $21 | $42 | $- | $- |
| Fair value — end of year | $414 | $359 | $917 | $791 | $86 | $76 |
| Change in benefit obligation: |
| Benefit obligation — beginning of year | $343 | $357 | $682 | $665 | $65 | $65 |
| Service cost | $- | $- | $15 | $16 | $1 | $- |
| Interest cost | $21 | $21 | $26 | $24 | $4 | $4 |
| Participants' contributions | $- | $- | $2 | $1 | $- | $- |
| Actuarial (gain) loss | $36 | $(5) | $60 | $(33) | $1 | $3 |
| Benefits paid | $(11) | $(9) | $(36) | $(35) | $(6) | $(7) |
| Settlements | $(23) | $(21) | $- | $- | $- | $- |
| Currency impact | $- | $- | $23 | $44 | $- | $- |
| Benefit obligation — end of year | $366 | $343 | $772 | $682 | $65 | $65 |
| Overfunded (underfunded) status of PBO | $48 | $16 | $145 | $109 | $21 | $11 |
| Amounts recognized in the consolidated balance sheet consist of: |
| Other assets | $51 | $19 | $236 | $174 | $21 | $11 |
| Employee compensation and benefits | $- | $(1) | $- | $- | $- | $- |
| Retirement and post-retirement benefits | $(3) | $(2) | $(91) | $(65) | $- | $- |
| Total net asset (liability) | $48 | $16 | $145 | $109 | $21 | $11 |
| Amounts Recognized in Accumulated Other Comprehensive Income (Loss): |
| Actuarial (gains) losses | $31 | $66 | $37 | $43 | $(6) | $4 |
| Prior service costs (benefits) | $- | $- | $- | $- | $(1) | $(2) |
| Total | $31 | $66 | $37 | $43 | $(7) | $2 |

## Investment Policies and Strategies as of October 31, 2024

- In the U.S., target asset allocations for our retirement and post-retirement benefit plans were approximately 50 percent to equities and approximately 50 percent to fixed income investments. Our DPSP target asset allocation is approximately 60 percent to equities and approximately 40 percent to fixed income investments.
- Outside the U.S., our target asset allocation (excluding annuity contracts in the U.K.) ranges from zero to 60 percent to equities, from 38 percent to 100 percent to fixed income investments, and from zero to 25 percent to real estate, depending on the plan.
- All plans' assets are broadly diversified.
- In December 2021 and December 2023, we entered into insurance buy-in contracts for the U.K. defined benefit plan which were funded from existing pension plan assets with no adjustment to the benefit obligations.

## Fair Value

The measurement of the fair value of pension and post-retirement plan assets uses the valuation methodologies and the inputs as described in Note 13, "Fair Value Measurements".

The following tables present the fair value of U.S. Defined Benefit Plans assets and U.S. Post-Retirement Benefit Plans assets classified under the appropriate level of the fair value hierarchy as of October 31, 2024 and 2023.

### Fair Value Measurement

#### Assets Measured at Fair Value

##### U.S. Defined Benefit Plans

| October 31, 2024 | Quoted Prices in Active Markets for Identical Assets (Level 1) | Significant Other Observable Inputs (Level 2) | Significant Unobservable Inputs (Level 3) | Not Subject to Leveling (1) |
| --- | --- | --- | --- | --- |
| Fixed Income | $138 | - | - | $174 |
| Other Investments | 1 | - | - | - |
| Total assets measured at fair value | $359 | $44 | - | $1 | $314 |

(1) Investments measured at the net asset value per share (or its equivalent) practical expedient have not been classified in the fair value hierarchy.

##### Change in Balances of Level 3 Assets

| Years Ended October 31 | 2024 | 2023 |
| --- | --- | --- |
| Balance, beginning of year | $1 | $2 |
| Realized gains/(losses) | - | - |
| Unrealized gains/(losses) | - | - |
| Purchases, sales, issuances, and settlements | - | $(1) |
| Transfers in (out) | - | - |
| Balance, end of year | $1 | $1 |

#### U.S. Post-Retirement Benefit Plans

| October 31, 2024 | Quoted Prices in Active Markets for Identical Assets (Level 1) | Significant Other Observable Inputs (Level 2) | Significant Unobservable Inputs (Level 3) | Not Subject to Leveling (1) |
| --- | --- | --- | --- | --- |
| Cash and Cash Equivalents | $1 | - | - | $1 |
| Equity | 42 | $11 | - | 31 |
| Fixed Income | 42 | - | - | 42 |
| Other Investments | 1 | - | $1 | - |
| Total assets measured at fair value | $86 | $11 | $1 | $74 |

(1) Investments measured at the net asset value per share (or its equivalent) practical expedient have not been classified in the fair value hierarchy.

##### Change in Balances of Level 3 Assets

| Years Ended October 31 | 2024 | 2023 |
| --- | --- | --- |
| Balance, beginning of year | $1 | $1 |
| Realized gains/(losses) | - | - |
| Unrealized gains/(losses) | - | - |
| Purchases, sales, issuances, and settlements | - | - |
| Transfers in (out) | - | - |
| Balance, end of year | $1 | $1 |

#### Non-U.S. Defined Benefit Plans

| October 31, 2024 | Quoted Prices in Active Markets for Identical Assets (Level 1) | Significant Other Observable Inputs (Level 2) | Significant Unobservable Inputs (Level 3) | Not Subject to Leveling (1) |
| --- | --- | --- | --- | --- |
| Cash and Cash Equivalents | $26 | $12 | - | - |
| Equity | 389 | - | - | 84 |
| Fixed Income | 352 | $159 | - | 133 |
| Annuity Contract | 150 | - | $150 | - |
| Total assets measured at fair value | $917 | $171 | $150 | $217 |

(1) Investments measured at the net asset value per share (or its equivalent) practical expedient have not been classified in the fair value hierarchy.

##### Change in Balances of Level 3 Assets

| Years Ended October 31 | 2024 | 2023 |
| --- | --- | --- |
| Balance, beginning of year | $86 | $92 |
| Unrealized gains (losses) | 4 | $(5) |
| Purchases, sales, issuances, and settlements | $(7) | $(6) |
| Transfers in (out) | 60 | - |
| Currency impact | 7 | 5 |
| Balance, end of year | $150 | $86 |

### Benefit Obligations and Fair Value of Plan Assets

| 2024 | 2023 |
| --- | --- |
| Benefit Obligation | Benefit Obligation |
| Fair Value of Plan Assets | Fair Value of Plan Assets |
| PBO | PBO |
| U.S. defined benefit plans where PBO exceeds the fair value of plan assets | $3 | $3 |
| - | - | - | - |
| U.S. defined benefit plans where fair value of plan assets exceeds PBO | $363 | $414 | $340 | $359 |
| Total | $366 | $414 | $343 | $359 |
| Non-U.S. defined benefit plans where PBO exceeds the fair value of plan assets | $249 | $157 | $197 | $132 |
| Non-U.S. defined benefit plans where fair value of plan assets exceeds PBO | $523 | $760 | $485 | $659 |
| Total | $772 | $917 | $682 | $791 |
| ABO | ABO |
| U.S. defined benefit plans where ABO exceeds the fair value of plan assets | $3 | - | $3 | - |
| U.S. defined benefit plans where the fair value of plan assets exceeds ABO | $363 | $414 | $340 | $359 |
| Total | $366 | $414 | $343 | $359 |
| Non-U.S. defined benefit plans where ABO exceeds the fair value of plan assets | $241 | $157 | $192 | $132 |
| Non-U.S. defined benefit plans where fair value of plan assets exceeds ABO | $518 | $760 | $480 | $659 |
| Total | $759 | $917 | $672 | $791 |

### Contributions and Estimated Future Benefit Payments

During fiscal year 2025, we expect to make no contributions to the U.S. defined benefit plans and the Post-Retirement Medical Plans. We expect to contribute $19 million to plans outside the U.S.

Expected Future Benefit Payments:

| | U.S. Defined Benefit Plans | Non-U.S. Defined Benefit Plans | U.S. Post-Retirement Benefit Plans |
| --- | --- | --- | --- |
| 2025 | $30 | $37 | $7 |
| 2026 | $29 | $38 | $7 |
| 2027 | $29 | $39 | $8 |
| 2028 | $31 | $40 | $8 |
| 2029 | $27 | $40 | $7 |
| 2030 - 2034 | $126 | $208 | $29 |

### Assumptions

Assumptions used to calculate the net periodic cost (benefit):

| For years ended October 31 | 2024 | 2023 | 2022 |
| --- | --- | --- | --- |
| U.S. defined benefit plans: |
| Discount rate | 6.50% | 6.00% | 2.75% |
| Expected long-term return on assets | 6.00% | 5.00% | 5.00% |
| Non-U.S. defined benefit plans: |
| Discount rate | 1.78 - 5.63% | 1.50 - 4.77% | 0.29 - 1.76% |
| Average increase in compensation levels | 2.00 - 3.25% | 2.00 - 3.25% | 2.00 - 3.50% |
| Expected long-term return on assets | 4.00 - 5.00% | 3.25 - 5.50% | 2.75 - 5.50% |
| Interest crediting rate for cash balance plans | 0.50 - 1.80% | 0.50 - 2.10% | 0.30 - 0.50% |
| U.S. post-retirement benefits plans: |
| Discount rate | 6.60% | 6.00% | 2.75% |
| Expected long-term return on assets | 6.00% | 5.00% | 5.00% |
| Current medical cost trend rate | 6.50% | 7.00% | 6.00% |
| Ultimate medical cost trend rate | 4.75% | 4.75% | 4.50% |
| Medical cost trend rate decreases to ultimate rate in year | 2029 | 2029 | 2027 |

Assumptions used to calculate the benefit obligation:

| As of the Years Ending October 31 | 2024 | 2023 |
| --- | --- | --- |
| U.S. defined benefit plans: |
| Discount rate | 5.50% | 6.50% |
| Non-U.S. defined benefit plans: |
| Discount rate | 0.95 - 5.31% | 1.78 - 5.63% |
| Average increase in compensation levels | 2.00 - 3.25% | 2.00 - 3.25% |
| Interest crediting rate for cash balance plans | 0.75 - 1.80% | 0.50 - 1.80% |
| U.S. post-retirement benefits plans: |
| Discount rate | 5.50% | 6.60% |
| Current medical cost trend rate | 6.00% | 6.50% |
| Ultimate medical cost trend rate | 4.75% | 4.75% |
| Medical cost trend rate decreases to ultimate rate in year | 2029 | 2029 |

## Restructuring and Other Related Costs

### Summary of Restructuring Plans

| | Workforce Reduction | Consolidation of Excess Facilities | Total |
| --- | --- | --- | --- |
| Balance at October 31, 2022 | - | - | - |
| Income statement expense | $33 | $13 | $46 |
| Non-cash settlements | $(1) | $(8) | $(9) |
| Cash payments | $(1) | - | $(1) |
| Balance at October 31, 2023 | $31 | $5 | $36 |
| Income statement expense | $75 | $1 | $76 |
| Non-cash settlements | $(7) | $(1) | $(8) |
| Cash payments | $(86) | $(5) | $(91) |
| Balance at October 31, 2024 | $13 | - | $13 |
| Total restructuring expense since inception of all plans | $122 |

### Fiscal Year 2024 Plan ("FY24 Plan")

| Workforce Reduction |
| --- |
| Balance at October 31, 2023 | - |
| Income statement expense | $72 |
| Non-cash settlements | $(7) |
| Cash payments | $(54) |
| Balance at October 31, 2024 | $11 |
| Total restructuring expense since inception of FY24 Plan | $72 |

### Fiscal Year 2023 Plan ("FY23 Plan")

| | Workforce Reduction | Consolidation of Excess Facilities | Total |
| --- | --- | --- | --- |
| Balance at October 31, 2022 | - | - | - |
| Income statement expense | $33 | $13 | $46 |
| Non-cash settlements | $(1) | $(8) | $(9) |
| Cash payments | $(1) | - | $(1) |
| Balance at October 31, 2023 | $31 | $5 | $36 |
| Income statement expense | $3 | $1 | $4 |
| Non-cash settlements | - | $(1) | $(1) |
| Cash payments | $(32) | $(5) | $(37) |
| Balance at October 31, 2024 | $2 | - | $2 |
| Total restructuring expense since inception of the FY23 Plan | $50 |

## Guarantees

### Standard Warranty

We accrue for standard warranty costs based on historical trends in actual warranty charges over the past 12 months. The accrual i


### Standard Warranty
- We accrue for standard warranty costs based on historical trends in actual warranty charges over the past 12 months.
- The standard warranty accrual balances are held in other accrued and other long-term liabilities on our consolidated balance sheet.
- Our standard warranty terms typically extend to one year from the date of delivery, depending on the product.

| October 31, | 2024 | 2023 |
| --- | --- | --- |
| (in millions) |  |  |
| Standard warranty accrual, beginning balance | $29 | $30 |
| Accruals for warranties including change in estimates | 58 | 57 |
| Settlements made during the period | $(57) | $(58) |
| Standard warranty accrual, ending balance | $30 | $29 |
| Accruals for warranties due within one year | $30 | $29 |

### Bank Guarantees
- Guarantees consist primarily of outstanding standby letters of credit and bank guarantees.
- As of October 31, 2024 and 2023, the guarantees were approximately $37 million and $39 million, respectively.

### Indemnifications in Connection with Transactions
- We have agreed to indemnify certain parties, their affiliates and/or other related parties against certain damages and expenses that might occur in the future.
- These indemnifications may cover a variety of liabilities, including, but not limited to, employee, tax, environmental, intellectual property, litigation and other liabilities related to the business conducted prior to the date of the transaction.

### Indemnifications to Officers and Directors
- Our corporate bylaws require that we indemnify our officers and directors, as well as those who act as directors and officers of other entities at our request, against expenses, judgments, fines, settlements and other amounts actually and reasonably incurred in connection with any proceedings arising out of their services.
- We purchase standard insurance to cover claims or a portion of the claims made against our directors and officers.

### Other Indemnifications
- Many of our standard contracts provide remedies to our customers and others with whom we enter into contracts, such as defense, settlement, or payment of judgment for intellectual property claims related to the use of our products.
- We also provide protection to these parties against claims related to undiscovered liabilities, additional product liability or environmental obligations.

## 18. Commitments and Contingencies

### Other Purchase Commitments
- Our contractual obligations with suppliers under "other purchase commitments" were approximately $136 million.

### Contingencies
- We are involved in lawsuits, claims, investigations and proceedings, including, but not limited to, intellectual property, commercial, real estate, environmental and employment matters, which arise in the ordinary course of business.
- There are no matters pending that we currently believe are reasonably possible of having a material impact to our business, consolidated financial condition, results of operations or cash flows.

## 19. Short-Term Debt

### Credit Facilities
- On June 7, 2023, we entered into a new credit agreement with a group of financial institutions which provides for a $1.5 billion five-year unsecured credit facility that will expire on June 7, 2028, and an incremental revolving credit facility in an aggregate amount of up to $750 million.
- On June 2, 2023, we entered into an Uncommitted Money Market Line Credit agreement with Societe Generale which provides for an aggregate borrowing capacity of $300 million.

### Commercial Paper
- Under our U.S. commercial paper program, we may issue and sell unsecured, short-term promissory notes in the aggregate principal amount not to exceed $1.5 billion with up to 397-day maturities.
- As of October 31, 2024, we had borrowings of $40 million outstanding under our U.S. commercial paper program and had a weighted average annual interest rate of 4.92 percent.

### Other Loans
- In September 2024, we completed the BIOVECTRA acquisition and assumed two interest-free loans from the Strategic Innovation Fund ("SIF") and two interest-free loans with the Atlantic Canada Opportunities Agency ("ACOA").

## 20. Long-Term Debt

### Term Loan Facility
- On April 15, 2022, we entered into a term loan agreement with a group of financial institutions, which provided for a $600 million delayed draw term loan that will mature on April 15, 2025.
- During the year ended October 31, 2024, we repaid in full the outstanding $600 million principal amount of our term loan facility.

### Other Loans
- In September 2024, we completed the BIOVECTRA acquisition and assumed two interest-free loans from the Strategic Innovation Fund ("SIF") and two interest-free loans with the Atlantic Canada Opportunities Agency ("ACOA").

### Senior Notes
- The following table summarizes the company's long-term senior notes:

| October 31, 2024 | October 31, 2023 |
| --- | --- |
| (in millions) |  |
| 2026 Senior Notes | $299 | $299 |
| 2027 Senior Notes | 596 | - |
| 2029 Senior Notes | 496 | 496 |
| 2030 Senior Notes | 497 | 496 |
| 2031 Senior Notes | 845 | 844 |
| 2034 Senior Notes | 593 | - |
| Total Senior Notes | $3,326 | $2,135 |


In February 2016, we executed three forward-starting pay fixed/receive variable interest rate swaps for the notional amount of $300 million in connection with future interest payments to be made on our 2026 senior notes issued on September 15, 2016. The swap arrangements were terminated on September 15, 2016 with a payment of $10 million, and we recognized this as a deferred loss in accumulated other comprehensive income (loss) which is being amortized to interest expense over the life of the 2026 senior notes. The remaining loss to be amortized related to the interest rate swap agreements at October 31, 2024 was $2 million.

## 2027 Senior Notes

On September 9, 2024, we issued an aggregate principal amount of $600 million in senior notes ("2027 senior notes"). The 2027 senior notes were issued at 99.866% of their principal amount. The notes will mature on September 9, 2027, and bear interest at a fixed rate of 4.20% per annum. The interest is payable semi-annually on March 9th and September 9th of each year and payments will commence on March 9, 2025.

## 2029 Senior Notes

On September 16, 2019, we issued an aggregate principal amount of $500 million in senior notes ("2029 senior notes"). The 2029 senior notes were issued at 99.316% of their principal amount. The notes will mature on September 15, 2029, and bear interest at a fixed rate of 2.75% per annum. The interest is payable semi-annually on March 15th and September 15th of each year and payments commenced on March 15, 2020.

In August 2019, we executed treasury lock agreements for $250 million in connection with future interest payments to be made on our 2029 senior notes issued on September 16, 2019. We designated the treasury lock as a cash flow hedge. The treasury lock contracts were terminated on September 6, 2019 and we recognized a deferred loss of $6 million in accumulated other comprehensive income which is being amortized to interest expense over the life of the 2029 senior notes. The remaining loss to be amortized related to the treasury lock agreements at October 31, 2024 was $3 million.

## 2030 Senior Notes

On June 4, 2020, we issued an aggregate principal amount of $500 million in senior notes ("2030 senior notes"). The 2030 senior notes were issued at 99.812% of their principal amount. The 2030 senior notes will mature on June 4, 2030, and bear interest at a fixed rate of 2.10% per annum. The interest is payable semi-annually on June 4th and December 4th of each year and payments commenced on December 4, 2020.

## 2031 Senior Notes

On March 12, 2021, we issued an aggregate principal amount of $850 million in senior notes ("2031 senior notes"). The 2031 senior notes were issued at 99.822% of their principal amount. The 2031 senior notes will mature on March 12, 2031, and bear interest at a fixed rate of 2.30% per annum. The interest is payable semi-annually on March 12th and September 12th of each year and payments commenced on September 12, 2021.

## 2034 Senior Notes

On September 9, 2024, we issued an aggregate principal amount of $600 million in senior notes ("2034 senior notes"). The 2034 senior notes were issued at 99.638% of their principal amount. The 2034 senior notes will mature on September 9, 2034, and bear interest at a fixed rate of 4.75% per annum. The interest is payable semi-annually on March 9th and September 9th of each year and payments will commence on March 9, 2025.

All outstanding senior notes listed above are unsecured and rank equally in right of payment with all of our other senior unsecured indebtedness.

## Stockholders' Equity

### Stock Repurchase Program

On February 16, 2021 we announced that our board of directors had approved a share repurchase program (the "2021 repurchase program") designed, among other things, to reduce or eliminate dilution resulting from issuance of stock under the company's employee equity incentive programs. The 2021 repurchase program which commenced on February 18, 2021, authorized the purchase of up to $2.0 billion, excluding excise taxes, of our common stock at the company's discretion and had no fixed termination date. The 2021 repurchase program did not require the company to acquire a specific number of shares and could be suspended, amended or discontinued at any time. During the year ended October 31, 2022, we repurchased and retired 8.4 million shares for $1,139 million under this authorization. During the year ended October 31, 2023, we repurchased and retired 661,739 shares for $99 million, excluding excise taxes, under this authorization. On March 1, 2023, the 2021 repurchase program was terminated and the remaining authorization of $339 million expired.

On January 9, 2023, we announced that our board of directors had approved a share repurchase program (the "2023 repurchase program") designed, among other things, to reduce or eliminate dilution resulting from issuance of stock under the company's employee equity incentive programs. The 2023 repurchase program authorizes the purchase of up to $2.0 billion, excluding excise taxes, of our common stock at the company's discretion and has no fixed termination date. The 2023 repurchase program does not require the company to acquire a specific number of shares and may be suspended, amended or discontinued at any time. The 2023 repurchase program commenced on March 1, 2023, and also terminated and replaced the 2021 repurchase program. During the year ended October 31, 2023, we repurchased and retired 3.9 million shares for $476 million, excluding excise taxes, under this authorization. During the year ended October 31, 2024 we repurchased and retired 8.4 million shares for $1,150 million, excluding excise taxes, under this authorization. As of October 31, 2024, we had remaining authorization to repurchase up to approximately $374 million of our common stock under the 2023 repurchase program.

On May 29, 2024, we announced that our board of directors had approved a new share repurchase program (the "2024 repurchase program") designed, among other things, to reduce or eliminate dilution resulting from issuance of stock under the company's employee equity incentive programs. The 2024 repurchase program authorizes the purchase of up to $2.0 billion, excluding excise taxes, of our common stock at the company's discretion and has no fixed termination date. The 2024 repurchase program does not require the company to acquire a specific number of shares and may be suspended, amended or discontinued at any time. The 2024 repurchase program became effective on August 1, 2024 and will commence upon the termination of our 2023 repurchase program.

The Inflation Reduction Act of 2022, which was enacted into law on August 16, 2022, imposed a nondeductible 1% excise tax on the net value of certain stock repurchases made after December 31, 2022. During the year ended October 31, 2024, we recorded the applicable excise taxes payable of approximately $10 million as an incremental cost of the shares repurchased and a corresponding liability for the excise tax payable in other accrued liabilities on our consolidated balance sheet. In fiscal year 2023, we recorded excise taxes payable of approximately $3 million related to shares repurchased in 2023 and paid the tax in 2024.

### Cash Dividends on Shares of Common Stock

During the year ended October 31, 2024, cash dividends of $0.944 per share, or $274 million were declared and paid on the company's outstanding common stock. During the year ended October 31, 2023, cash dividends of $0.900 per share, or $265 million were declared and paid on the company's outstanding common stock. During the year ended October 31, 2022, cash dividends of $0.840 per share, or $250 million were declared and paid on the company's outstanding common stock.

On November 20, 2024, we declared a quarterly dividend of $0.248 per share of common stock, or approximately $71 million which will be paid on January 22, 2025, to shareholders of record as of the close of business on December 31, 2024. The timing and amounts of any future dividends are subject to determination and approval by our board of directors.

### Accumulated Other Comprehensive Income (Loss)

Changes in accumulated other comprehensive income (loss) by component and related tax effects for the years ended October 31, 2024 and 2023 were as follows:

| | Net defined benefit pension cost and post retirement plan costs | Foreign currency translation | Prior service credits | Actuarial Losses | Unrealized gains (losses) on derivatives | Total |
| --- | --- | --- | --- | --- | --- | --- |
| As of October 31, 2022 | $(335) | $123 | $(155) | $20 | $(347) | - |
| Other comprehensive income (loss) before reclassifications | 33 | - | (17) | (4) | 12 | - |
| Amounts reclassified out of accumulated other comprehensive income (loss) | - | (1) | 2 | - | 1 | - |
| Tax (expense) benefit | 1 | - | 5 | 1 | 7 | - |
| Other comprehensive income (loss) | 34 | (1) | (10) | (3) | 20 | - |
| As of October 31, 2023 | $(301) | $122 | $(165) | $17 | $(327) | - |
| Other comprehensive income (loss) before reclassifications | (11) | - | 65 | (9) | 45 | - |
| Amounts reclassified out of accumulated other comprehensive income (loss) | (8) | (1) | (12) | (2) | (23) | - |
| Tax (expense) benefit | (3) | - | - | 3 | - | - |
| Other comprehensive income (loss) | (22) | (1) | 53 | (8) | 22 | - |
| As of October 31, 2024 | $(323) | $121 | $(112) | $9 | $(305) | - |

Reclassifications out of accumulated other comprehensive income (loss) for the years ended October 31, 2024 and 2023 were as follows (in millions):

| Details about Accumulated Other Comprehensive Income (Loss) components | Amounts Reclassified from Other Comprehensive Income (Loss) |  | Affected line item in statement of operations |
| --- | --- | --- | --- |
|  | 2024 | 2023 |  |
| Foreign currency translation | $8 | $- | Other income (expense), net |
| Unrealized gain (loss) on derivatives | $4 | $2 | Cost of products |
| Unrealized gain (loss) on derivatives | $(2) | $(2) | Interest expense |
| Net defined benefit pension cost and post retirement plan costs: |  |  |  |
| Actuarial net gain (loss) | 12 | $(2) | Other income (expense) |
| Prior service benefit | 1 | 1 | Other income (expense) |
| Total reclassifications for the period | $18 | $(1) | - |

Amounts in parentheses indicate reductions to income and increases to other comprehensive income.

Reclassifications out of accumulated other comprehensive income (loss) of actuarial net gain (loss) and prior service benefit in respect of retirement plans and post retirement pension plans are included in the computation of net periodic benefit cost (income) (see Note 15, "Retirement Plans and Post Retirement Pension Plans").

### 23. SUBSEQUENT EVENT

#### New Segment Structure

In November 2024, we announced a change in our organizational structure to support our market-focused, customer-centric strategy. Our diagnostics and genomics segment combined with our liquid chromatography and mass spectrometry instrument platforms will form our new life sciences and diagnostics markets segment. Our chemistries and supplies, laboratory automation and robotics, and software and informatics divisions moved from our life sciences and applied markets segment to our Agilent CrossLab segment. The remaining divisions in our life sciences and applied markets segment which includes our gas chromatography, gas chromatography mass spectrometry, remarketed instruments, spectroscopy and vacuum divisions will form our new applied markets segment.

Following this re-organization, we will have three businesses - life sciences and diagnostics markets, applied markets and Agilent CrossLab - each of which will comprise a reportable segment. All historical segment financial information will be recast to conform to this new reporting structure in our financial statements and accompanying notes, beginning with our Form 10-Q filing for the first quarter of fiscal year 2025.


On September 20, 2024, BIOVECTRA was excluded from Agilent's assessment of the effectiveness of the company's internal control over financial reporting as of October 31, 2024. BIOVECTRA constituted less than 1 percent of total revenue for the period ending October 31, 2024 and 3 percent of total assets, excluding acquired goodwill and other intangible assets, as of October 31, 2024.

The effectiveness of Agilent's internal control over financial reporting as of October 31, 2024 has been audited by PricewaterhouseCoopers LLP, an independent registered public accounting firm, as stated in their report which appears in Item 8 of this Annual Report on Form 10-K.

### Changes in Internal Control Over Financial Reporting

There were no changes in Agilent's internal control over financial reporting that occurred during the last fiscal quarter that have materially affected, or are reasonably likely to materially affect, the company's internal control over financial reporting.

### Other Information

#### Rule 10b5-1 Trading Arrangements

During the three months ended October 31, 2024, none of Agilent's officers or directors adopted, modified or terminated a "Rule 10b5-1 trading arrangement" or "non-Rule 10b5-1 trading arrangement" as each term is defined in Item 408 of Regulation S-K.

#### Disclosure Regarding Foreign Jurisdictions that Prevent Inspections

Not applicable.

## Part III

### Item 10. Directors, Executive Officers and Corporate Governance

Information regarding Agilent's directors, executive officers, Audit and Finance Committee, and code of ethics is incorporated by reference from the company's Proxy Statement for the Annual Meeting of Stockholders to be held on March 13, 2025.

### Item 11. Executive Compensation

Information about compensation of Agilent's named executive officers and directors is incorporated by reference from the Proxy Statement.

### Item 12. Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters

Information about security ownership of certain beneficial owners and management is incorporated by reference from the Proxy Statement.

### Equity Compensation Plan Information

As of October 31, 2024, Agilent had 3,140,791 securities to be issued upon exercise of outstanding options, warrants and rights, with a weighted-average exercise price of $134, and 40,911,061 securities remaining available for future issuance under equity compensation plans approved by security holders.

### Item 13. Certain Relationships and Related Transactions, and Director Independence

Information about certain relationships and related transactions, and director independence, is incorporated by reference from the Proxy Statement.

### Item 14. Principal Accounting Fees and Services

Information about principal accountant fees and services, as well as related pre-approval policies, is incorporated by reference from the Proxy Statement.

## Part IV

### Item 15. Exhibits and Financial Statement Schedules

1. Financial Statements
2. Financial Statement Schedule
3. Exhibits


### Exhibit Number: 10.18
**Description:** Form of Stock Award Agreement under the 2018 Stock Plan for Long-Term Performance Program Awards (for awards made after November 13, 2018).*
**Form:** 10-K
**Date:** 12/20/2018
**Exhibit Number:** 10.18
**Filed Herewith:** 10.19

### Exhibit Number: 10.19
**Description:** Form of Stock Award Agreement under the 2018 Stock Plan for Standard Awards granted to Employees (for awards made after November 14, 2023)*
**Form:** 10-K
**Date:** 12/20/2023
**Exhibit Number:** 10.19
**Filed Herewith:** 10.20

### Exhibit Number: 10.20
**Description:** Form of Stock Option Award Agreement under the 2018 Stock Plan for non-U.S. Employees (for awards made after November 14, 2023)*
**Form:** 10-K
**Date:** 12/20/2023
**Exhibit Number:** 10.20
**Filed Herewith:** 10.21

### Exhibit Number: 10.21
**Description:** Form of Stock Award Agreement under the 2018 Stock Plan for Long-Term Performance Program Awards (for awards made after November 14, 2023)*
**Form:** 10-K
**Date:** 12/20/2023
**Exhibit Number:** 10.21
**Filed Herewith:** 10.22

### Exhibit Number: 10.22
**Description:** Form of Stock Award Agreement under the 2018 Stock Plan for Retention Awards granted to Employees (for awards made on or after November 14, 2023)*
**Form:** 10-K
**Date:** 12/20/2023
**Exhibit Number:** 10.22
**Filed Herewith:** 125

### Exhibit Number: 10.23
**Description:** Agilent Technologies, Inc. Supplemental Benefit Retirement Plan (Amended and Restated Effective May 20, 2014).*
**Form:** 10-K
**Date:** 12/21/2017
**Exhibit Number:** 10.17
**Filed Herewith:** 

### Exhibit Number: 10.24
**Description:** Agilent Technologies, Inc. Long-Term Performance Program (Amended and Restated through November 1, 2005).*
**Form:** 10-Q
**Date:** 3/9/2006
**Exhibit Number:** 10.63
**Filed Herewith:** 

### Exhibit Number: 10.25
**Description:** Agilent Technologies, Inc. 2005 Deferred Compensation Plan for Non-Employee Directors (Amended and Restated Effective November 18, 2009).*
**Form:** 10-K
**Date:** 12/21/2009
**Exhibit Number:** 10.39
**Filed Herewith:** 

### Exhibit Number: 10.26
**Description:** Agilent Technologies, Inc. 2005 Deferred Compensation Plan (Amended and Restated Effective May 20, 2014).*
**Form:** 10-K
**Date:** 12/21/2017
**Exhibit Number:** 10.20
**Filed Herewith:** 

### Exhibit Number: 10.27
**Description:** Agilent Technologies, Inc. 2010 Performance‑Based Compensation Plan for Covered Employees. (as adopted on November 19. 2014)
**Form:** DEF14A
**Date:** 2/6/2015
**Exhibit Number:** Annex A
**Filed Herewith:** 

### Exhibit Number: 10.28
**Description:** Form of Amended and Restated Indemnification Agreement between Agilent Technologies, Inc. and Directors of the Company, Section 16 Officers and Board‑elected Officers of the Company.*
**Form:** 8-K
**Date:** 4/10/2008
**Exhibit Number:** 10.1
**Filed Herewith:** 

### Exhibit Number: 10.29
**Description:** Form of Tier I Change of Control Severance Agreement between Agilent Technologies, Inc. and the Chief Executive Officer*
**Form:** 10-K
**Date:** 12/22/2014
**Exhibit Number:** 10.35
**Filed Herewith:** 

### Exhibit Number: 10.30
**Description:** Form of Amended and Restated Change of Control Severance Agreement between Agilent Technologies, Inc. and Section 16 Officers (other than the Company's Chief Executive Officer).*
**Form:** 8-K
**Date:** 4/10/2008
**Exhibit Number:** 10.3
**Filed Herewith:** 

### Exhibit Number: 10.31
**Description:** Form of Tier II Change of Control Severance Agreement between Agilent Technologies, Inc. and Section 16 Officers (other than the Company's Chief Executive Officer)*
**Form:** 10-K
**Date:** 12/22/2014
**Exhibit Number:** 10.37
**Filed Herewith:** 

### Exhibit Number: 10.32
**Description:** Form of New Executive Officer Change of Control Severance Agreement between Agilent Technologies, Inc. and specified executives of the Company (for executives hired, elected or promoted after July 14, 2009).*
**Form:** 10-K
**Date:** 12/21/2009
**Exhibit Number:** 10.5
**Filed Herewith:** 

### Exhibit Number: 10.33
**Description:** Form of Tier III Change of Control Severance Agreement between Agilent Technologies, Inc. and specified executives of the Company*
**Form:** 10-K
**Date:** 12/22/2014
**Exhibit Number:** 10.39
**Filed Herewith:** 

### Exhibit Number: 10.34
**Description:** Tax Matters Agreement, dated August 1, 2014, by and between Agilent Technologies, Inc. and Keysight Technologies, Inc.
**Form:** 8-K
**Date:** 8/5/2014
**Exhibit Number:** 10.1
**Filed Herewith:** 

### Exhibit Number: 10.35
**Description:** Employee Matters Agreement, dated August 1, 2014, by and between Agilent Technologies, Inc. and Keysight Technologies, Inc.
**Form:** 8-K
**Date:** 8/5/2014
**Exhibit Number:** 10.2
**Filed Herewith:** 

### Exhibit Number: 10.36
**Description:** Intellectual Property Matters Agreement, dated August 1, 2014, by and between Agilent Technologies, Inc. and Keysight Technologies, Inc.
**Form:** 8-K
**Date:** 8/5/2014
**Exhibit Number:** 10.3
**Filed Herewith:** 

### Exhibit Number: 10.37
**Description:** Trademark License Agreement, dated August 1, 2014, by and between Agilent Technologies, Inc. and Keysight Technologies, Inc.
**Form:** 8-K
**Date:** 8/5/2014
**Exhibit Number:** 10.4
**Filed Herewith:** 

### Exhibit Number: 10.38
**Description:** Real Estate Matters Agreement, dated August 1, 2014, by and between Agilent Technologies, Inc. and Keysight Technologies, Inc.
**Form:** 8-K
**Date:** 8/5/2014
**Exhibit Number:** 10.5
**Filed Herewith:** 

### Exhibit Number: 10.39
**Description:** Credit Agreement, dated June 7, 2023, by and among the Company, the Lenders party thereto and BNP Paribas, as Administrative Agent.
**Form:** 8-K
**Date:** 6/13/2023
**Exhibit Number:** 10.1
**Filed Herewith:** 126

### Exhibit Number: 10.40
**Description:** Incremental Assumption Agreement dated as of April 21, 2021, by and among the Company, the Lenders party thereto and BNP Paribas, as Administrative Agent
**Form:** 8-K
**Date:** 4/22/2021
**Exhibit Number:** 10.1
**Filed Herewith:** 

### Exhibit Number: 10.41
**Description:** Term Loan Agreement, dated as of April 15, 2022, among the Company, the lenders party thereto, Wells Fargo Bank, National Association, as administrative agent.
**Form:** 8-K
**Date:** 4/19/2022
**Exhibit Number:** 10.1
**Filed Herewith:** 

### Exhibit Number: 10.42
**Description:** Letter of Terms and Conditions of U.S. Indefinite Relocation and U.S. Domestic Relocation Agreement, each by and among Michael R. McMullen and the Company*
**Form:** 10-Q
**Date:** 3/8/2016
**Exhibit Number:** 10.1
**Filed Herewith:** 

### Exhibit Number: 10.43
**Description:** Letter of Terms and Conditions of U.S. Indefinite Relocation and U.S. Domestic Relocation Agreement, each by and among Robert McMahon and the Company*
**Form:** 10-K
**Date:** 12/20/2018
**Exhibit Number:** 10.41
**Filed Herewith:** 

### Exhibit Number: 10.44
**Description:** Letter of Terms and Conditions Localization Program by and among Padraig McDonnell and the Company*
**Form:** 10-Q
**Date:** 6/1/2020
**Exhibit Number:** 10.2
**Filed Herewith:** 

### Exhibit Number: 10.45
**Description:** Agilent Technologies, Inc. Excess Benefit Retirement Plan (Amended and Restated Effective May 20, 2014)*
**Form:** 10-K
**Date:** 12/21/2017
**Exhibit Number:** 10.4
**Filed Herewith:** 

### Exhibit Number: 19.1
**Description:** Insider Trading Policy
**Filed Herewith:** X

### Exhibit Number: 21.1
**Description:** Significant subsidiaries of Agilent Technologies, Inc. as of October 31, 2024.
**Filed Herewith:** X

### Exhibit Number: 23.1
**Description:** Consent of Independent Registered Public Accounting Firm.
**Filed Herewith:** X

### Exhibit Number: 24.1
**Description:** Powers of Attorney. Contained in the signature page of this Annual Report on Form 10-K.
**Filed Herewith:** X

### Exhibit Number: 31.1
**Description:** Certification of Chief Executive Officer pursuant to Section 302 of the Sarbanes‑Oxley Act of 2002.
**Filed Herewith:** X

### Exhibit Number: 31.2
**Description:** Certification of Chief Financial Officer pursuant to Section 302 of the Sarbanes‑Oxley Act of 2002.
**Filed Herewith:** X

### Exhibit Number: 32.1
**Description:** Certification of Chief Executive Officer pursuant to Section 906 of the Sarbanes‑Oxley Act of 2002.
**Filed Herewith:** X

### Exhibit Number: 32.2
**Description:** Certification of Chief Financial Officer pursuant to Section 906 of the Sarbanes‑Oxley Act of 2002.
**Filed Herewith:** X

### Exhibit Number: 97.1
**Description:** Agilent Technologies, Inc. Executive Compensation Clawback Policy
**Filed Herewith:** X

### Exhibit Number: 101.INS
**Description:** XBRL Instance Document - the instance document does not appear in the Interactive Data File because its XBRL tags are embedded within the Inline XBRL document.
**Filed Herewith:** X

### Exhibit Number: 101.SCH
**Description:** XBRL Taxonomy Extension Schema Document.
**Filed Herewith:** X

### Exhibit Number: 101.CAL
**Description:** XBRL Taxonomy Extension Calculation Linkbase Document.
**Filed Herewith:** X

### Exhibit Number: 101.LAB
**Description:** XBRL Taxonomy Extension Label Linkbase Document.
**Filed Herewith:** X

### Exhibit Number: 101.PRE
**Description:** XBRL Taxonomy Extension Presentation Linkbase Document.
**Filed Herewith:** X

### Exhibit Number: 101.DEF
**Description:** XBRL Taxonomy Extension Definition Linkbase Document.
**Filed Herewith:** X

*    Indicates management contract or compensatory plan, contract or arrangement.

## Item 16. Form 10-K Summary

None.

## SIGNATURES

Pursuant to the requirements of Section 13 or 15(d) of the Securities Exchange Act of 1934, the registrant has duly caused this report to be signed on its behalf by the undersigned, thereunto duly authorized.

AGILENT TECHNOLOGIES, INC.

BY
/s/  PADRAIG MCDONNELL
Padraig McDonnell
President and Chief Executive Officer
Date: December 19, 2024

## POWER OF ATTORNEY

KNOW ALL PERSONS BY THESE PRESENTS, that each person whose signature appears below constitutes and appoints Bret DiMarco and P. Diana Chiu, or either of them, his or her attorneys-in-fact, for such person in any and all capacities, to sign any amendments to this report and to file the same, with exhibits thereto, and other documents in connection therewith, with the Securities and Exchange Commission, hereby ratifying and confirming all that any of said attorneys-in-fact, or substitute or substitutes, may do or cause to be done by virtue hereof. Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant and in the capacities and on the dates indicated.

Signature | Title | Date
--- | --- | ---
/s/ PADRAIG MCDONNELL | Director, President and Chief Executive Officer (Principal Executive Officer) | December 19, 2024
/s/ ROBERT W. MCMAHON | Senior Vice President and Chief Financial Officer (Principal Financial Officer) | December 19, 2024
/s/ RODNEY GONSALVES | Vice President, Corporate Controllership (Principal Accounting Officer) | December 19, 2024
/s/ KOH BOON HWEE | Chairman of the Board of Directors | December 19, 2024
/s/ MALA ANAND | Director | December 19, 2024
/s/ OTIS W. BRAWLEY, M.D. | Director | December 19, 2024
/s/ G. MIKAEL DOLSTEN, M.D., PH.D. | Director | December 19, 2024
/s/ HEIDI KUNZ | Director | December 19, 2024
/s/ DANIEL K. PODOLSKY, M.D. | Director | December 19, 2024
/s/ SUE H. RATAJ | Director | December 19, 2024
/s/ GEORGE A. SCANGOS, Ph.D. | Director | December 19, 2024
/s/ DOW R. WILSON | Director | December 19, 2024

In [56]:
class BedrockAIClient:
    def __init__(self, model_id: str = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"):
        self.bedrock_client = boto3.client('bedrock-runtime')
        self.model_id = model_id
        print(f"Using model: {self.model_id}")
    
    def generate_response(self, prompt: str, max_tokens: int = 10000) -> str:
        try:
            body = {
                "anthropic_version": "bedrock-2023-05-31",
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": max_tokens,
                "temperature": 0.1
            }
            
            response = self.bedrock_client.invoke_model(
                modelId=self.model_id,
                body=json.dumps(body)
            )
            
            response_body = json.loads(response['body'].read())
            
            if 'choices' in response_body and len(response_body['choices']) > 0:
                print(1)
                return response_body['choices'][0]['message']['content'].strip()
            else:
                return response_body
                
        except Exception as e:
            return f"Error: Unable to generate response - {str(e)}"

ai_client = BedrockAIClient()

test_response = ai_client.generate_response('Bonjour tu vas bien ?')
print(f"Test response: {test_response['content'][0]['text']}...")

Using model: global.anthropic.claude-sonnet-4-5-20250929-v1:0
Test response: Bonjour ! Je vais bien, merci de demander ! 😊

Comment allez-vous ? Que puis-je faire pour vous aider aujourd'hui ?...


In [18]:
# Enregistre le text en MD dans S3 Bucket (peux etre utiliser pour faire le RAG)

folder_uri = "s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/UserDataUpload/"
filename = f"rapport_10K_{datetime.now():%Y%m%d_%H%M%S}.md"
# --- Extraire bucket et prefix à partir de l’URI ---
parsed = urlparse(folder_uri)
bucket_name = parsed.netloc
prefix = parsed.path.lstrip("/")  # "notes/projets/"

# --- Construire le chemin complet du fichier ---
file_key = f"{prefix.rstrip('/')}/{filename}"

# --- Upload vers S3 ---
s3 = boto3.client("s3")
s3.put_object(
    Bucket=bucket_name,
    Key=file_key,
    Body=text_in_english_a.encode("utf-8"),
    ContentType="text/markdown"
)

print(f"✅ Fichier ajouté à : s3://{bucket_name}/{file_key}")


✅ Fichier ajouté à : s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/UserDataUpload/rapport_10K_20251102_161637.md


None


In [22]:
import boto3
import pandas as pd
import numpy as np
import logging
import sagemaker
import mlflow
import os
from datetime import datetime, timezone
from sagemaker.modules import Session
from sagemaker_studio import Project

project = Project()
session = boto3.Session()
bucket_root = project.s3.root
role = project.iam_role
s3_parts = bucket_root.replace("s3://", "").split("/")
bucket = s3_parts[0]
prefix = "/".join(s3_parts[1:])
print(f"Using Bucket: {bucket}")
print(f"Using prefix: {prefix}")
print(f"Using Role: {role}")

catalog = project.connection().catalog()
id = 0
for db in catalog.databases:
    print(f"Index {id}: {db}")
    id += 1


Using Bucket: csv-file-store-6d12f630
Using prefix: dzd-4giwkqu9tkh07k/alsdvbccweepsg/dev
Using Role: arn:aws:iam::851725179216:role/datazone_usr_role_alsdvbccweepsg_cr5d9nlcbub7o0
Index 0: Database(name='salesmarketing', catalog_id='851725179216', location_uri='s3://csv-file-store-6d12f630/dzd-4giwkqu9tkh07k/alsdvbccweepsg/dev/data/catalogs/')
